In [5]:
%%file reprophylo.py
reprophyloversion=1.3
############################################################################################
if False:
    """
    ReproPhylo version 1 
    
    General purpose phylogenetics package for reproducible and experimental analysis
    
    Amir Szitenebrg
    A.Szitenberg@Hull.ac.uk
    Szitenberg@gmail.com
    
    David H Lunt
    D.H.Lunt@Hull.ac.uk
    
    EvoHull.org
    University of Hull
    
    
    Developed with:
    CPython 2.7.6
    IPython 1.2.1
    ete2 2.2rev1056
    biopython 1.64
    dendropy 3.12.0
    cloud 2.8.5
    numpy 1.8.2
    matplotlib 1.3.1
    pandas
    
    RAxML 8
    Phylobayes 3
    Trimal 1
    Muscle 
    Mafft 7
    Pal2nal 14
    """
##############################################################################################



from Bio import SeqIO
import os, csv, sys, dendropy, re, time, random, glob, platform, warnings, rpgit, ast, gb_syn,css
import HTML, inspect, shutil
import subprocess as sub
from Bio.Seq import Seq
import numpy as np
import matplotlib.pyplot as plt
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation, CompoundLocation
from Bio.Align.Applications import MafftCommandline, MuscleCommandline
from StringIO import StringIO 
from Bio import AlignIO 
from Bio.Phylo.Applications import RaxmlCommandline
from Bio.Align import MultipleSeqAlignment
from Bio.SeqUtils import GC
from ete2 import *
from collections import Counter
#import pandas as pd
import math
import __builtin__




##############################################################################################
class Locus:
##############################################################################################

    """ Configure the loci stored in the ReproPhylo Project.
        
    >>> locus = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
    >>> print(locus)
    Locus(char_type=dna, feature_type=CDS, name=coi, aliases=cox1; COX1; coi; COI; CoI)
    """

    char_type = 'NotSet'
    feature_type = 'NotSet'
    name = 'NotSet'
    aliases = []

    def __init__(self, char_type=char_type, feature_type=feature_type,
                 name=name, aliases=aliases):

        
        self.char_type = char_type
        self.feature_type = feature_type
        self.name = name
        self.aliases = aliases
        
        valid = ['dna','prot']
        if not self.char_type in valid:
            raise ValueError('self.char_type should be \'dna\' or \'prot\'')
        if not type(self.feature_type) is str:
            raise ValueError('self.feature_type should be a string')
        if not type(self.name) is str:
            raise ValueError('self.name should be a string')
        if not type(self.aliases) is list:
            raise ValueError('self.aliases should be a list')
        else:
            for a in self.aliases:
                if not type(a) is str:
                    raise ValueError('aliases in self.aliases have to be strings')
            


    def __str__(self):
        aliases_str = ('; ').join(self.aliases)
        return ('Locus(char_type='+self.char_type+', feature_type='+self.feature_type+
                ', name='+self.name+', aliases='+aliases_str+')')

    
    
##############################################################################################
class Concatenation:
##############################################################################################

    """This class is used to configure concatenations given loci and rules.
    
    >>> coi = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
    >>> ssu = Locus('dna', 'rRNA', '18S', ['18S rRNA','SSU rRNA'])
    >>> bssu = Locus('dna', 'rRNA', '16S', ['16S rRNA'])
    >>> lsu = Locus('dna', 'rRNA', '28S', ['28S rRNA', 'LSU rRNA'])
    >>> alg11 = Locus('dna', 'CDS', 'ALG11', ['ALG11'])
    >>> loci = [coi, ssu, bssu, lsu, alg11]
    >>> concatenation = Concatenation(name='combined', loci=loci,
    ...                                otu_meta='OTU_name',
    ...                                otu_must_have_all_of=['coi'],
    ...                                otu_must_have_one_of =[['16S','28S'],['ALG11','18S']],
    ...                                define_trimmed_alns=["MuscleDefaults@dummyTrimMethod"])
    >>> print(str(concatenation))
    Concatenation named combined, with loci coi,18S,16S,28S,ALG11,
    of which coi must exist for all species
    and at least one of each group of [ 16S 28S ][ ALG11 18S ] is represented.
    Alignments with the following names: MuscleDefaults@dummyTrimMethod are prefered
    """
    
    otu_must_have_all_of = []
    otu_must_have_one_of = 'any'
    define_trimmed_alns = [] #should be Locus_name@Alignment_method_name@Trimming_mathod_name
    
    feature_id_dict = {}
    
    def __init__(self,
                 name,
                 loci,
                 otu_meta,
                 otu_must_have_all_of = otu_must_have_all_of,
                 otu_must_have_one_of = otu_must_have_one_of,
                 define_trimmed_alns = define_trimmed_alns):
        self.name = name
        self.loci = loci
        self.otu_meta = otu_meta
        self.otu_must_have_all_of = otu_must_have_all_of
        self.otu_must_have_one_of = otu_must_have_one_of
        if isinstance(otu_must_have_all_of,str):
            raise IOError('The keyword \'otu_must_have_all_of\' has to be a list')
        if isinstance(otu_must_have_one_of[0],str) and not otu_must_have_one_of == 'any':
            raise IOError('The keyword \'otu_must_have_one_of\' has to be a list of lists')
        if self.otu_must_have_one_of == 'any':
            self.otu_must_have_one_of = [[l.name for l in self.loci]]
        self.feature_id_dict = {} # Will hold the feature_id list for each otu
        self.define_trimmed_alns = define_trimmed_alns # To choose between alternative
        # alignments of the same locus
        self.used_trimmed_alns = {} #To hold the alignment chosen for each locus
        
        # Validate loci list
        seen = []
        for locus in loci:
            if not isinstance(locus, Locus):
                raise TypeError("Expecting Locus object in loci list")
            if locus.name in seen:
                raise NameError('Locus ' + locus.name + ' appears more than once in self.loci')
            else:
                seen.append(locus.name)
      
                
                
    def __str__(self):
        loci_names = [i.name for i in self.loci]
        loci_string = ''
        for l in loci_names:
            loci_string += l+','
        loci_string = loci_string[:-1]
        must_have = ''
        for i in self.otu_must_have_all_of:
            must_have += i+','
        must_have = must_have[:-1]
        trimmed_alignmnets_spec = ''
        one_of = ''
        for i in self.otu_must_have_one_of:
            one_of += '[ '
            for j in i:
                one_of += j+' '
            one_of += ']'
        if (self.define_trimmed_alns) > 0:
            for i in self.define_trimmed_alns:
                trimmed_alignmnets_spec += i
        return ("Concatenation named %s, with loci %s,\n"
                "of which %s must exist for all species\n"
                "and at least one of each group of %s is represented.\n"
                "Alignments with the following names: %s are prefered"
                % (self.name, loci_string, must_have, one_of, trimmed_alignmnets_spec))
        
        
        
##############################################################################################
if False:
    """
    Reprophylo Project Utilities
    
    Used in the Project class but are not in the classe's methods
    """
##############################################################################################


## Git management

__builtin__.git = False


# git log template
gitline = "<<<<\n%s\nSTDOUT:\n%s\nSTDERR:%s\n>>>>\n"

def undate_git_log(pj, out, err):
    if not err:
        err = 'None'
    if not out:
        out = 'None'
    pj.git_log += gitline%(str(time.asctime()),str(out), str(err))

def start_git(pj):
    __builtin__.git = True # flag it on
    cwd = os.getcwd() 
    if os.path.isdir(cwd + '/.git'):
        # a repo exists, check it belongs to this project by checking the description
        try:
            assert open(cwd + '/.git/description').read().strip().rstrip() == pj.pickle_name.strip().rstrip()
            warnings.warn('Git repository exists for this Project')
        except:
            raise RuntimeError('The Git repository in the CWD does not belong to this project. Either the pickle'+
                              ' moved, or this is a preexsisting repo. Try one of the following: Delete the local '+
                              ' .Git dir if you don\'t need it, move the pickle and the notebook to a new work dir,'+
                              ' or if possible, move them back to their original location. You may also disable Git'+
                              ' by with stop_git().')
    else:
        # start a rep
        out, err = rpgit.gitInit()
        undate_git_log(pj, out, err)
        # write the pickle name as the repo description
        hndl = open(cwd + '/.git/description', 'wt')
        hndl.write(pj.pickle_name.strip().rstrip())
        hndl.close()
        warnings.warn('The new repository is called %s.'%open(cwd + '/.git/description', 'r').read().rstrip())
    
    # list scripts and notebooks
    import fnmatch
    matches = []
    for root, dirnames, filenames in os.walk(cwd):
        for filename in fnmatch.filter(filenames, '*.py'):
            matches.append(os.path.join(root, filename))
        for filename in fnmatch.filter(filenames, '*.ipynb'):
            matches.append(os.path.join(root, filename))
    # git add scripts and notebooks
    for match in matches:
        out, err = rpgit.gitAdd(match)
        undate_git_log(pj, out, err)
    # commit scripts and notebooks
    comment = "%i script file(s) from %s" % (len(matches), time.asctime())
    out, err = rpgit.gitCommit(comment)
    undate_git_log(pj, out, err)
    
    
    
def stop_git(pj):
    __builtin__.git = False # flad it off
    cwd = os.getcwd()
    
    # list, git add and git commit scripts and notebooks
    # list
    import fnmatch
    matches = []
    for root, dirnames, filenames in os.walk(cwd):
        for filename in fnmatch.filter(filenames, '*.py'):
            matches.append(os.path.join(root, filename))
        for filename in fnmatch.filter(filenames, '*.ipynb'):
            matches.append(os.path.join(root, filename))
    # add
    for match in matches:
        out, err = rpgit.gitAdd(match)
        undate_git_log(pj, out, err)
    # commit
    comment = "%i script file(s) from %s" % (len(matches), time.asctime())
    out, err = rpgit.gitCommit(comment)
    undate_git_log(pj, out, err)
    
## end git management

def platform_report():
    
    """ 
    Prints machine specs, os specs and dependencies at time of execution
    
    >>> isinstance(platform_report(), list)
    True
    """
    import pkg_resources
    modules = [] # and their versions
    for i in ('ete2','biopython','dendropy','cloud'):
        try:
            modules.append(i+' version: '+
                           pkg_resources.get_distribution(i).version)
        except:
            pass
    modules.append('reprophylo version %s'%str(reprophyloversion))
                   
    return(['Platform: '+platform.platform(aliased=0, terse=0),
            'Processor: '+platform.processor(),
            'Python build: '+platform.python_build()[0] + platform.python_build()[1],
            'Python compiler: '+platform.python_compiler(),
            'Python implementation: ' +platform.python_implementation(),
            'Python version: ' + platform.python_version()]+
             modules+
            ['User: ' +platform.uname()[1]])



def write_alns(pj, format = 'fasta'):
    """
    Writes untrimmed sequence alignment files that are in pj in a biopython format
    """
    
    if len(pj.alignments.keys()) == 0:
        raise IOError('Align the records first')
    else:
        for key in pj.alignments:
            AlignIO.write(pj.alignments[key], key+'_aln.'+format, format)



def keep_feature(feature, loci):
    
    """ Returns true if a feature's type is in one of the loci and if the gene
    or product qualifiers is in the aliases of one of the loci, for data collection
    from a genbank or embl file
    
    # making a dummy feature
    >>> coi = Locus('dna','CDS','coi', ['cox1','COX1','coi','COI','CoI'])
    >>> location = FeatureLocation(1,100)
    >>> feature = SeqFeature()
    >>> feature.location = location
    >>> feature.type = 'CDS'
    >>> feature.qualifiers['gene'] = ['CoI']
    
    # testing if fits any of the Project Locus objects
    >>> a = keep_feature(feature, [coi])
    >>> print(a)
    True"""
    
    keep = 0
    for g in loci:
        if not g.name in g.aliases:
            g.aliases.append(g.name)
        if feature.type == 'source':
            keep = 1
        elif feature.type == g.feature_type:
            qual = None
            if 'gene' in feature.qualifiers.keys():
                qual = 'gene'
            elif 'product' in feature.qualifiers.keys():
                qual = 'product'
            if qual and feature.qualifiers[qual][0] in g.aliases:
                keep = 1
    if keep == 1:
        return True
    else:
        return False
    return
  


def dwindle_record(record, loci):
    
    """ 
    Retains only features that are called by Locus objects and records with features that are
    called by Locus objects
    
    # Making a dummy locus    
    >>> coi = Locus('dna','CDS','coi', ['cox1','COX1','coi','COI','CoI'])
    
    # Making a dummy record with a feature that fits a Locus object (kept_feature)
    # and a feature that does not (dwindled_feature)
    >>> location = FeatureLocation(1,100)
    >>> kept_feature = SeqFeature()
    >>> kept_feature.location = location
    >>> kept_feature.type = 'CDS'
    >>> kept_feature.qualifiers['gene'] = ['CoI']
    >>> dwindled_feature = SeqFeature()
    >>> dwindled_feature.location = location
    >>> dwindled_feature.type = 'rRNA'
    >>> dwindled_feature.qualifiers['gene'] = ['LSU']
    >>> s = 'atgc'*1000
    >>> record = SeqRecord(seq=Seq(s, IUPAC.ambiguous_dna), id='1', description='spam')
    >>> record.features.append(kept_feature)
    >>> record.features.append(dwindled_feature)
    >>> print(len(record.features))
    2
    
    # Dwindling the record
    >>> a = dwindle_record(record, [coi])
    >>> print(len(record.features))
    1
    """
    
    dwindled_features = []
    feature_count = 0
    for feature in record.features:
        if keep_feature(feature, loci)== True:
            # determine feature id       
            if feature.type == 'source' and not 'feature_id' in feature.qualifiers.keys():
                feature.qualifiers['feature_id'] = [record.id + '_source']
            elif not 'feature_id' in feature.qualifiers.keys():
                feature.qualifiers['feature_id'] = [record.id + '_f' + str(feature_count)]
                feature_count += 1
            # determine prop ambiguity and GC content       
            if not feature.type == 'source':
                feature_seq = feature.extract(record.seq)
                degen = len(feature_seq)
                for i in ['A','T','G','C','U','a','t','g','c','u']:
                    degen -= feature_seq.count(i)
                feature.qualifiers['GC_content'] = [str(GC(feature_seq))]
                feature.qualifiers['nuc_degen_prop'] = [str(float(degen)/len(feature_seq))]
                if 'translation' in feature.qualifiers.keys():
                    transl = feature.qualifiers['translation'][0]
                    degen = 0
                    for i in ['B', 'X', 'Z', 'b', 'x', 'z']:
                        degen += transl.count(i)
                    feature.qualifiers['prot_degen_prop'] = [str(float(degen)/len(transl))]                    
            dwindled_features.append(feature)
    record.features = dwindled_features
    return record
            
 
    
def is_embl_or_gb(input_filename):
    suffixes = ['.gb','.embl']
    gb = False
    for s in suffixes:
        if s in input_filename:
            gb = True
    return gb



def parse_input(input_filename, fmt):
    return SeqIO.parse(input_filename, fmt)



def list_to_string(List):
    
    """
    Handles list printing as a nice string in the pj.write(format="csv") method
    
    >>> L = ['a','b','b']
    >>> print(list_to_string(L))
    a;b;b
    """
    
    string = ''
    for i in List:
        if type(i) is str and '\n' in i:
            string += lines_to_line(i).rstrip()+';'
        else:
            string += str(i)+';'
    return string[:-1]




def lines_to_line(lines):
    
    """
    Replaces newline with space in the pj.write(format="csv") method
    """
    
    lines = lines.split('\n')
    return (' ').join(lines)




def type_to_single_line_str(var):
    
    """
    Returns any type as a one line string for the pj.write(format="csv") method
    """
    
    if type(var) is str and '\n' in var:
        return lines_to_line(var)
    elif type(var) is str or type(var) is int or type(var) is float:
        return str(var)
    elif type(var) is list and len(var) == 1:
        return str(var[0])
    elif type(var) is list and len(var) > 0:
        return list_to_string(var)
    else:
        return var



def get_qualifiers_dictionary(project, feature_id):
    
    """
    Takes sequence record annotation, source qualifiers and feature qualifiers and puts them
    in a flat dictionary
    
    This is being replaced by __get_qualifiers_dictionary__ which deals with the records as a dict
    and is much faster. Eventually, records will be handled as a dict throughout, instead of as
    a list.
        
    # Making a dummy locus    
    >>> coi = Locus('dna','CDS','coi', ['cox1','COX1','coi','COI','CoI'])
    
    # Making a dummy Project
    >>> pj = Project([coi], git=False)
    
    # making a dummy record
    >>> s = 'atgc'*1000
    >>> location = FeatureLocation(1,100)
    >>> feature = SeqFeature()
    >>> feature.location = location
    >>> feature.type = 'CDS'
    >>> feature.qualifiers['gene'] = ['CoI']
    >>> feature.qualifiers['feature_id'] = ['1_f0']
    >>> source = SeqFeature()
    >>> source.location = FeatureLocation(0,3999)
    >>> source.type = 'source'
    >>> source.qualifiers['organism'] = ['Tetillda radiata']
    >>> record = SeqRecord(seq=Seq(s, IUPAC.ambiguous_dna), id='1', description='spam')
    >>> record.features.append(feature)
    >>> record.features.append(source)
    >>> record.annotations["evidence"] = 'made up'
    >>> pj.records = [record]
    
    # executing get_qualifiers_dictionary()
    >>> qual_dict = get_qualifiers_dictionary(pj, '1_f0')
    >>> qual_items = qual_dict.items()
    >>> qual_items.sort(key = lambda i: i[0])
    >>> for key, val in qual_items: print(key.ljust(20,' ') + val.ljust(20,' '))
    annotation_evidence made up             
    feature_id          1_f0                
    gene                CoI                 
    record_id           1                   
    source_organism     Tetillda radiata    
    """
    if type(feature_id) is list and len(feature_id) > 1:
        raise IOError('get_qualifiers_dictionary takes one feature_id at a time')
    if type(feature_id) is list:
        feature_id = feature_id[0]
    record_id = feature_id.rpartition('_')[0]
    qualifiers_dictionary={'record_id': record_id}
    for record in project.records:
        if record.id == feature_id.rpartition('_')[0]:
            for annotation in record.annotations.keys():
                qualifiers_dictionary['annotation_'+annotation]=record.annotations[annotation]
            for feature in record.features:
                if feature.type == 'source':
                    for qualifier in feature.qualifiers.keys():
                        qualifiers_dictionary['source_'+qualifier]=feature.qualifiers[qualifier][0]
                elif feature.qualifiers['feature_id'][0] == feature_id:
                    for qualifier in feature.qualifiers.keys():
                        qualifiers_dictionary[qualifier]=feature.qualifiers[qualifier][0]
    return qualifiers_dictionary

def __get_qualifiers_dictionary__(project, feature_id):
    
    """
    This will replace the public version. It uses the Project._records_dict to  pull 
    the record using the record id, instead of iterating Project.records, which is very slow.
    
    It requires Project.__records_list_to_dict__() to execute beforehand.
    """
    if type(feature_id) is list and len(feature_id) > 1:
        raise IOError('get_qualifiers_dictionary takes one feature_id at a time')
    if type(feature_id) is list:
        feature_id = feature_id[0]
    record_id = feature_id.rpartition('_')[0]
    record = project._records_dict[record_id]
    qualifiers_dictionary={'record_id': record_id}
    for annotation in record.annotations.keys():
        qualifiers_dictionary['annotation_'+annotation]=record.annotations[annotation]
    for feature in record.features:
        if feature.type == 'source':
            for qualifier in feature.qualifiers.keys():
                qualifiers_dictionary['source_'+qualifier]=feature.qualifiers[qualifier][0]
        elif feature.qualifiers['feature_id'][0] == feature_id:
            for qualifier in feature.qualifiers.keys():
                qualifiers_dictionary[qualifier]=feature.qualifiers[qualifier][0]
    return qualifiers_dictionary

def seq_format_from_suffix(suffix):
    
    """
    Guesses input format from suffix
    
    >>> print(seq_format_from_suffix('gb'))
    genbank
    """
    
    suffixes = {'fasta': ['fas','fasta','fa','fna'],
                'genbank': ['gb','genbank'],
                'embl': ['embl']}
    found = False
    for key in suffixes.keys():
        if suffix in suffixes[key]:
            found = True
            return key
    if not found:
        raise RuntimeError(suffix+' is not a recognised suffix of an unaligned sequence file')



def read_feature_quals_from_tab_csv(csv_filename):
                   
    """
    This is used to update feature qualifiers from a tab delimited file
    """               
                   
    import re
    header = open(csv_filename, 'r').readlines()[0].rstrip().split('\t')
    feature_id_col = header.index('feature_id')
    taxonomy_col = header.index('taxonomy')
    seq_col = header.index('seq')
    translation_col = None
    if 'translation' in header:
        translation_col = header.index('translation')
    csv_info = {}
    for line in [l.rstrip().split('\t') for l in open(csv_filename, 'r').readlines()[1:]]:
        if not line[0] in csv_info.keys():
            csv_info[line[0]] = {'source':{},
                                 'taxonomy':[],
                                 'features':{}
                                 }
        if csv_info[line[0]]['taxonomy'] == []:
            csv_info[line[0]]['taxonomy'] = line[taxonomy_col].split(';')
        csv_info[line[0]]['features'][line[feature_id_col]] = {}
        get_source = False
        if csv_info[line[0]]['source'] == {}:
            get_source = True
        for i in range(len(header)):
            if get_source and 'source:_' in header[i]:
                qual_name = re.sub('source:_','',header[i])
                if not line[i] == 'null' and not line[i] == '' and line[i]:
                    csv_info[line[0]]['source'][qual_name] = line[i].split(';')
            elif (not 'source:_' in header[i] and not line[i] == 'null' and not line[i] == '' and line[i] and
                  not i in [seq_col, translation_col, taxonomy_col, feature_id_col]):
                csv_info[line[0]]['features'][line[feature_id_col]][header[i]] = line[i].split(';')
    return csv_info


## Alignment statistics
                   
def count_positions(aln_column):
    counts = {}
    for i in aln_column:
        if i in counts.keys():
            counts[i] += 1
        else:
            counts[i] = 1
    return counts

def global_aln_stats(aln_obj):
    total_gaps = 0
    prop_list = []
    non_uniform_count = aln_obj.get_alignment_length()
    parsimony_informative = 0
    for i in range(aln_obj.get_alignment_length()):
        total_gaps += aln_obj[:, i].count('-')
        prop_list.append(aln_obj[:, i].count('-')/float(len(aln_obj)))
        if len(count_positions(aln_obj[:, i]).keys()) == 1:
            non_uniform_count -= 1
        elif (len(count_positions(aln_obj[:, i]).keys()) == 2 and
              '-' in  count_positions(aln_obj[:, i]).keys()):
            non_uniform_count -= 1
        if len([p for p in count_positions(aln_obj[:, i]).keys() if (p != '-' and  count_positions(aln_obj[:, i])[p] > 1)]) > 1:
            parsimony_informative += 1
    mean_gap_prop = sum(prop_list)/aln_obj.get_alignment_length()
    return (mean_gap_prop, non_uniform_count, parsimony_informative)

def count_undetermined_lines(aln_obj, cutoff=0):
    count = 0
    ids = []
    
    if aln_obj.get_alignment_length() < cutoff*2:
        warnings.warn('The cutoff to exclude a sequence is more than half of the alignmnet length')
    elif aln_obj.get_alignment_length() <= cutoff:
        raise RuntimeWarning('The cutoff to exclude a sequence is as long or longer than the alignment')
    
    for seq in aln_obj:
        if str(seq.seq).count('-') >= aln_obj.get_alignment_length()-cutoff:
            count += 1
            ids.append(seq.id)
    return count, ids

def count_collapsed_aln_seqs(aln_obj):
    count = 1
    seen_seqs = [str(aln_obj[0].seq)]
    for seq in aln_obj[1:]:
        str_seq = str(seq.seq)
        if len([s for s in seen_seqs if (str_seq in s or s in str_seq or s == str_seq)]) == 0:
            count += 1
        seen_seqs.append(str_seq)
    return count

def aln_summary(aln_obj, cutoff=0):
    lines = ["Alignment length: %i" % aln_obj.get_alignment_length(),
             "Number of rows: %i" % len(aln_obj),
             "Unique sequences: %i"%count_collapsed_aln_seqs(aln_obj),
             "Average gap prop.: %f\nVariable columns: %i\nParsimony informative: %i"
             %global_aln_stats(aln_obj),
             "Undetermined sequences: %i"%(count_undetermined_lines(aln_obj, cutoff=cutoff)[0]),
             "Undetermined sequence cutoff: %i"%cutoff
             ]
    return [lines, len(aln_obj), count_undetermined_lines(aln_obj, cutoff=cutoff), count_collapsed_aln_seqs(aln_obj)]        
            
##
                   
def loci_list_from_csv(loci):
    """
    Parse the loci csv file given to Project
    """               
    
    # verify format
    if any(len(line.split(',')) >= 4 for line in open(loci, 'r').readlines()):
        pass
    else:
        raise IOError("File %s has no valid loci of format char_type,feature_type,name,aliases"%loci)
        
        
    loci_dict = {}
    loci_list = []
    for line in [line.rstrip() for line in open(loci, 'r').readlines() if len(line.rstrip()) > 0]:
        # verify format
        if len(line.split(',')) < 4:
            raise IOError("The line %s in file %s is missing arguments. Needs at least char_type,feature_type,name,aliases"%
                          (line.rstrip(), loci))
        # look for synonyms
        else:
            group = None
            try:
                group = int(line.rstrip().split(',')[-1])
            except:
                pass
            
            if group:
                locus_exists = False
                for name in loci_dict:
                    if 'group' in loci_dict[name].keys() and loci_dict[name]['group'] == group:
                        loci_dict[name]['aliases'] += line.split(',')[3:-1]
                        locus_exists = True
                if not locus_exists:
                    loci_dict[line.split(',')[2]] = {'group': int(line.rstrip().split(',')[-1]),
                                                     'char_type': line.split(',')[0],
                                                     'feature_type': line.split(',')[1],
                                                     'aliases': line.split(',')[3:-1]
                                                     }
            else:
                loci_dict[line.split(',')[2]] = {'group': None,
                                                 'char_type': line.split(',')[0],
                                                 'feature_type': line.split(',')[1],
                                                 'aliases': line.split(',')[3:]
                                                 }
                
            
            
    for name in loci_dict:
        loci_list.append(Locus(loci_dict[name]['char_type'],
                               loci_dict[name]['feature_type'],
                               name,
                               loci_dict[name]['aliases']))
    return loci_list

def parse_paup_charset(nexus_filename):
    
    """ 
    Takes a nexus file with PAUP style charset commands.
    Returns a dictionary with partition names as keys and a list of
    integers representing the start and end of the partition as a value.
    Position count starts from 0.
    
    Handles paup commands of the following format:
    CHARSET  locus_name=1-129;
    or
    charset locus_name = 1 - 129 ;
    """
    
    try:
        AlignIO.read(nexus_filename, 'nexus')
    except:
        n = len(list(AlignIO.parse(nexus_filename, 'nexus')))
        raise IOError('Cannot handle more then one matrix in %s. Got %i matrices'%
                     (nexus_filename, n))
    
    charsets = {}
    charset_lines = [l for l in open(nexus_filename,'r').readlines() if 
                     (l.startswith('CHARSET') or l.startswith('charset'))]
    if len(charset_lines) == 0:
        raise IOError("There are no CHARSET commands in %s"%nexus_filename)
    for line in charset_lines:
        try:
            info = line.split()[1].split(';')[0]
            locus_name, range = info.split('=')
            locus_name = locus_name.strip().rstrip()
            start = int(range.split('-')[0].strip().rstrip())-1
            end = int(range.split('-')[1].strip().rstrip())-1
            charsets[locus_name] = [start,end]
        except:
            raise IOError('Expects "charset set_name = start_int - end_int;"'+
                          ' (case insensitive, spaces around the "=" or "-" not mandatory). Got %s'%line)
    return charsets
        
def pj_from_nexus_w_charset(nexus_filename, output_dir, char_type,
                            feature_type, project=False, pickle=False, git=False):
    
    """ 
    Takes a nexus file with PAUP style charset commands as input.
    Creates a separate fasta file for each alignment partition
    Returns a list of fasta filenames and a list of Locus objects
    If project==True, returns a Project instance with the loci, alignments and records instead
    """
    
    from reprophylo import Locus
    from Bio import AlignIO
    
    charsets = parse_paup_charset(nexus_filename)
    
    alignment =  AlignIO.read(nexus_filename, 'nexus')
    filenames = []
    loci_list = []
    for locus_name in charsets:
        s = charsets[locus_name][0]
        e = charsets[locus_name][1]
        outname = "%s/%s.fasta"%(output_dir,locus_name)
        AlignIO.write(alignment[:, s:e], outname, 'fasta')
        filenames.append(outname)
        loci_list.append(Locus(char_type, feature_type, locus_name, [locus_name]))
    
    if project:
        from reprophylo import Project
        pj = Project(loci_list, pickle=pickle, git=git)
        i=1
        for f in filenames:
            locus_name = f.split('/')[-1].split('.')[0]
            print '%i/%i reading %s'%(i,len(filenames), locus_name)
            i += 1
            pj.read_alignment(f, char_type, feature_type, locus_name)
        return pj
            
    else:  
        return filenames, loci_list

##############################################################################################
class Project:
##############################################################################################
    
    """
    The Project class contians all the data and has methods to analyze it. It allows for
    experimental analysis by running alternative analyses and formally comparing the 
    outputs. The pickle_pj() function allows to pickle the project, including the data,
    intermediates and results, as well as a description of the methods.It allows for a rerun
    of the whole analysis as is, as well as for a reconfiguration of the analysis or addition
    of data. If git is installed, it can be called by 'import rpgit'. As a result, git can be 
    initiated using start_git(). A git repository will be created in the CWD, if it doesn't already exist. 
    Input datasets, .py, .ipynb and .pkl files in the CWD will be version controlled. 
    Version control can be paused in the middle of the script
    by calling stop_git() and restarted by calling start_git() again.
    """

    def __init__(self, loci, pickle=None, git=True):
        
        """
        # making dummy loci
        >>> coi = Locus('dna','CDS','coi',['COX1','cox1'])
        >>> ssu = Locus('dna','rRNA','18S',['18S','SSU'])
        
        # Making a Project object
        >>> pj = Project([coi,ssu], git=False)
        >>> print(str(pj))
        Project object with the loci coi,18S,
        """
        self.records = []
        self._records_dict = {}
        self.starttime = str(time.asctime())
        self.user = None
        if os.path.isfile('USER'):
            self.user = []
            for line in open('USER','r').readlines():
                key, arg = line.rstrip().split('=')
                self.user.append([key, arg])
        self.loci = loci
        self.records_by_locus = {}
        self.concatenations = []
        self.alignments = {}
        self.trimmed_alignments = {}
        self.trees = {}
        self.used_methods = {}
        self.sets = {}
        self.git_log = ''
        
        self.pickle_name=pickle
        
        if self.pickle_name and os.path.exists(self.pickle_name):
            raise IOError('Pickle %s exists. If you want to keep using it do pj=unpickle_pj(\'%s\') instead.'%(self.pickle_name,self.pickle_name))
        
        if git and not self.pickle_name:
            raise IOError('Must have pickle to run Git. Either specify pickle or git=False')
        elif git:
            start_git(self)
         
        self.defaults = {'raxmlHPC': programspath+'raxmlHPC-PTHREADS-SSE3',
                         'mafft': 'mafft',
                         'muscle': programspath+'muscle',
                         'trimal': programspath+'trimal',
                         'pb': programspath+'pb',
                         'bpcomp': programspath+'bpcomp',
                         'tracecomp': programspath+'tracecomp',
                         'fasttree': programspath+'FastTreeMP',
                         'pal2nal': programspath+'pal2nal.pl',
                         # PROGRAM PLUG
                         # 'program name': programpath+'the basic command'
                        }
    
        seen = []
        if isinstance(loci,list):
            for locus in loci:
                if not isinstance(locus, Locus):
                    raise TypeError("Expecting Locus object in loci list. "+locus+
                                    " not a Locus object")
                if locus.name in seen:
                    raise NameError('Locus ' + locus.name + ' apears more than once in self.loci')
                else:
                    seen.append(locus.name)
        elif isinstance(loci,str):
            self.loci = loci_list_from_csv(loci)
            #print 'Read the following loci from file %s:'%loci
            #for l in self.loci:
                #print str(l)
        self.aln_summaries = []
        
        if self.pickle_name:
            pickle_pj(self, self.pickle_name, track=False)
        
        if __builtin__.git and self.pickle_name:
            import rpgit
            comment = "%s from %s" % (str(self), time.asctime())
            out, err = rpgit.gitAdd(self.pickle_name)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)
                
                
    def __str__(self):
        loci_string = ''
        for i in self.loci:
            loci_string += i.name+','
        return 'Project object with the loci '+loci_string


    def __records_list_to_dict__(self):
        self._records_dict = SeqIO.to_dict(self.records)
        
    def last_git_log(self):
        print self.git_log.split('<<<<')[-1]
        
    def show_commits(self):
        print rpgit.gitLog()[0]
        
    ###################################
    # Project methods for reading data
    ###################################  



    def read_embl_genbank(self, input_filenames_list):
        
        """
        Read a file from Genbank of EMBL
        
        >>> input_filenames = ['test-data/test.gb']
        >>> locus = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
        >>> pj = Project([locus], git=False)
        >>> print(len(pj.records))
        0
        >>> pj.read_embl_genbank(input_filenames)
        >>> print(len(pj.records))
        89
        """
        
        if __builtin__.git:
            import rpgit
        else:
            warnings.warn('Version control off')    
        generators = []
        for input_filename in input_filenames_list:
            if __builtin__.git:
                import rpgit
                out, err = rpgit.gitAdd(input_filename)
                undate_git_log(self, out, err)
            generators.append(parse_input(input_filename, 'gb'))
            for generator in generators:
                for record in generator:
                    dwindled_record = dwindle_record(record, self.loci)
                    if len(record.features) > 1:
                        self.records.append(dwindled_record)
                    elif len(record.features) == 1 and not record.features[0].type == 'source':
                        self.records.append(dwindled_record)
        if __builtin__.git:
            import rpgit
            comment = "%i genbank/embl data file(s) from %s" % (len(input_filenames_list), time.asctime()) 
            for filename in input_filenames_list:
                out, err = rpgit.gitAdd(filename)
                undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match) 
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment) 
            undate_git_log(self, out, err)
            
        
        
    def read_denovo(self, input_filenames, char_type, format = 'fasta'):
        
        """
        Include records from a fasta file. Fasta file records will be given record ids 
        of the form 'denovo1'. The record.id and record.description will be placed in a
        source feature under the 'original_id' and 'original_desc' qualifiers. Denovo sequences
        require the use of the add_feature_to_record() method in order to be included in the
        anaysis.
        

        >>> input_filenames = ['test-data/test.gb']
        >>> locus = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
        >>> pj = Project([locus], git=False)
        >>> print(len(pj.records))
        0
        >>> pj.read_embl_genbank(input_filenames)
        >>> print(len(pj.records))
        89
        >>> input_filenames = ['test-data/test.fasta']
        >>> pj.read_denovo(input_filenames, 'dna')
        1
        >>> print(len(pj.records))
        90
        
        # Since the denovo sequence has no feature it is not included
        >>> pj.extract_by_locus()
        >>> print(len(pj.records_by_locus['coi']))
        89
        
        # Making a feature for the denovo record.
        >>> pj.add_feature_to_record('denovo0', 'CDS',  qualifiers={'gene': 'coi'})
        'denovo0_f0'
        >>> pj.extract_by_locus()
        >>> print(len(pj.records_by_locus['coi']))
        90
        """
        
        if __builtin__.git:
            import rpgit
        else:
            warnings.warn('Version control off')
        
        count = 0
        # start the counter where it stoped the last time we read denovo things
        for record in self.records:
            if 'denovo' in record.id:
                serial = int(record.id[6:])
                if serial >= count:
                    count = serial+1
        for input_filename in input_filenames:
            if __builtin__.git:
                import rpgit
                rpgit.gitAdd(input_filename)
            denovo = SeqIO.parse(input_filename, format)
            for record in denovo:
                source = SeqFeature(FeatureLocation(0, len(record.seq)), type='source', strand=1)
                source.qualifiers['original_id'] = [record.id]
                source.qualifiers['original_desc'] = [(' ').join(record.description.split()[1:])]
                record.id = 'denovo'+str(count)
                record.name = record.id
                source.qualifiers['feature_id'] = [record.id+'_source']
                record.features = [source]
                if '-' in str(record.seq):
                    record.seq = Seq(str(record.seq).replace('-',''))
                    warnings.warn("Reseting gaps in records from %s"%input_filename)
                if char_type == 'prot':
                    record.seq.alphabet = IUPAC.protein
                    #feature = SeqFeature(FeatureLocation(0, len(record.seq)), type='Protein', strand=1)
                    #feature.qualifiers['translation'] = [str(record.seq)]
                    #feature.qualifiers['protonly']=['true']
                elif char_type == 'dna':
                    record.seq.alphabet = IUPAC.ambiguous_dna
                count += 1
                self.records.append(record)
                
        if __builtin__.git:
            import rpgit
            comment = "%i denovo data file(s) from %s" % (len(input_filenames), time.asctime())
            for filename in input_filenames:
                out, err = rpgit.gitAdd(filename)
                undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)
        return count       
    
    def read_alignment(self, filename, char_type, feature_type, locus_name, format="fasta", aln_method_name = "ReadDirectly", exclude=[]):
    
        if __builtin__.git:
            import rpgit
        else:
            warnings.warn('Version control off')
            
        if not any([locus.name == locus_name for locus in self.loci]):
            raise RuntimeError("Locus %s does not exist"%locus_name)
        elif not [locus for locus in self.loci if locus.name == locus_name][0].char_type == char_type:
            raise RuntimeError("%s is not a %s locus"%(locus_name, char_type))
        elif not [locus for locus in self.loci if locus.name == locus_name][0].feature_type == feature_type:
            raise RuntimeError("The feature_type %s is not %s"%(locus_name, feature_type))
            
        count = 0
        # start the counter where it stoped the last time we read denovo things
        for record in self.records:
            if 'denovo' in record.id:
                serial = int(record.id[6:])
                if serial >= count:
                    count = serial+1
        # Read the alignment:
        raw_aln_input = list(AlignIO.read(filename, format))
        
        
        
        # make records
        records = []
        aln_records = []
        for record in raw_aln_input:
            total_seq_len = len(record.seq)
            if (str(record.seq).count('-') + str(record.seq).count('.') + 
                str(record.seq).count('?') + str(record.seq).count('N') +
                str(record.seq).count('n') + str(record.seq).count('X') + 
                str(record.seq).count('x')) == total_seq_len:
                print 'dropping seq %s in locus %s: missing data'%(locus_name,record.id)
            elif not record.id in exclude:
                # remove gaps
                new_record = SeqRecord(seq=Seq(str(record.seq).replace('-','').replace('.','')))
                aln_record = SeqRecord(seq=Seq(str(record.seq).replace('.','-')))
                
                #set alphabet
                if char_type == 'prot':
                    new_record.seq.alphabet = IUPAC.protein
                    aln_record.seq.alphabet = IUPAC.protein
                    
                elif char_type == 'dna':
                    new_record.seq.alphabet = IUPAC.ambiguous_dna
                    aln_record.seq.alphabet = IUPAC.ambiguous_dna
                    
                # set denovo record id
                new_record.id = 'denovo%i'%count
                new_record.name = new_record.id
                
                # set source and first feature
                source = SeqFeature(FeatureLocation(0, len(new_record.seq)), type='source', strand=1)
                source.qualifiers['original_id'] = [record.id]
                source.qualifiers['original_desc'] = [(' ').join(record.description.split()[1:])]
                source.qualifiers['feature_id'] = [new_record.id+'_source']
                feature = SeqFeature(FeatureLocation(0, len(new_record.seq)), type=feature_type, strand=1)
                feature.qualifiers['feature_id'] = [new_record.id+'_f0']
                feature.qualifiers['gene'] = [locus_name]
                feature_seq = feature.extract(new_record.seq)
                degen = len(feature_seq)
                if char_type == 'dna':    
                    for i in ['A','T','G','C','U','a','t','g','c','u']:
                        degen -= feature_seq.count(i)
                    feature.qualifiers['GC_content'] = [str(GC(feature_seq))]
                    feature.qualifiers['nuc_degen_prop'] = [str(float(degen)/len(feature_seq))]
                    warnings.warn("To get translations, add a feature manually")
                elif char_type == 'prot': 
                    degen = 0
                    for i in ['B', 'X', 'Z', 'b', 'x', 'z']:
                        degen += feature_seq.count(i)
                    feature.qualifiers['prot_degen_prop'] = [str(float(degen)/len(feature_seq))]   
                new_record.features = [source, feature]
                
                aln_record.id = 'denovo%i_f0'%count
                
                count += 1
                
                records.append(new_record)
                aln_records.append(aln_record)
            
        token = "%s@%s"%(locus_name, aln_method_name)
        
        if token in self.alignments.keys():
            warnings.warn("Replacing alignment %s"%token)
        # need to add the denovo id's inside the alignment    
        self.alignments[token] = MultipleSeqAlignment(aln_records)
        self.records += records
        self.extract_by_locus()
        
        if __builtin__.git:
            import rpgit
            comment = "Alignment file %s" % (time.asctime())
            out, err = rpgit.gitAdd(filename)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)

    def add_feature_to_record(self, record_id, feature_type, location='full', qualifiers={}):
    
        """
        # Making a dummy locus    
        >>> coi = Locus('dna','CDS','coi', ['cox1','COX1','coi','COI','CoI'])
    
        # Making a dummy Project
        >>> pj = Project([coi], git=False)
    
        # making a dummy record
        >>> s = 'atgc'*1000
        >>> source = SeqFeature()
        >>> source.location = FeatureLocation(0,3999)
        >>> source.type = 'source'
        >>> record = SeqRecord(seq=Seq(s, IUPAC.ambiguous_dna), id='1', description='spam')
        >>> record.features.append(source)
        >>> pj.records = [record]
        >>> print(len(pj.records[0].features))
        1
        
        # adding a feature to a record in the pj
        >>> import warnings
        >>> warnings.simplefilter("ignore")
        >>> pj.add_feature_to_record('1', 'CDS', qualifiers={'gene': 'madeuplocus'})
        '1_f0'
        >>> print(len(pj.records[0].features))
        2
        """
    
        for record in self.records:
            if record.id == record_id:
                #determine new feature id
                feature_id = None
                serials = []
                if len(record.features) > 0:
                    for feature in record.features:
                        if 'feature_id' in feature.qualifiers.keys():
                            if '_f' in feature.qualifiers['feature_id'][0]:
                                f = feature.qualifiers['feature_id'][0]
                                serials.append(int(f.split('_')[1][1:]))
                    serials.sort(reverse = True)
                if len(serials) > 0:
                    feature_id = record.id + '_f' + str(serials[0]+1)
                else:
                    feature_id = record.id + '_f0'
                feature = None
                if location == 'full':
                    feature = SeqFeature(FeatureLocation(0, len(record.seq)),
                                         type=feature_type,
                                         strand=1)
                elif isinstance(location, list):
                    for i in location:
                        if not isinstance(i, list):
                            raise RuntimeError('\'location\' takes either \'full\' or a list of lists')
                    if len(location) == 1:
                        feature = SeqFeature(FeatureLocation(int(location[0][0])-1,int(location[0][1])),
                                             type=feature_type, strand=int(location[0][2]))
                    elif len(location) > 1:
                        list_of_locations = []
                        for i in location:
                            start = int(i[0]-1)
                            end = int(i[1])
                            strand = int(i[2])
                            list_of_locations.append(FeatureLocation(start,end,strand=strand))
                        feature = SeqFeature(CompoundLocation(list_of_locations),type=feature_type)
                feature.qualifiers['feature_id'] = [feature_id]
                
                
                
                if len(qualifiers.keys()) > 0:
                    for key in qualifiers.keys():
                        feature.qualifiers[key] = [qualifiers[key]]
                if (('codon_start' in qualifiers.keys()) and
                    ('transl_table' in qualifiers.keys())):
                    cds = feature.extract(record.seq)
                    if str(qualifiers['codon_start']) == '2':
                        cds = cds[1:]
                    elif str(qualifiers['codon_start']) == '3':
                        cds = cds[2:]
                    translation = cds.translate(table=int(qualifiers['transl_table']))
                    if len(translation)*3 < float(0.9)*len(cds):
                        raise RuntimeWarning('The translation of feature '+feature_id+' uses less than 90%'+
                                             ' of the coding sequence')
                    feature.qualifiers['translation'] = [str(translation)]
                
                
                feature_seq = feature.extract(record.seq)
                degen = len(feature_seq)
                for i in ['A','T','G','C','U','a','t','g','c','u']:
                    degen -= feature_seq.count(i)
                feature.qualifiers['GC_content'] = [str(GC(feature_seq))]
                feature.qualifiers['nuc_degen_prop'] = [str(float(degen)/len(feature_seq))]
                if 'translation' in feature.qualifiers.keys():
                    transl = feature.qualifiers['translation'][0]
                    degen = 0
                    for i in ['B', 'X', 'Z', 'b', 'x', 'z']:
                        degen += transl.count(i)
                    feature.qualifiers['prot_degen_prop'] = [str(float(degen)/len(transl))]   
                
                record.features.append(feature)
                return feature_id



    ##############################################
    # Project methods for managing concatenations
    ##############################################  


 
    def add_concatenation(self, concatenation_object):
        
        """
        add a Concatenation object to the Project
        
        # making dummy loci
        >>> coi = Locus('dna','CDS','coi',['COX1','cox1'])
        >>> ssu = Locus('dna','rRNA','18S',['18S','SSU'])
        >>> lsu = Locus('dna','rRNA','28S',['28S','LSU'])
        >>> loci = [coi,ssu,lsu]
        
        # making dummy Concatenation
        >>> combined = Concatenation(name='combined', 
        ...                          loci=loci, otu_meta='OTU_dict',
        ...                          otu_must_have_all_of=['coi'],
        ...                          otu_must_have_one_of =[['18S','28S']],
        ...                          define_trimmed_alns=["MafftLinsi@Gappyout"])
        >>> print(str(combined))
        Concatenation named combined, with loci coi,18S,28S,
        of which coi must exist for all species
        and at least one of each group of [ 18S 28S ] is represented.
        Alignments with the following names: MafftLinsi@Gappyout are prefered
        
        # making a dummy Project
        >>> pj = Project(loci, git=False)
        
        # Including the Concatenation in the Project
        >>> pj.add_concatenation(combined)
        >>> print(len(pj.concatenations))
        1
        """
        
        if isinstance(concatenation_object, Concatenation):
            # correct characters offending raxml, back up original values
            meta = concatenation_object.otu_meta
            self.copy_paste_within_feature(meta, "original_%s"%meta)
            offensive = ['\t','\r','\n', "'", '"', ',', ' ',
                         ';', ':', ']','[','(',')','/']
            for r in self.records:
                for f in r.features:
                    if meta in f.qualifiers.keys():
                        for o in offensive:
                            if o in f.qualifiers[meta][0]:
                                print (("found raxml offensive char %s in OTU %s. Replacing with '_ro_'."+
                                        "Backing up original in the qualifier original_%s.")%(o, f.qualifiers[meta][0], meta))
                                f.qualifiers[meta][0] = f.qualifiers[meta][0].replace(o,'_ro_')
            
            seen = []
            for s in self.concatenations:
                seen.append(s.name)
            if concatenation_object.name in seen:
                raise NameError('Concatenation ' + concatenation_object.name +
                                ' apears more than once in self.concatenations')
            else:
                self.concatenations.append(concatenation_object)
        else:
            raise TypeError("Expecting Concatenation object")

    
    #def make_concatenation_alignments(self):
        
    #    """
    #    Concatenates a trimmed alignment based on each of the Concatenation objects and adds them
    #    to the pj.trimmed_alignments dictionary. While a trimmed alignment of an individual locus will have a key
    #    following the patten "locus_name@alignment_method_name@trimming_method_name, the key for a concatenated
    #    trimmed alignment will be the Concatenation object name attribute.
    #    """
    #    for s in self.concatenations:
    #        
    #        # get a non-redundant list of OTUs stored in 'meta', such as voucher specimen
    #        meta = s.otu_meta
    #        OTU_list = []
    #        for record in self.records:
    #            for feature in record.features:
    #                if not feature.type == 'source':
    #                    qualifiers_dictionary = get_qualifiers_dictionary(self,
    #                                                                      feature.qualifiers['feature_id'])
    #                    if (meta in qualifiers_dictionary.keys() and
    #                        not qualifiers_dictionary[meta] in OTU_list):
    #                        OTU_list.append(qualifiers_dictionary[meta])
    #                        
    #        
    #    
    #
    #        included_individuals = {} #included_individuals[otu][locus]=feautre_id
    #        
    #        #Get the correct trimmed alignment tokens
    #        keys_of_trimmed_alignments_to_use_in_concat = []
    #        for locus in s.loci:
    #            trimmed_aln = None
    #            all_locus_trimmed_alns_in_pj = []
    #            for key in self.trimmed_alignments.keys():
    #                if locus.name == key.split('@')[0]:
    #                    all_locus_trimmed_alns_in_pj.append(key)
    #            if len(all_locus_trimmed_alns_in_pj) == 1:
    #                trimmed_aln = all_locus_trimmed_alns_in_pj[0]
    #            elif len(all_locus_trimmed_alns_in_pj) == 0:
    #                raise RuntimeError('Locus '+locus.name+' have no trimmed alignments')
    #            else:
    #                s.define_trimmed_alns.sort(key = lambda i: i.count('@'), reverse=True)
    #                for definition in s.define_trimmed_alns:
    #                    if definition.count('@') == 2 and locus.name == definition.split('@')[0]:
    #                        trimmed_aln = definition
    #                    elif definition.count('@') == 1 and any([definition in i for i in all_locus_trimmed_alns_in_pj]):
    #                        trimmed_aln = locus.name+'@'+definition
    #                    else:
    #                        raise RuntimeError("Could not determine which alignment/trimming alternative to use for locus '"+
    #                                            locus.name+"' out of "+str(all_locus_trimmed_alns_in_pj))
    #            if trimmed_aln:
    #                keys_of_trimmed_alignments_to_use_in_concat.append(trimmed_aln)
    #            else:
    #                raise RuntimeError('Could not find trimmed aln for locus '+locus.name+' given the rulls '+str(s.define_trimmed_alns))
    #        
    #        #print "%i individuals will be included in the concatenations %s"%(len(included_individuals.keys()), s.name)
    #        
    #        #if len(included_individuals.keys()) < 4:
    #        #    raise RuntimeError("Concatenation %s has less than 4 OTUs and cannot be analyzed"%s.name)
    #        for otu in OTU_list:
    #            otu_features = {}
    #            use = True
    #            
    #            # Check first rule
    #            for locus in s.otu_must_have_all_of:
    #                token = [t for t in keys_of_trimmed_alignments_to_use_in_concat if "%s@"%locus in t][0]
    #                feature_ids = [r.id for r in self.trimmed_alignments[token]]
    #                feature_found = False
    #                count = 0
    #                for feature_id in feature_ids:
    #                    qualifiers = get_qualifiers_dictionary(self, feature_id)
    #                    if meta in qualifiers.keys() and otu == qualifiers[meta]:
    #                        count += 1
    #                        feature_found = True
    #                        otu_features[locus] = feature_id
    #                if count > 1:
    #                    raise RuntimeError("%s is not unique in %s"%(otu, locus))
    #                if not feature_found:
    #                    use = False
    #                    
    #            # Check second rule
    #            if use:
    #                for group in s.otu_must_have_one_of:
    #                    if isinstance(group,str):
    #                        raise IOError('The keyword \'otu_must_have_one_of\' has to be a list of lists')
    #                    feature_found = False
    #                    for locus in group:
    #                        token = [t for t in keys_of_trimmed_alignments_to_use_in_concat if "%s@"%locus in t][0]
    #                        feature_ids = [r.id for r in self.trimmed_alignments[token]]
    #                        count = 0
    #                        for feature_id in feature_ids:
    #                            qualifiers = get_qualifiers_dictionary(self, feature_id)
    #                            if meta in qualifiers.keys() and otu == qualifiers[meta]:
    #                                count += 1
    #                                feature_found = True
    #                                otu_features[locus] = feature_id
    #                        if count > 1:
    #                            raise RuntimeError("%s is not unique in %s"%(otu, locus))
    #                    if not feature_found:
    #                        use = False
    #            if use:
    #                included_individuals[otu] = otu_features
    #        
    #        # printing a table of the alignment
    #        included_indivduals_table = ''
    #        loci_names = [l.name for l in s.loci]
    #        line = 'OTU'.ljust(30,' ')
    #        for name in loci_names:
    #            line += name.ljust(20,' ')
    #        included_indivduals_table += line+'\n'
    #        for otu in included_individuals.keys():
    #            line = otu.ljust(30,' ')
    #            for locus_name in loci_names:
    #                if locus_name in included_individuals[otu].keys():
    #                    line += included_individuals[otu][locus_name].ljust(15,' ')
    #                else:
    #                    line += ''.ljust(15,' ')
    #            included_indivduals_table += line+'\n'
    #        print "Concatenation %s will have the following data"%s.name
    #        print included_indivduals_table
    #        
    #        # remove partitions with less then 4 sequences
    #        for name in loci_names:
    #            if len([otu for otu in included_individuals.keys() if name in included_individuals[otu].keys()]) < 4:
    #                print (("Locus %s has less then 4 sequences in concatenation %s and where excluded "+
    #                                     "from the concatenation")%(name,s.name))
    #                for key in keys_of_trimmed_alignments_to_use_in_concat:
    #                    if name in key:
    #                        keys_of_trimmed_alignments_to_use_in_concat.remove(key)
    #                        
    #                                    
    #        
    #        # build alignment
    #        # concat_records = []
    #        alignment = []
    #        for individual in included_individuals.keys():
    #            sequence = ''
    #            for key in keys_of_trimmed_alignments_to_use_in_concat:
    #                locus_name = key.split('@')[0]                    
    #                length = len(self.trimmed_alignments[key][0])
    #                s.used_trimmed_alns[key] = length
    #                if locus_name in included_individuals[individual].keys():
    #                    for record in self.trimmed_alignments[key]:
    #                        if record.id == included_individuals[individual][locus_name]:
    #                            sequence += str(record.seq)
    #                else:
    #                    sequence += '?'*length
    #            concat_sequence = SeqRecord(seq = Seq(sequence), id = individual, description = '')
    #            alignment.append(concat_sequence)
    #        self.trimmed_alignments[s.name] = MultipleSeqAlignment(alignment)                
    #        s.feature_id_dict = included_individuals    

    def make_concatenation_alignments(self):
        
        """
        Concatenates a trimmed alignment based on each of the Concatenation objects and adds them
        to the pj.trimmed_alignments dictionary. While a trimmed alignment of an individual locus will have a key
        following the patten "locus_name@alignment_method_name@trimming_method_name, the key for a concatenated
        trimmed alignment will be the Concatenation object name attribute.
        """
        
        self.__records_list_to_dict__()
        
        for s in self.concatenations:
            
            # get a non-redundant list of OTUs stored in 'meta', such as voucher specimen
            meta = s.otu_meta
            OTU_list = []
            for record in self.records:
                for feature in record.features:
                    if not feature.type == 'source':
                        qualifiers_dictionary = __get_qualifiers_dictionary__(self,
                                                                          feature.qualifiers['feature_id'])
                        if (meta in qualifiers_dictionary.keys() and
                            not qualifiers_dictionary[meta] in OTU_list):
                            OTU_list.append(qualifiers_dictionary[meta])
                            
            
        
    
            included_individuals = {} #included_individuals[otu][locus]=feautre_id
            
            #Get the correct trimmed alignment tokens
            keys_of_trimmed_alignments_to_use_in_concat = []
            for locus in s.loci:
                trimmed_aln = None
                all_locus_trimmed_alns_in_pj = []
                for key in self.trimmed_alignments.keys():
                    if locus.name == key.split('@')[0]:
                        all_locus_trimmed_alns_in_pj.append(key)
                if len(all_locus_trimmed_alns_in_pj) == 1:
                    trimmed_aln = all_locus_trimmed_alns_in_pj[0]
                elif len(all_locus_trimmed_alns_in_pj) == 0:
                    raise RuntimeError('Locus '+locus.name+' have no trimmed alignments')
                else:
                    s.define_trimmed_alns.sort(key = lambda i: i.count('@'), reverse=True)
                    for definition in s.define_trimmed_alns:
                        if definition.count('@') == 2 and locus.name == definition.split('@')[0]:
                            trimmed_aln = definition
                        elif definition.count('@') == 1 and any([definition in i for i in all_locus_trimmed_alns_in_pj]):
                            trimmed_aln = locus.name+'@'+definition
                        else:
                            raise RuntimeError("Could not determine which alignment/trimming alternative to use for locus '"+
                                                locus.name+"' out of "+str(all_locus_trimmed_alns_in_pj))
                if trimmed_aln:
                    keys_of_trimmed_alignments_to_use_in_concat.append(trimmed_aln)
                else:
                    raise RuntimeError('Could not find trimmed aln for locus '+locus.name+' given the rulls '+str(s.define_trimmed_alns))
            
            #print "%i individuals will be included in the concatenations %s"%(len(included_individuals.keys()), s.name)
            
            #if len(included_individuals.keys()) < 4:
            #    raise RuntimeError("Concatenation %s has less than 4 OTUs and cannot be analyzed"%s.name)
            for otu in OTU_list:
                otu_features = {}
                use = True
                
                # Check first rule
                for locus in s.otu_must_have_all_of:
                    token = [t for t in keys_of_trimmed_alignments_to_use_in_concat if "%s@"%locus in t][0]
                    feature_ids = [r.id for r in self.trimmed_alignments[token]]
                    feature_found = False
                    count = 0
                    for feature_id in feature_ids:
                        qualifiers = __get_qualifiers_dictionary__(self, feature_id)
                        if meta in qualifiers.keys() and otu == qualifiers[meta]:
                            count += 1
                            feature_found = True
                            otu_features[locus] = feature_id
                    if count > 1:
                        raise RuntimeError("%s is not unique in %s"%(otu, locus))
                    if not feature_found:
                        use = False
                        
                # Check second rule
                if use:
                    for group in s.otu_must_have_one_of:
                        if isinstance(group,str):
                            raise IOError('The keyword \'otu_must_have_one_of\' has to be a list of lists')
                        feature_found = False
                        for locus in group:
                            token = [t for t in keys_of_trimmed_alignments_to_use_in_concat if "%s@"%locus in t][0]
                            feature_ids = [r.id for r in self.trimmed_alignments[token]]
                            count = 0
                            for feature_id in feature_ids:
                                qualifiers = __get_qualifiers_dictionary__(self, feature_id)
                                if meta in qualifiers.keys() and otu == qualifiers[meta]:
                                    count += 1
                                    feature_found = True
                                    otu_features[locus] = feature_id
                            if count > 1:
                                raise RuntimeError("%s is not unique in %s"%(otu, locus))
                        if not feature_found:
                            use = False
                if use:
                    included_individuals[otu] = otu_features
            
            # printing a table of the alignment
            included_indivduals_table = ''
            loci_names = [l.name for l in s.loci]
            line = 'OTU'.ljust(30,' ')
            for name in loci_names:
                line += name.ljust(20,' ')
            included_indivduals_table += line+'\n'
            for otu in included_individuals.keys():
                line = otu.ljust(30,' ')
                for locus_name in loci_names:
                    if locus_name in included_individuals[otu].keys():
                        line += included_individuals[otu][locus_name].ljust(15,' ')
                    else:
                        line += ''.ljust(15,' ')
                included_indivduals_table += line+'\n'
            print "Concatenation %s will have the following data"%s.name
            print included_indivduals_table
            
            # remove partitions with less then 4 sequences
            for name in loci_names:
                if len([otu for otu in included_individuals.keys() if name in included_individuals[otu].keys()]) < 4:
                    print (("Locus %s has less then 4 sequences in concatenation %s and where excluded "+
                                         "from the concatenation")%(name,s.name))
                    for key in keys_of_trimmed_alignments_to_use_in_concat:
                        if name in key:
                            keys_of_trimmed_alignments_to_use_in_concat.remove(key)
                            
                                        
            
            # build alignment
            # concat_records = []
            alignment = []
            for individual in included_individuals.keys():
                sequence = ''
                for key in keys_of_trimmed_alignments_to_use_in_concat:
                    locus_name = key.split('@')[0]                    
                    length = len(self.trimmed_alignments[key][0])
                    s.used_trimmed_alns[key] = length
                    if locus_name in included_individuals[individual].keys():
                        for record in self.trimmed_alignments[key]:
                            if record.id == included_individuals[individual][locus_name]:
                                sequence += str(record.seq)
                    else:
                        sequence += '?'*length
                concat_sequence = SeqRecord(seq = Seq(sequence), id = individual, description = '')
                alignment.append(concat_sequence)
            self.trimmed_alignments[s.name] = MultipleSeqAlignment(alignment)                
            s.feature_id_dict = included_individuals 
    
        self._records_dict = {}
        
    ###################################################
    # Project methods for modifying feature qualifiers
    ###################################################



    def write(self, filename, format = 'genbank'):
        
        """
        Write the records in the project in any Biopython format, as csv or as nexml.
        
        The csv table has a line for each feature (ie multiplt lines for records
        with multiple non-source featue). Each line will include the records annotations,
        the source feature qualifiers and the qualifiers of the feature itself. (ie, in a
        record with several features, the record annotations and the source feature qualifiers
        will be repeted in several lines, once for each non-source feature in the record.
        The csv file is primarily usefull for reviewing and editing feature qualifiers
        
        The nexml format only includes the trees from the pj.trees dictionary, but all 
        the metadata is included as leaf metadata, including the sequences and the
        trimmed and alligned sequences for each leaf.
        """
        if format == 'nexml':
            self.write_nexml(filename)
        if format == 'phyloxml':
            self.write_phyloxml(filename)
        elif format == 'genbank' or format == 'embl':
            SeqIO.write(self.records, filename, format)
        elif format == 'csv':
            # get titles for source and othe features
            source_qualifiers = []
            feature_qualifiers = []
            for record in self.records:
                for feature in record.features:
                    for key in feature.qualifiers.keys():
                        if feature.type == 'source' and not key in source_qualifiers:
                            source_qualifiers.append(key)
                        elif not feature.type == 'source' and not key in feature_qualifiers:
                            feature_qualifiers.append(key)
            
            with open(filename, 'wb') as csvfile:
                linewriter = csv.writer(csvfile, delimiter='\t',
                                        quotechar='"',
                                        quoting=csv.QUOTE_MINIMAL)
                linewriter.writerow(['record_id','seq']+['source:_'+q for q in source_qualifiers]+['taxonomy']+feature_qualifiers)
                for record in self.records:
                    seq = ''
                    if len(record.seq) <= 10:
                        seq = str(record.seq)[0:11]
                    else:
                        seq = str(record.seq)[0:6] + '...' + str(record.seq)[-5:]
                    
                    
                    line_start = [record.id, seq]
                    source = None
                    for feature in record.features:
                        if feature.type == 'source':
                            source = feature
                    if not source == None:
                        for qual in source_qualifiers:
                            if qual in source.qualifiers.keys():
                                line_start.append(type_to_single_line_str(source.qualifiers[qual]))
                            else:
                                line_start.append('null')
                    elif source == None:
                        for qual in source_qualifiers:
                            line_start.append('null')
                    if 'taxonomy' in record.annotations.keys():
                        line_start.append(type_to_single_line_str(record.annotations['taxonomy']))
                    else:
                        line_start.append(['null'])
                    for feature in record.features:
                        if not feature.type == 'source':
                            line = list(line_start)
                            for qual in feature_qualifiers:
                                if qual in feature.qualifiers.keys() and qual == 'translation':
                                    trans = feature.qualifiers[qual][0]
                                    if len(trans)<=10:
                                        line.append(trans)
                                    else:
                                        line.append(trans[:6] + '...' + trans[-5:])
                                elif qual in feature.qualifiers.keys():
                                    line.append(type_to_single_line_str(feature.qualifiers[qual]))
                                else:
                                    line.append('null')
                            linewriter.writerow(line)
        
        if __builtin__.git:
            import rpgit
            comment = "Records %s text file from %s" % (format, time.asctime())
            out, err = rpgit.gitAdd(filename)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)

    def correct_metadata_from_file(self,csv_file):
        metadata = read_feature_quals_from_tab_csv(csv_file)
        new_records = []
        for record in self.records:
            if record.id in metadata.keys():
                #print 'making new record'
                new_record = record
                record_corrected_metadata = metadata[record.id]
                new_record.annotations['taxonomy'] = metadata[record.id]['taxonomy']
                for feature in new_record.features:
                    if feature.type == 'source':
                        feature.qualifiers = metadata[record.id]['source']
                    else:
                        feature_id = feature.qualifiers['feature_id']
                        translation = None
                        if 'translation' in feature.qualifiers.keys():
                            translation = feature.qualifiers['translation']
                        feature.qualifiers = metadata[record.id]['features'][feature_id[0]]
                        feature.qualifiers['feature_id'] = feature_id
                        if translation:
                            feature.qualifiers['translation'] = translation
                new_records.append(new_record)
            else:
                #print 'using old records'
                new_records.append(record)
        
        self.records = new_records
        
        if __builtin__.git:
            import rpgit
            comment = "Corrected metadata CSV file from %s" % (time.asctime())
            out, err = rpgit.gitAdd(csv_file)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)
                
    def if_this_then_that(self, IF_THIS, IN_THIS, THEN_THAT, IN_THAT, mode = 'whole'):
        
        
        """
        Searches pj.records for features that have the value IF_THIS in the qualifier IN_THIS
        and places the value THEN_THAT in the qualifier IN_THAT, which either exists or is new.
        
        The IF_THIS value can either match completely (mode = 'whole') or just to a part (mode = 'part')
        of the target qualifier value
        
        The following demonstartes all the feature qualifier editing methods
        
        # Make a dummy pj with a locus and with records
        >>> input_filenames = ['test-data/test.gb']
        >>> locus = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
        >>> pj = Project([locus], git=False)
        >>> pj.read_embl_genbank(input_filenames)
        
        # copying a source qualifier into the feature qualifiers so that it
        # will be available for editing (source qualifiers are kept imutable)
        >>> pj.add_qualifier_from_source('organism')
        
        # populate a new qualifier based on the data in another
        # Here we will take only the the genus name from the organism qualifier
        # and put it in a new qualifier
        # We use mode='part' because our search phrase (the genus name)
        # fits only the start of the organism name
        >>> tetillid_genera = ['Tetilla', 'Cinachyrella', 'Craniella']
        >>> for g in tetillid_genera:
        ...     pj.if_this_then_that(g, 'organism', g, 'genus', mode='part')
        
        # Now we will add a sample id to all the sequences which belong to
        # sample TAU_25617
        >>> pj.add_qualifier(['JX177913.1_f0',
        ...                  'JX177935.1_f0',
        ...                  'JX177965.1_f0'],
        ...                  'specimen_voucher',
        ...                  'TAU_25617')
    
        # We are using a copy paste approch to unite the data from 
        # differen qualifiers under on qualifiers
        >>> pj.copy_paste_within_feature('variant', 'strain_or_variant')
        >>> pj.copy_paste_within_feature('strain', 'strain_or_variant')
        
        # Now we print the qualifier of a random feature as an example
        >>> qual_dict = get_qualifiers_dictionary(pj, 'JX177913.1_f0')
        >>> qual_items = qual_dict.items()
        >>> qual_items.sort(key = lambda i: i[0])
        >>> for key, val in qual_items:
        ...     print(key.ljust(40,' ') + type_to_single_line_str(val)[:5]+'...')
        GC_content                              37.28...
        annotation_accessions                   JX177...
        annotation_data_file_division           INV...
        annotation_date                         05-SE...
        annotation_gi                           39293...
        annotation_keywords                     ...
        annotation_organism                     Cinac...
        annotation_references                   locat...
        annotation_sequence_version             1...
        annotation_source                       mitoc...
        annotation_taxonomy                     Eukar...
        codon_start                             2...
        db_xref                                 GI:39...
        feature_id                              JX177...
        gene                                    cox1...
        genus                                   Cinac...
        nuc_degen_prop                          0.0...
        organism                                Cinac...
        product                                 cytoc...
        prot_degen_prop                         0.0...
        protein_id                              AFM91...
        record_id                               JX177...
        source_country                          Panam...
        source_db_xref                          taxon...
        source_feature_id                       JX177...
        source_identified_by                    Ilan,...
        source_mol_type                         genom...
        source_note                             autho...
        source_organelle                        mitoc...
        source_organism                         Cinac...
        source_specimen_voucher                 DH_S2...
        specimen_voucher                        TAU_2...
        transl_table                            4...
        translation                             MIGSG...
        
        # Note that GC content and the porportion of degenerate positions
        # have been automatically included. They will be plotted in the report
        """
        
        for record in self.records:
            for feature in record.features:
                if not feature.type == 'source':
                    if IN_THIS in feature.qualifiers.keys():
                        if not type(feature.qualifiers[IN_THIS]) is list:
                            feature.qualifiers[IN_THIS] = [feature.qualifiers[IN_THIS]]
                        for i in feature.qualifiers[IN_THIS]:
                            if mode == 'whole':
                                if i == IF_THIS:
                                    feature.qualifiers[IN_THAT] = [THEN_THAT]
                            elif mode == 'part':
                                if IF_THIS in i:
                                    feature.qualifiers[IN_THAT] = [THEN_THAT]
    


    def add_qualifier(self, feature_ids, name, value):
        if not type(value) is list:
                    value = [value]
        for record in self.records:
            for feature in record.features:
                if feature.qualifiers['feature_id'][0] in feature_ids:
                    feature.qualifiers[name] = value



    def add_qualifier_from_source(self, qualifier):
        for record in self.records:
            source = None
            for feature in record.features:
                if feature.type == 'source':
                    source = feature
            value = None
            if not source == None:
                if qualifier in source.qualifiers.keys():
                    value = source.qualifiers[qualifier]       
            if not value == None:
                if not type(value) is list:
                    value = [value]
                for feature in record.features:
                    if not feature.type == 'source':
                        feature.qualifiers[qualifier] = value
    


    def copy_paste_within_feature(self, from_qualifier, to_qualifier):
        for record in self.records:
            for feature in record.features:
                if not feature.type == 'source':
                    if from_qualifier in feature.qualifiers.keys():
                        feature.qualifiers[to_qualifier] = feature.qualifiers[from_qualifier]
                  
                        
                        
    def copy_paste_from_features_to_source(self, from_feature_qual, to_source_qual):
        for record in self.records:
            source = None
            values_from_features = []
            for feature in record.features:
                if not feature.type == 'source':
                    if from_feature_qual in feature.qualifiers.keys():
                        if not feature.qualifiers[from_feature_qual] in values_from_features:
                            values_from_features += feature.qualifiers[from_feature_qual]
                else:
                    source = feature
            if source == None:
                source = SeqFeature(FeatureLocation(0, len(record.seq)), type='source', strand=1)
                source.qualifiers['feature_id'] = record.id + '_source'
                record.features = [source] + record.features
            if not to_source_qual in source.qualifiers.keys():
                source.qualifiers[to_source_qual] = values_from_features
           
                
                
    def species_vs_loci(self, outfile_name):
        
        """
        Makes a csv file showing the count of each unique value in the source_organism qualifier
        for each locus
        """
        
        species_vs_loci = {}
        for record in self.records:
            organism = 'undef'
            for feature in record.features:
                if feature.type == 'source':
                    if 'organism' in feature.qualifiers.keys():
                        organism = feature.qualifiers['organism'][0]
            if not organism in species_vs_loci.keys():
                species_vs_loci[organism] = {}    
            for feature in record.features:
                if not feature.type == 'source':
                    for locus in self.loci:
                        if not locus.name in locus.aliases:
                            locus.aliases.append(locus.name)
                        if 'gene' in feature.qualifiers.keys():
                            if feature.qualifiers['gene'][0] in locus.aliases:
                                if not locus.name in species_vs_loci[organism].keys():
                                    species_vs_loci[organism][locus.name] = 1
                                else:
                                    species_vs_loci[organism][locus.name] += 1
                        elif 'product' in feature.qualifiers.keys():
                            if feature.qualifiers['product'][0] in locus.aliases:
                                if not locus.name in species_vs_loci[organism].keys():
                                    species_vs_loci[organism][locus.name] = 1
                                else:
                                    species_vs_loci[organism][locus.name] += 1
        with open(outfile_name, 'wb') as csvfile:
            linewriter = csv.writer(csvfile, delimiter='\t',
                                    quotechar='|',
                                    quoting=csv.QUOTE_MINIMAL)
            loci_names = []
            for g in self.loci:
                loci_names.append(g.name)
            linewriter.writerow(['species']+loci_names)
            for organism in sorted(list(species_vs_loci.keys())):
                line = [organism]
                for name in loci_names:
                    if name in species_vs_loci[organism].keys():
                        line.append(str(species_vs_loci[organism][name]))
                    else:
                        line.append('0')
                linewriter.writerow(line)
                
                
                
    ######################################                
    # Project methods to analyze the data
    ######################################
    
    
        
    def extract_by_locus(self):
        
        """
        
        >>> input_filenames = ['test-data/test.gb']
        >>> coi = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
        >>> lsu = Locus('dna', 'rRNA', '28S', ['28s','28S','LSU rRNA','28S ribosomal RNA','28S large subunit ribosomal RNA'])
        >>> pj = Project([coi, lsu], git=False)
        >>> pj.read_embl_genbank(input_filenames)
        >>> pj.extract_by_locus()
        >>> print(len(pj.records_by_locus['coi']))
        89
        >>> print(len(pj.records_by_locus['28S']))
        48
        """
        
        data_by_locus = {}
        for locus in self.loci:
            if not locus.name in locus.aliases:
                locus.aliases.append(locus.name)
                locus.aliases.append(locus.name.replace('_',' '))
            records = []
            for record in self.records:
                for feature in record.features:
                    if ((feature.type == locus.feature_type or
                         # equate CDS and Protein feature types to allow reading protein sequences from 
                         # a mix of DNA gb files and protein fasta files that were given a Protein feature
                         # type
                         (feature.type in ['CDS','Protein'] and locus.feature_type in ['CDS','Protein'])) and
                        
                        (('gene' in feature.qualifiers.keys() and
                          feature.qualifiers['gene'][0] in locus.aliases) 
                         or
                         ('product' in feature.qualifiers.keys() and 
                          feature.qualifiers['product'][0] in locus.aliases))
                        
                        ):
                        if locus.char_type == 'dna':
                            if not type(record.seq.alphabet) == IUPAC.IUPACProtein:
                                S = feature.extract(record.seq)
                            else:
                                raise RuntimeError('Trying to read DNA from protein only records')
                        elif locus.char_type == 'prot':
                            if not type(record.seq.alphabet) == IUPAC.IUPACProtein:
                                if 'translation' in feature.qualifiers.keys():
                                    S = Seq(feature.qualifiers['translation'][0], IUPAC.protein)
                                else:
                                    raise RuntimeError('Trying to read protein from DNA records without translation info')
                            else:
                                S = feature.extract(record.seq)
                        feature_record = SeqRecord(seq = S, id = feature.qualifiers['feature_id'][0],
                                                   description = '')
                        records.append(feature_record)
            data_by_locus[locus.name] = records
        self.records_by_locus = data_by_locus

    def exclude(self, start_from_max=True, **kwargs):
        keep_safe = self.records_by_locus
        self.extract_by_locus()
        locus_names = [i.name for i in self.loci]
        for key, value in kwargs.iteritems():
            if key in locus_names:
                if value == 'all':
                    self.records_by_locus[key] = []
                else:
                    subset = []
                    locus_feature_ids = [i.id.rpartition('_')[0] for i in self.records_by_locus[key]]
                    if not all(i in locus_feature_ids for i in value):
                        print [i for i in value if not i in locus_feature_ids]
                        warnings.warn('Not all records to exclude exist in locus. Typos?')
                    if not start_from_max and not keep_safe == {}:
                        for record in keep_safe[key]:
                            if not record.id.rpartition('_')[0] in [i for i in value]:
                                subset.append(record)
                        self.records_by_locus[key] = subset
                    else:
                        for record in self.records_by_locus[key]:
                            if not record.id.rpartition('_')[0] in [i for i in value]:
                                subset.append(record)
                        self.records_by_locus[key] = subset
            else:
                warnings.warn('Locus name %s not recognised'%key)

    def include(self, start_from_null=True, **kwargs):
        keep_safe = self.records_by_locus
        self.extract_by_locus()
        locus_names = [i.name for i in self.loci]
        for key, value in kwargs.iteritems():
            if key in locus_names:
                if value == 'all':
                    pass
                else:
                    subset = []
                    locus_feature_ids = [i.id.rpartition('_')[0] for i in self.records_by_locus[key]]
                    if not all(i in locus_feature_ids for i in value):
                        print [i for i in value if not i.split('_')[0] in locus_feature_ids]
                        warnings.warn('Not all records to include exist in locus. Typos?')
                    for record in self.records_by_locus[key]:
                        if record.id.rpartition('_')[0] in [i for i in value]:
                            subset.append(record)
                    self.records_by_locus[key] = subset
                    if not start_from_null and not keep_safe == {}:
                        self.records_by_locus[key] = subset+keep_safe[key]
            else:
                warnings.warn('Locus name %s not recognised'%key)

    def filter_by_seq_length(self, locus_name, min_length=0, max_length=None):
        if self.records_by_locus == {}:
            self.extract_by_locus()
        subset = [r for r in self.records_by_locus[locus_name] if len(r) >= min_length]
        if max_length:
            subset = [r for r in subset if len(r) <= max_length]
        self.records_by_locus[locus_name] = subset
        
    def filter_by_gc_content(self, locus_name, min_percent_gc=0, max_percent_gc=None):
        if self.records_by_locus == {}:
            self.extract_by_locus()
        subset = [r for r in self.records_by_locus[locus_name] if GC(r.seq) >= min_percent_gc]
        if max_percent_gc:
            subset = [r for r in subset if GC(r.seq) <= max_percent_gc]
        self.records_by_locus[locus_name] = subset
                

    def write_by_locus(self, format = 'fasta'):
        
        """
        Write the unaligned sequences into file in any Biopython format, one file per locus
        """
        
        if self.records_by_locus == {}:
            self.extract_by_locus
        for key in self.records_by_locus.keys():
            SeqIO.write(self.records_by_locus[key], key+'.'+format, format)



    def align(self, alignment_methods=[], pal2nal='defaults'):
        
            """
            Configured by an AlnConf object
            """
            if pal2nal == 'defaults':
                pal2nal = self.defaults['pal2nal']
            seen_loci = []
            for method in alignment_methods:
                method.timeit.append(time.time())
                method.platform = platform_report()
                if method.program_name == 'muscle':
                    method.platform.append('Program and version: '+
                                           os.popen(method.cmd + ' -version').read().split(' by')[0])
                    method.platform.append('Program reference:\nEdgar 2004: MUSCLE: multiple sequence '+
                                           'alignment with high accuracy and high throughput. '+
                                           'Nucleic Acids Research 32(5):1792-1797')
                elif method.program_name == 'mafft':
                    p = sub.Popen(method.cmd+" --version", shell=True, stderr=sub.PIPE, stdout=sub.PIPE)
                    
                    try:
                         method.platform.append('Program and version: '+
                                                p.communicate()[1].splitlines()[3].strip().split(' (')[0])
                    except:
                        # new version of MAFFT  7.221 sends version to stderr
                        try:
                             method.platform.append('Program and version: '+
                                                    p.communicate()[1].split(' (')[0])
                        except:
                            method.platform.append('Program and version: not_retrived')
                        
                    method.platform.append('Program reference:Katoh, Standley 2013 (Molecular Biology and Evolution 30:772-780) '+
                                           'MAFFT multiple sequence alignment software version 7: improvements in performance and usability.')
                
                
                
                # PROGRAM PLUG
                # NOTE: THIS ADDS THE PROGRAM AND REFERENCE INFO TO THE CONF OBJECT
                #
                # elif method.program_name == 'the name of the program':
                #    p = sub.Popen(method.cmd+" command that writes version", shell=True, stderr=sub.PIPE, stdout=sub.PIPE)
                #    method.platform.append('Program and version: '+
                #                           # 1 for stderr, 0 for stdout
                #                           p.communicate()[1].splitlines()[# get the line and split])
                #    method.platform.append('Program reference: write the reference here')
                
                
                if  method.CDSAlign:
                    p = sub.Popen('pal2nal.pl', shell=True, stderr=sub.PIPE, stdout=sub.PIPE)
                    method.platform[-2] += '\nPal2Nal '+ p.communicate()[1].splitlines()[1].split('(')[1].split(')')[0]
                    method.platform[-1] += ['\nMikita Suyama, David Torrents, and Peer Bork (2006) PAL2NAL: robust '+ 
                                            'conversion of protein sequence alignments into the corresponding codon '+
                                            'alignments.Nucleic Acids Res. 34, W609-W612.'][0]
                for locus in method.loci:
                    if locus.name in seen_loci:
                        # Uncomment this to restrict each locus to one Conf object
                        #raise RuntimeError('locus '+locus.name+' is in more than one AlnConf objects')
                        pass
                    else:
                        seen_loci.append(locus.name)
                        
                    # This will put a fasta alignment in stdout    
                    stdout, stderr = method.command_lines[locus.name]()
                    
                    # This will make a MultipleSeqAlignment from the fasta alignment
                    align = AlignIO.read(StringIO(stdout), "fasta",  alphabet=IUPAC.protein)
                    
                    # This will run pal2nal
                    if method.CDSAlign and locus.feature_type == 'CDS' and locus.char_type == 'dna':
                        
                        # These will check the protein alignment have the same otus and 
                        # the sequences are the same length
                        for seq in align:
                            found = 0
                            for s in method.CDS_in_frame[locus.name]:
                                if s.id == seq.id:
                                    found = 1
                            if found == 0:
                                raise RuntimeError(seq.id + ' is not in the CDS sequences')
                        for s in method.CDS_in_frame[locus.name]:
                            found = 0
                            for seq in align:
                                if s.id == seq.id:
                                    found = 1
                            if found == 0:
                                raise RuntimeError(seq.id + ' is not in the protein sequences')
                        for seq in method.CDS_in_frame[locus.name]:    
                            for prot in align:
                                if prot.id == seq.id:
                                    i = 0
                                    for p in str(prot.seq):
                                        if not p == '-':
                                            i += 1
                                    if not i*3 == len(seq.seq):
                                        raise RuntimeError('nuc and prot seqs have unmatched lengths for '+seq.id)
                                        
                        # This will write an input protein alignment file for pal2nal
                        aln_filename = method.id+'_'+locus.name+'.aln'
                        AlignIO.write(align, aln_filename, 'fasta')
                        
                        # This will run pal2nal
                        cds_filename = method.id+'_CDS_in_frame_'+locus.name+'.fasta'
                        stdout = os.popen(pal2nal+' '+aln_filename+' '+cds_filename + ' -nostderr -codontable %i'%method.codontable).read()
                        
                        # This will make a MultipleSeqAlignment out of the pal2nal output
                        align = AlignIO.read(StringIO(stdout), "clustal",  alphabet=IUPAC.ambiguous_dna)
                        
                        # Maybe this will replace pal2nal
                        #from Bio import CodonAlign
                        #codon_aln = CodonAlign.build(align, method.CDS_in_frame[locus.name])
                        #align = codon_aln
                        
                    # This will list all the temp files written during the analysis
                    method_files = glob.glob(method.id+'_*')
                    
                    # This will get summary statistics of the alignment,
                    # will remove alignments with less than four unique sequences
                    # and will remove undetermined sequences.
                    [summary_lines, num_lines, num_undeter, num_collapsed_aln_seqs] = aln_summary(align)
                    summary = 'Alignment name: '+locus.name+'@'+method.method_name+'\n'
                    for line in summary_lines:
                        summary += line+'\n'
                    if num_lines < 4:
                        line = 'Alignment %s has less than 4 sequences and will be dropped'%(locus.name+'@'+method.method_name)
                        print line
                        summary += line+'\n'
                    elif num_undeter[0] > 0:
                        line = 'Alignment %s has undetermined sequences and will be dropped'%(locus.name+'@'+method.method_name)
                        print line
                        summary += line+'\n'
                    elif num_collapsed_aln_seqs < 4:
                        line = 'Alignment %s has less than 4 unique sequences and will be dropped'%(locus.name+'@'+method.method_name)
                        print line
                        summary += line+'\n'
                        
                    else:
                        # This will palce the MultipleSeqAlignment in the Project
                        self.alignments[locus.name+'@'+method.method_name] = align
                    self.aln_summaries.append(summary)
                    
                # This will measure the running time    
                method.timeit.append(time.time())
                method.timeit.append(method.timeit[2]-method.timeit[1])
                
                # This will delete the temp files
                for f in method_files:
                    os.remove(f)
                self.used_methods[method.method_name] = method


            if self.pickle_name:
                pickle_pj(self, self.pickle_name, track=False)
        
            if __builtin__.git and self.pickle_name:

                comment = ''
                for method in alignment_methods:
                    comment += '%s\n'%(str(method))

                import rpgit
                out, err = rpgit.gitAdd(self.pickle_name)
                undate_git_log(self, out, err)
                cwd = os.getcwd()
                import fnmatch
                matches = []
                for root, dirnames, filenames in os.walk(cwd):
                    for filename in fnmatch.filter(filenames, '*.py'):
                        matches.append(os.path.join(root, filename))
                    for filename in fnmatch.filter(filenames, '*.ipynb'):
                        matches.append(os.path.join(root, filename))
                for match in matches:
                    out, err = rpgit.gitAdd(match)
                    undate_git_log(self, out, err)
                out, err = rpgit.gitCommit(comment)
                undate_git_log(self, out, err)
                
                

    def write_alns(self, id=['feature_id'], format = 'fasta'):
        filenames = []
        if len(self.alignments.keys()) == 0:
            raise IOError('Align the records first')
        else:
            for key in self.alignments:
                aln = self.alignments[key]
                records = []
                for s in aln:
                    qualifiers = get_qualifiers_dictionary(self, s.id)
                    new_id = ""
                    for i in id:
                        if i in qualifiers.keys():
                            new_id += qualifiers[i]+'_'
                    if new_id == "":
                        new_id = s.id
                    else:
                        new_id = new_id[:-1]
                    records.append(SeqRecord(seq=s.seq, id=new_id))
                SeqIO.write(records, key+'_aln.'+format, format)
                filenames.append(key+'_aln.'+format)
        return filenames



    def write_trimmed_alns(self, id=['feature_id'], format = 'fasta'):
        filenames = []
        if len(self.trimmed_alignments.keys()) == 0:
            raise IOError('Align and trimmed the records first')
        else:
            for key in self.trimmed_alignments.keys():
                aln = self.trimmed_alignments[key]
                records = []
                for s in aln:
                    qualifiers = get_qualifiers_dictionary(self, s.id)
                    new_id = ""
                    for i in id:
                        if i in qualifiers.keys():
                            new_id += qualifiers[i]+'_'
                    if new_id == "":
                        new_id = s.id
                    else:
                        new_id = new_id[:-1]
                    records.append(SeqRecord(seq=s.seq, id=new_id))
                SeqIO.write(records, key+'_trimmed_aln.'+format, format)
                filenames.append(key+'_trimmed_aln.'+format)
        return filenames

    def show_aln(self, token, id=['feature_id']):
        aln_obj=None
        if token in self.alignments.keys():
            aln_obj = self.alignments[token]
        elif token in self.trimmed_alignments.keys():
            aln_obj = self.trimmed_alignments[token]
        locus_name = token.split('@')[0]
        char_type = [locus.char_type for locus in self.loci if locus.name == locus_name][0]
        
        records = []
        for s in aln_obj:
            qualifiers = get_qualifiers_dictionary(self, s.id)
            new_id = ""
            for i in id:
                if i in qualifiers.keys():
                    new_id += qualifiers[i]+'_'
            if new_id == "":
                new_id = s.id
            else:
                new_id = new_id[:-1]
            records.append(SeqRecord(seq=s.seq, id=new_id))
            
        title_length = max([len(r.id) for r in records])+2
        
        # colors
        dna_colors = {'a':'green',
                      'A':'green',
                      'T':'red',
                      't':'red',
                      'U':'red',
                      'u':'red',
                      'g':'gray',
                      'G':'gray',
                      'c':'blue',
                      'C':'blue'
                      }
        protein_colors = {'R':'blueviolet',
                          'r':'blueviolet',
                          
                          'K':'cornflowerblue',
                          'k':'cornflowerblue',
                          
                          'E':'red',
                          'e':'red',
                          
                          'D':'crimson',
                          'd':'crimson',
                          
                          'I':'gold',
                          'i':'gold',
                          
                          'L':'yellow',
                          'l':'yellow',
                          
                          'V':'moccasin',
                          'v':'moccasin',
                          
                          'A':'lemonchiffon',
                          'a':'lemonchiffon',
                          
                          'C':'palegreen',
                          'c':'palegreen',
                          
                          'H':'paleturquoise',
                          'h':'paleturquoise',
                          
                          'M':'hotpink',
                          'm':'hotpink',
                          
                          'N':'pink',
                          'n':'pink',
                          
                          'Q':'yellow',
                          'q':'yellow',
                          
                          'F':'darkseagreen',
                          'f':'darkseagreen',
                          
                          'Y':'darkcyan',
                          'y':'darkcyan',
                          
                          'W':'steelblue',
                          'w':'steelblue',
                          
                          
                          'S':'thistle',
                          's':'thistle',
                          
                          'T':'lavender',
                          't':'lavender',
                          
                          'G':'darkgray',
                          'g':'darkgray',
                          
                          'P':'gainsboro',
                          'p':'gainsboro',
                          }
        colors = None
        if char_type == 'dna':
            colors = dna_colors
        elif char_type == 'prot':
            colors = protein_colors
        linelength = (len(records[0].seq)+len(records[0].id)+3)*10
        html_string = '<html><head></head>\n'
        html_string += '<body><pre><font face="Courier New">\n'
        for r in records:
            html_string += r.id.ljust(title_length, '.')
            for p in str(r.seq):
                c = 'white'
                if p in colors.keys():
                    c = colors[p]
                html_string += '<font style="BACKGROUND-COLOR: %s">%s</font>'%(c, p)
            html_string += "<br>"
        html_string +=  '</font></pre></body></html>' 
        
        import webbrowser
        path = os.path.abspath("%s.html"%token)
        url = 'file://' + path
        with open(path, 'w') as f:
            f.write(html_string)
        webbrowser.open_new_tab(url)
        
    
    def tree(self, raxml_methods, bpcomp='default', bpcomp_burnin=0.2, bpcomp_step=10, bpcomp_cutoff=0.01):
        
        if bpcomp == 'default':
            bpcomp = self.defaults['bpcomp']
        
        for raxml_method in raxml_methods:
            raxml_method.timeit.append(time.time())
            raxml_method.platform = platform_report()
            if isinstance(raxml_method, RaxmlConf):
                raxml_method.platform.append('Program and version: '+ raxml_method.cmd +
                                             os.popen(raxml_method.cmd + ' -version').readlines()[2].split('This is ')[1].split(' released')[0])
                raxml_method.platform.append('Program reference: '+
                                             'A. Stamatakis: RAxML Version 8: A tool for Phylogenetic Analysis '+
                                             'and Post-Analysis of Large Phylogenies. In Bioinformatics, 2014.')
            elif isinstance(raxml_method, PbConf):
                p = sub.Popen(raxml_method.cmd+" -v", shell=True, stderr=sub.PIPE, stdout=sub.PIPE)
                raxml_method.platform.append('Program and version: '+p.communicate()[1].splitlines()[-1])
                raxml_method.platform.append('Program reference: '
                                             'N. Lartillot, T. Lepage and S. Blanquart, 2009: PhyloBayes 3: '+
                                             'a Bayesian software package for phylogenetic reconstruction and '+
                                             'molecular dating. Bioinformatics Vol. 25 no. 17.')
            # PROGRAM PLUG
            # NOTE: THIS METHOD SERVES ALL PHYLO PROGRAMS ALTHOUGH THE ITERATOR IS 
            # CALLED raxml_method
            # THIS GETS THE VERSION AND REFERENCE OF THE PROGRAM
            
            # elif isinstance(raxml_method, Conf object name):
            #    p = sub.Popen(raxml_method.cmd+" command that writes version", shell=True, stderr=sub.PIPE, stdout=sub.PIPE)
            #    raxml_method.platform.append('Program and version: '+
            #                           # 1 for stderr, 0 for stdout
            #                           p.communicate()[1].splitlines()[# get the line and split])
            #    raxml_method.platform.append('Program reference: write the reference here')
                
            for trimmed_alignment in raxml_method.command_lines.keys():
                for cline in raxml_method.command_lines[trimmed_alignment]:
                    if isinstance(raxml_method, RaxmlConf):
                        stdout, stderr = cline()
                    elif isinstance(raxml_method, PbConf):
                        sub.call(cline, shell=True)
                        
                    # PROGRAM PLUG
                    # THIS RUNS THE PROGRAM
                    # elif isinstance(raxml_method, Conf Object Name):
                    #     sub.call(cline, shell=True)
                    
                    
                t = None
                if isinstance(raxml_method, RaxmlConf):
                    if raxml_method.preset == 'fa':
                        t = Tree('RAxML_bipartitions.'+raxml_method.id+'_'+trimmed_alignment+'0')
                    elif raxml_method.preset == 'fD_fb':
                        t = Tree('RAxML_bipartitions.'+raxml_method.id+'_'+trimmed_alignment+'1')
                    elif raxml_method.preset == 'fd_b_fb':
                        t = Tree('RAxML_bipartitions.'+raxml_method.id+'_'+trimmed_alignment+'2')
                    elif raxml_method.preset == 'fd_fJ' or raxml_method.preset == 'fF_fJ':
                        tree_filename = 'RAxML_fastTreeSH_Support.'+raxml_method.id+'_'+trimmed_alignment+'1'
                        t = Tree(open(tree_filename,'r').read().replace('[','[&&NHX:support='))
                        
                elif isinstance(raxml_method, PbConf):
                    base_name = "%s_%s"%(raxml_method.id, trimmed_alignment)
                    trees_file = "%s.1.treelist"%base_name
                    trees_per_chain = len(open(trees_file,'r').readlines())
                    # find the number of chains
                    nchains = raxml_method.cline_args['nchain'].split()[0]
                    chain_names = ''
                    for i in range(1,int(nchains)+1):
                        chain_names += "%s.%i "%(base_name, i)
                    chain_names = chain_names[:-1]
                    bpcomp_cline = "%s -c %f -x %i %i %s"%(bpcomp,
                                                           bpcomp_cutoff,
                                                           int(trees_per_chain*bpcomp_burnin),
                                                           int(bpcomp_step),
                                                           chain_names)
                    sub.call(bpcomp_cline, shell=True)
                    t = Tree("bpcomp.con.tre")
                    for l in t:
                        l.support = 0
                        
                # PROGRAM PLUG
                # NOTE: THIS IS SIMPLIFIED. MIGHT WORK WITH SOMETHING LIKE
                # FASTTREE. SEE MORE EXAMPLES ABOVE
                # THIS SECTION MAKES A Tree OBJECT OUT OF THE OUTPUT FILE
                
                # elif isinstance(raxml_method, Conf object name):
                #    base_name = "%s_%s"%(raxml_method.id, trimmed_alignment)
                #    tree_file = "the form of the output file with the %s"%base_name
                #    t = Tree(tree_file)
                    
            
                # This puts the Conf id in the tree 
                for n in t.traverse():
                    n.add_feature('tree_method_id', str(raxml_method.id)+'_'+trimmed_alignment)
                t.dist = 0
                t.add_feature('tree_method_id', str(raxml_method.id)+'_'+trimmed_alignment)
                
                
                # This gets all the metadta from pj.records and puts it
                # on the tree leaves
                loci_names = [i.name for i in  self.loci]       
                concat_names = [c.name for c in self.concatenations]
                if trimmed_alignment.partition('@')[0] in loci_names:
                        
                        for leaf in t:
                            records = self.records
                            feature = ''
                            feature_source = ''
                            record = ''
                            for r in records:
                                if r.id in leaf.name:
                                    record = r
                                    for f in r.features:
                                        if f.type == 'source':
                                            feature_source = f
                                        elif f.qualifiers['feature_id'][0] == leaf.name:
                                            feature = f
                            for a in record.annotations.keys():
                                label = 'annotation_'+a
                                leaf.add_feature(label, record.annotations[a])
                            for f_source_qual in feature_source.qualifiers.keys():
                                label = 'source_'+f_source_qual
                                leaf.add_feature(label, feature_source.qualifiers[f_source_qual][0])
                            for f_qual in feature.qualifiers.keys():
                                leaf.add_feature(f_qual, feature.qualifiers[f_qual][0])
                        for l in t:
                            t.add_feature('tree_id', trimmed_alignment+'@'+raxml_method.method_name)
                            
                        # This puts a Tree object and a string representation of the tree
                        # in the project
                        self.trees[trimmed_alignment+'@'+raxml_method.method_name] = [t,t.write(features=[])]
                        
                elif trimmed_alignment in concat_names:
                        # This does the same as above, but instead of dealing with gene trees
                        # it deals with supermatrix trees
                        s = filter(lambda i: i.name == trimmed_alignment, self.concatenations)[0]
                        for leaf in t:
                            records = self.records
                            feature = ''
                            feature_source = ''
                            record = ''
                            for r in records:
                                for feature in r.features:
                                    if not feature.type == 'source':
                                        qual_dict = get_qualifiers_dictionary(self, feature.qualifiers['feature_id'])
                                        if s.otu_meta in qual_dict.keys() and qual_dict[s.otu_meta] == leaf.name:
                                            for key in qual_dict.keys():
                                                leaf.add_feature(key, qual_dict[key])
                        for l in t:
                            t.add_feature('tree_id', s.name+'@mixed@mixed@'+raxml_method.method_name)
                        self.trees[s.name+'@mixed@mixed@'+raxml_method.method_name] = [t,t.write(features=[])]
                        
            # This times the analysis            
            raxml_method.timeit.append(time.time())
            raxml_method.timeit.append(raxml_method.timeit[2]-raxml_method.timeit[1])
            
            # This deletes temp files
            if not raxml_method.keepfiles:
                for file_name in os.listdir(os.curdir):
                            if raxml_method.id.partition('_')[0] in file_name:
                                os.remove(file_name)
            self.used_methods[raxml_method.method_name] = raxml_method

        if self.pickle_name:
            pickle_pj(self, self.pickle_name, track=False)
        
        if __builtin__.git and self.pickle_name:
            
            comment = ''
            for raxml_method in raxml_methods:
                comment += '%s\n'%(str(raxml_method))
            
            import rpgit
            out, err = rpgit.gitAdd(self.pickle_name)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)

    def clear_tree_annotations(self):
        for tree in self.trees.keys():
            t = Tree(self.trees[tree][1])
            t.dist = 0
            self.trees[tree][0] = t



    def write_nexml(self, output_name):
        D = dendropy.DataSet()
        tree_list = []
        
        loci_names = []
        for locus in self.loci:
            loci_names.append(locus.name)
        
        for tree_name in self.trees.keys():
            #get aligned and trimmd aligned sequences as leaf features
            t = self.trees[tree_name][0]
            for l in t:
                loc_name = tree_name.split('@')[0]
                trim_aln_name = tree_name.rpartition('@')[0]
                aln_name = None
                if loc_name in loci_names:
                    aln_name = tree_name.rsplit('@',2)[0]
                else:
                    trim_aln_name = trim_aln_name.split('@')[0]
                
                otu_feature = 'feature_id'
                if not aln_name: # then it is a concatenation
                    for c in self.concatenations:
                        if c.name == loc_name:
                            otu_feature = c.otu_meta
                            
                if aln_name: # Then it is a locus
                    leaf_feature_value = getattr(l, otu_feature)
                    alignment = self.alignments[aln_name]
                    for record in alignment:
                        if record.id == leaf_feature_value:
                            l.add_feature('aligned_sequence',str(record.seq))
                t_aln = self.trimmed_alignments[trim_aln_name]
                    
                leaf_feature_value = getattr(l, otu_feature)
                for record in t_aln:
                    if record.id == leaf_feature_value:
                        l.add_feature('aligned_trimmed_sequence',str(record.seq))
                    
            tree_string = self.trees[tree_name][0].write(features=[])
            tree = dendropy.Tree()
            tree.read_from_string(tree_string, schema='newick', extract_comment_metadata = True)
            tree_list.append(tree)
        TL = dendropy.TreeList(tree_list)    
        D.add_tree_list(TL)
            
        D.write_to_path(
            output_name,
            'nexml',
            suppress_annotations=False,
            annotations_as_nhx=False,
            exclude_trees=False)

    def write_phyloxml(self, filename):

        phyloxmls = []

        from Bio import Phylo
        from StringIO import StringIO

        for token in self.trees.keys():
            tree = Phylo.read(StringIO(self.trees[token][0].write(format=2)), 'newick')
            xmltree = tree.as_phyloxml()
            xmltree.name = token

            aln= None
            trimaln= None

            alnlookup = None
            trimalnlookup = None

            if token.split('@')[0] in [l.name for l in self.loci]:

                aln = self.alignments['%s@%s'%(token.split('@')[0], token.split('@')[1])]
                alnlookup = dict((rec.id, str(rec.seq)) for rec in aln)

                trimaln = self.trimmed_alignments['%s@%s@%s'%(token.split('@')[0],
                                                    token.split('@')[1],
                                                    token.split('@')[2])]
                trimalnlookup = dict((rec.id, str(rec.seq)) for rec in trimaln)

            elif token.split('@')[0] in [c.name for c in self.concatenations]:

                trimaln = self.trimmed_alignments[token.split('@')[0]]
                trimalnlookup = dict((rec.id, str(rec.seq)) for rec in trimaln)

            else:
                raise RuntimeError('No locus or concatenation for tree %s'%token)

            for clade in xmltree.get_terminals():
                typ = 'dna'
                if (alnlookup and
                    [l for l in self.loci if l.name == token.split('@')[0]][0].char_type == 'prot'):
                    typ = 'protein'
                key = clade.name
                feautre_id = key
                if not alnlookup:
                    feature_id = (self.trees[token][0]&key).feature_id

                dbtype = 'NCBI'
                if alnlookup and 'denovo' in key:
                    dbtype= 'denovo'
                elif not alnlookup:
                    dbtype= 'otu'

                accession = Phylo.PhyloXML.Accession(key, dbtype)
                mol_seq = Phylo.PhyloXML.MolSeq(trimalnlookup[key], is_aligned=True)
                sequence = Phylo.PhyloXML.Sequence(type=typ,
                                                   accession=accession,
                                                   mol_seq=mol_seq,
                                                   annotations=[Phylo.PhyloXML.Annotation(evidence='trimmed')])

                if alnlookup:
                    mol_seq = Phylo.PhyloXML.MolSeq(alnlookup[key], is_aligned=True)
                    sequence = Phylo.PhyloXML.Sequence(type=typ,
                                                       accession=accession,
                                                       mol_seq=mol_seq)


                clade.sequences.append(sequence)

            others = [Phylo.PhyloXML.Other(tag='trimmedaln',
                                           attributes={'treename':token},
                                           value=trimaln.format('phylip-relaxed'))]
            if aln:
                others.append(Phylo.PhyloXML.Other(tag='aln',
                                                   attributes={'treename':token},
                                                   value=aln.format('phylip-relaxed')))
            xmltree.other = others 

            phyloxmls.append(xmltree)  

        Phylo.write(phyloxmls, filename, 'phyloxml')
        

    def annotate(self, fig_folder,
    
                 root_meta,
                 root_value,
    
                 leaf_labels_txt_meta,
                 leaf_node_color_meta=None,
                 leaf_label_colors=None,
                 ftype='Verdana',
                 fsize=10,
    
                 node_bg_meta=None,
                 node_bg_color=None,
                 
                 node_support_dict=None,
                 support_bullet_size=5,
                 
                 heat_map_meta = None, #list
                 heat_map_colour_scheme=2,
                 
                 pic_meta=None,
                 pic_paths=None,
                 pic_w=None,
                 pic_h=None,
                 
                 multifurc=None,
                 branch_width=2,
                 branch_color='DimGray',
                 
                 scale = 1000,
                 
                 html = None
                 ): 
            
            stdout = sys.stdout
            if html:
                sys.stdout = open(html, 'wt')
                
            print '<html>'
            ts = TreeStyle()
            ts.show_leaf_name = False
            ts.scale = scale
            if node_support_dict:
                ts.legend_position=1
                ts.legend.add_face(TextFace('Node support: ', fsize=10), column=0)
                i = 1
                for color in sorted(node_support_dict.items(),key=lambda i: i[1][0], reverse=True):
                    ts.legend.add_face(CircleFace(radius = 4, color = color[0]), column=i)
                    i +=1 
                    ts.legend.add_face(TextFace(' '+str(node_support_dict[color[0]][0])+'-'+str(node_support_dict[color[0]][1]),
                                                fsize=10), column=i)
                    i += 1
                
            for tree in self.trees.keys():
                                       
                # set outgroup leaves, labels and label colors
                outgroup_list = []
                all_heatmap_profile_values = []
                leaves_for_heatmap = []
                
                for leaf in self.trees[tree][0]:
                    qualifiers_dictionary = get_qualifiers_dictionary(self, leaf.feature_id)
                    leaf_label = ''
                    for meta in leaf_labels_txt_meta:
                        leaf_label += qualifiers_dictionary[meta]+' '
                    leaf_label = leaf_label[:-1]
                    fgcolor = 'black'
                    if leaf_label_colors:
                        for colour_name in leaf_label_colors.keys():
                            if colour_name in qualifiers_dictionary[leaf_node_color_meta]:
                                fgcolor = leaf_label_colors[colour_name]
                    leaf_face = TextFace(leaf_label, fgcolor=fgcolor, ftype=ftype, fsize=fsize)
                    leaf.add_face(leaf_face,0)
                    if not root_value == 'mid' and root_meta in qualifiers_dictionary.keys() and root_value in qualifiers_dictionary[root_meta]:
                        outgroup_list.append(leaf)
                        
                    if heat_map_meta:
                        include = True
                        for i in heat_map_meta:
                            if not i in qualifiers_dictionary:
                                include = False
                        if include:
                            profile = []
                            deviation = []
                            for meta in heat_map_meta:
                                if meta in qualifiers_dictionary.keys():
                                    profile.append(float(qualifiers_dictionary[meta]))
                                    all_heatmap_profile_values.append(float(qualifiers_dictionary[meta]))
                                    deviation.append(0.0)
                            leaf.add_features(profile=profile)
                            leaf.add_features(deviation=deviation)
                            leaves_for_heatmap.append(leaf)
                    if pic_meta:
                        leaf_value = qualifiers_dictionary[pic_meta]
                        if leaf_value in pic_paths:
                            pic_face = ImgFace(pic_paths[leaf_value], width=pic_w, height=pic_h)
                            leaf.add_face(pic_face, 1)
                for leaf in leaves_for_heatmap:
                    leaf.add_face(ProfileFace(max_v=float(max(all_heatmap_profile_values)),
                                              min_v=float(min(all_heatmap_profile_values)), 
                                              center_v=float(float(max(all_heatmap_profile_values)+min(all_heatmap_profile_values))/2),
                                              width=50, height=30,
                                              style='heatmap',
                                              colorscheme=heat_map_colour_scheme),
                                    column=1, position="aligned")
                        
                        
                #set outgroup
                if outgroup_list == ['mid']:
                    try:
                        R = self.trees[tree][0].get_midpoint_outgroup()
                        self.trees[tree][0].set_outgroup(R)
                        print 'rooting tree '+tree+' at midpoint'
                    except:
                        print 'root in '+tree+' already set correctly?'
                    
                elif len(outgroup_list) == 1:
                    try:
                        self.trees[tree][0].set_outgroup(outgroup_list[0])
                    except:
                        print 'root in '+tree+' already set correctly?'
                elif len(outgroup_list) > 1:
                    try:
                        R = self.trees[tree][0].get_common_ancestor(outgroup_list)
                        self.trees[tree][0].set_outgroup(R)
                    except:
                        print 'root in '+tree+' already set correctly?'
                elif len(outgroup_list)==0:
                    try:
                        R = self.trees[tree][0].get_midpoint_outgroup()
                        self.trees[tree][0].set_outgroup(R)
                        print 'rooting tree '+tree+' at midpoint'
                    except:
                        print 'root in '+tree+' already set correctly?'
    
                # ladderize
                self.trees[tree][0].ladderize()
            
                ns = NodeStyle()
                ns['size'] = 0
                ns['fgcolor'] = 'black'
                ns['vt_line_width'] = branch_width
                ns['hz_line_width'] = branch_width
                ns['hz_line_color'] = branch_color
                ns['vt_line_color'] = branch_color
                for n in self.trees[tree][0].traverse():
                    n.set_style(ns)
                self.trees[tree][0].set_style(ns)
            
                if multifurc:
                    for n in self.trees[tree][0].traverse():
                        if n.support < multifurc and not n.is_leaf():
                            n.delete()
    
                # node bg colors
                if node_bg_color:
                    for key in node_bg_color.keys():
                        for node in self.trees[tree][0].get_monophyletic(values=[key], target_attr=node_bg_meta):
                            ns = NodeStyle(bgcolor=node_bg_color[key])
                            ns['size']=0
                            ns['fgcolor']='black'
                            ns['vt_line_width'] = branch_width
                            ns['hz_line_width'] = branch_width
                            ns['hz_line_color'] = branch_color
                            ns['vt_line_color'] = branch_color
                            node.set_style(ns)
                
    
                # node support
                if node_support_dict:
                    for node in self.trees[tree][0].traverse():
                        for key in node_support_dict.keys():
                            if (node.support <= node_support_dict[key][0] and
                                node.support > node_support_dict[key][1]):
                                node.add_face(CircleFace(radius = support_bullet_size,
                                                         color = key),column=0, position = "float")             
                    
                self.trees[tree][0].render(fig_folder + "/"+self.trees[tree][0].get_leaves()[0].tree_method_id+'.png',w=1000, tree_style=ts)
                print('<A href='+
                       fig_folder + "/" + self.trees[tree][0].get_leaves()[0].tree_method_id+'.png'+
                       '>'+self.trees[tree][0].get_leaves()[0].tree_method_id+
                       '</A><BR>')
            print '</html>'
            print fig_folder
            sys.stdout = stdout
     
            
    def trim(self, list_of_Conf_objects, cutoff=0):
        for m in list_of_Conf_objects:
            m.timeit.append(time.time())
            m.platform = platform_report()
            
            m.platform.append('Program and version: '+
                               os.popen(m.cmd + ' --version').readlines()[1].rstrip())
            m.platform.append('Program reference: '+
                              'Salvador Capella-Gutierrez; Jose M. Silla-Martinez; Toni Gabaldon. trimAl: '+
                              'a tool for automated alignment trimming in large-scale '+
                              'phylogenetic analyses. Bioinformatics 2009 25: 1972-1973.')
            
            # In this part, I need to take what I can out of the if isinstance
            # to facilitate addition of new Conf objects. It can be done like this
            # but there will be redundancy
            
            if isinstance(m, TrimalConf):
                import subprocess as sub
                for aln in m.command_lines.keys():
                    p = sub.Popen(m.command_lines[aln], shell=True, stdout=sub.PIPE)
                    stdout, stderr = p.communicate()
                    #stdout = os.system(m.command_lines[aln]).stdout
                    alphabet = IUPAC.ambiguous_dna
                    locus_name = aln.split('@')[0]
                    for locus in self.loci:
                        if locus.name == locus_name and locus.char_type == 'prot':
                            alphabet = IUPAC.protein
                    align = AlignIO.read(StringIO(stdout), "fasta",  alphabet=alphabet)
                    [summary_lines, num_lines, num_undeter, num_collapsed_aln_seqs] = aln_summary(align,
                                                                                                  cutoff=cutoff)
                    summary = 'Alignment name: '+aln+'\n'
                    for line in summary_lines:
                        summary += line+'\n'
                    if num_lines < 4:
                        line = 'Alignment %s has less than 4 sequences and will be dropped'%aln
                        print line
                        summary += line+'\n'
                    elif num_undeter[0] > 0:
                        line = 'Alignment %s has undetermined sequences (%i bp or less) which will be dropped: %s'%(aln, cutoff+1, num_undeter[1])
                        print line
                        summary += line+'\n'
                        records_wo_undeter = []
                        for record in align:
                            if not record.id in num_undeter[1]:
                                records_wo_undeter.append(record)
                        align =  MultipleSeqAlignment(records_wo_undeter)
                        self.trimmed_alignments[aln] = align
                    elif num_collapsed_aln_seqs < 4:
                        line = 'Alignment %s has less than 4 unique sequences and will be dropped'%aln
                        print line
                        summary += line+'\n'
                    else:
                        self.trimmed_alignments[aln] = align
                    self.aln_summaries.append(summary)
            for file_name in os.listdir(os.curdir):
                if m.id.partition('_')[0] in file_name:
                    os.remove(file_name)
            m.timeit.append(time.time())
            m.timeit.append(m.timeit[2]-m.timeit[1])
            self.used_methods[m.method_name] = m
            
        if self.pickle_name:
            pickle_pj(self, self.pickle_name)
        
        if __builtin__.git and self.pickle_name:
            
            comment = ''
            for m in list_of_Conf_objects:
                comment += '%s\n'%(str(m))
            
            import rpgit
            out, err = rpgit.gitAdd(self.pickle_name)
            undate_git_log(self, out, err)
            cwd = os.getcwd()
            import fnmatch
            matches = []
            for root, dirnames, filenames in os.walk(cwd):
                for filename in fnmatch.filter(filenames, '*.py'):
                    matches.append(os.path.join(root, filename))
                for filename in fnmatch.filter(filenames, '*.ipynb'):
                    matches.append(os.path.join(root, filename))
            for match in matches:
                out, err = rpgit.gitAdd(match)
                undate_git_log(self, out, err)
            out, err = rpgit.gitCommit(comment)
            undate_git_log(self, out, err)

    def report_seq_stats(self):        
        if len(self.records_by_locus.keys())>0:

            # This will make a list of seq length for each locus. Seq length are calced
            # using the record.seq in 'pj.records_by_locus'. 'pj.records_by_locus is a
            # dict with loci names as keys, and lists of SeqReocrd objects as values
            lengths_dict = {}
            for locus_name in self.records_by_locus.keys():
                lengths_dict[locus_name] = []
                for record in self.records_by_locus[locus_name]:
                    lengths_dict[locus_name].append(len(record.seq))
                    
            print "Distribution of sequence lengths".title()
            draw_boxplot(lengths_dict, 'Seq length (bp)', 'inline')
            
            
            for stat in ['GC_content']:#, 'nuc_degen_prop', 'prot_degen_prop']:
                title = 'Distribution of sequence statistic \"'+stat+'\"'
                print title.title()
                # This will make a dict with loci as keys and a list of stat values as
                # dict values.
                stat_dict = {}
                ylabel = 'GC ontent (%)'
                if not stat == 'GC_content':
                    ylabel = 'Ambiguous positions (prop)'
                for locus_name in self.records_by_locus.keys():
                    stat_dict[locus_name] = []
                    for i in self.records_by_locus[locus_name]:
                        for record in self.records:
                            for feature in record.features:
                                if feature.qualifiers['feature_id'][0] == i.id:
                                    if stat in feature.qualifiers.keys():
                                        stat_dict[locus_name].append(float(feature.qualifiers[stat][0]))
                
                draw_boxplot(stat_dict, ylabel, 'inline')
                
    ##################################               
    # Project methods to fetch objects
    ##################################
    
    def ft(self, token):
        
        """
        Will fetch the tree object which has the token in 
        its key, as long as there is only one
        """
        
        # check how many tree keys match the token
        keys = [key for key in self.trees.keys() if token in key]
        if len(keys) > 1:
            raise IOError("The token %s was found in more then one tree key: %s"
                           %(token, str(keys)))
        elif len(keys) == 0:
            raise IOError("The token %s was not found in any tree key"
                           %token)
        elif len(keys) == 1:
            print "returning tree object %s"%keys[0]
            return Tree(self.trees[keys[0]][1])
        
        
        
    def fa(self, token):
        
        """
        Will fetch the alignment object which has the token in 
        its key, as long as there is only one
        """
        
        from StringIO import StringIO
        
        # check how many aln keys match the token
        keys = [key for key in self.alignments.keys() if token in key]
        if len(keys) > 1:
            raise IOError("The token %s was found in more then one alignment key: %s"
                          %(token, str(keys)))
        elif len(keys) == 0:
            raise IOError("The token %s was not found in any alignment key"
                          %token)
        elif len(keys) == 1:
            print "returning alignment object %s"%keys[0]
            return AlignIO.read(StringIO(self.alignments[keys[0]].format('fasta')), 'fasta')        
        
        
    def fta(self, token):
        
        """
        Will fetch the trimmed alignment object which has the token in 
        its key, as long as there is only one
        """
        
        from StringIO import StringIO
        
        # check how many trimmed aln keys match the token
        keys = [key for key in self.trimmed_alignments.keys() if token in key]
        if len(keys) > 1:
            raise IOError("The token %s was found in more then one trimmed alignment key: %s"
                          %(token, str(keys)))
        elif len(keys) == 0:
            raise IOError("The token %s was not found in any trimmed alignment key"
                          %token)
        elif len(keys) == 1:
            print "returning trimmed alignment object %s"%keys[0]
            return AlignIO.read(StringIO(self.trimmed_alignments[keys[0]].format('fasta')), 'fasta')            
        
        
    def fr(self, locus_name, filter=None):
        
        """
        Will fetch the record objects of the specified locus, 
        as long as there is at least one.
        filter should be a list of lists. Every (sub)list is
        a pair of qualifier and value. If filter is specified,
        only records that have all the specified values in the
        specified qualifiers will be kept.
        """
        
        # check how many record keys match the token
        keys = [key for key in self.records_by_locus.keys() if locus_name in key]
        if len(keys) > 1:
            raise IOError("The locus name %s fits more then one locus: %s"
                          %(locus_name, str(keys)))
        elif len(keys) == 0:
            raise  IOError("The locus %s was not found"
                           %locus_name)
        elif len(keys) == 1:
            records = []
            if filter:
                for r in self.records_by_locus[keys[0]]:
                    qualifiers = get_qualifiers_dictionary(self, r.id)
                    get = True
                    for f in filter:
                        if not (f[0] in qualifiers.keys() and qualifiers[f[0]] == f[1]):
                            get = False
                    if get:
                        records.append(r) 
            else:
                for r in self.records_by_locus[keys[0]]:
                    records.append(r)
            print "returning records list of locus %s and filter %s"%(keys[0], str(filter))
            return records
        
   
    def propagate_metadata(self):
        for t in self.trees.keys():
            for l in self.trees[t][0]:
                feature_id = l.feature_id
                record_id = feature_id.rpartition('_')[0]
                record = [r for r in self.records if r.id == record_id][0]
                annotations = record.annotations
                source_qualifiers = [f for f in record.features if f.type == 'source'][0].qualifiers
                feature_qualifiers = [f for f in record.features if f.qualifiers['feature_id'][0] == feature_id][0].qualifiers
                for i in annotations:
                    l.add_feature("annotations_%s"%i,type_to_single_line_str(annotations[i]))
                for i in source_qualifiers:
                    l.add_feature("source_%s"%i,type_to_single_line_str(source_qualifiers[i]))
                for i in feature_qualifiers:
                    l.add_feature(i,type_to_single_line_str(feature_qualifiers[i]))
            self.trees[t][1] = self.trees[t][0].write(features=[])

            
##############################################################################################
if False:
    """Tools for loci explorations in a GenBank File"""
##############################################################################################

programspath = ""

def list_loci_in_genbank(genbank_filename, control_filename, loci_report = None):
    """
    Takes a genbank file, returns a loci csv file and a list of loci and their counts. The latter goes
    either to stdout or to a file.
    
    >>> list_loci_in_genbank("test-data/test.gb", "test-data/temp_loci.csv", loci_report = "test-data/temp_loci.txt")
    >>> assert open("test-data/temp_loci.csv",'r').read() == open("test-data/test_loci.csv",'r').read() 
    >>> import os
    >>> os.remove("test-data/temp_loci.csv")
    """
    
    stdout = sys.stdout
    if  loci_report: 
        sys.stdout = open(loci_report, 'w')
    
    genbank_synonyms = gb_syn.gb_syn()
    
    # Open GenBank file
    MelPCgenes = open(genbank_filename, 'rU')
   
    gene_dict = {} #set up a gene_dict dictionary
   
    # For each record
    
    for record in SeqIO.parse(MelPCgenes, 'genbank') :
        # Look at all features for this record
        for feature in record.features:
         
            # If it's a CDS or rRNA...
            if feature.type == 'CDS' or feature.type == 'rRNA':
                # If it contains some attribute called 'gene' save that
                if 'gene' in feature.qualifiers:
                    geneName = feature.qualifiers['gene'][0]
                    geneName = geneName.replace(',','_')
                    geneName = geneName.replace('/','_')

                    if feature.type+','+geneName in gene_dict:
                        gene_dict[feature.type+','+geneName]+=1
                    else:    
                        gene_dict[feature.type+','+geneName]=1
                    #print(geneName)

                # Else if it contains a 'product' qualifier
                elif 'product' in feature.qualifiers:
                    geneName = feature.qualifiers['product'][0]
                    geneName = geneName.replace(',','_')
                    geneName = geneName.replace('/','_') 

                    if feature.type+','+geneName in gene_dict:
                        gene_dict[feature.type+','+geneName]+=1
                    else:    
                        gene_dict[feature.type+','+geneName]=1
                    #print(geneName)

                else:
                    print 'Could not find either gene or product in '+record.id
                    #print feature.qualifiers

       
    #sorting happens via a list
   
    sorted_gene_names = gene_dict.items()
    sorted_gene_names.sort(key = lambda i: i[0].lower())
    control_file_lines = {}
   
   
    print('\n' + "There are " + str(len(sorted_gene_names)) + " gene names (or gene product names) detected")
    print("----------------------------------")
    print("Gene and count sorted by gene name")
    print("----------------------------------")
   
    for key, value in sorted_gene_names:
        #print key, value
        print(str(value) +" instances of " + key)
        feature_type = key.split(',')[0]
        alias = key.split(',')[1]
        gen_group = None
        for group in genbank_synonyms:
            if alias in group:
                gen_group = group
        if gen_group:
            if gen_group[0].replace(' ','_') in control_file_lines.keys():
                control_file_lines[gen_group[0].replace(' ','_')].append(alias)
            else:
                control_file_lines[gen_group[0].replace(' ','_')] = [feature_type, alias]
        else:
            name = alias.replace(' ','_').replace('/','_')
            control_file_lines[name] = [feature_type, alias]
                    
    control_file_handle = open(control_filename, 'wt')
    for line in sort(control_file_lines.keys()):  
        control_file_handle.write('dna,%s,%s'%(control_file_lines[line][0],line))
        for a in control_file_lines[line][1:]: 
            control_file_handle.write(',%s'%a)
        control_file_handle.write('\n')
                            
    control_file_handle.close()                 
   
    print("-------------------------------")
    print("Gene and count sorted by counts")
    print("-------------------------------")
    sorted_gene_names.sort(key = lambda i: int(i[1]), reverse=True)
    for key, value in sorted_gene_names:
        #print key, value
        print(str(value) +" instances of " + key)
    sys.stdout = stdout
    return   

AlnConfMethodsSection=\
"""\n
==============================
Core Methods section sentence:
==============================
The dataset(s) %s were aligned using the program %s [1].

Reference:
%s
"""

AlnConfPalNalMethodsSection=\
"""\n
==============================
Core Methods section sentence:
==============================
The dataset(s) %s were first aligned at the protein level using the program %s [1].
The resulting alignments served as guides to codon-align the DNA sequences using %s [2].

Reference:
[1]%s
[2]%s
"""

# ALIGNMENT PROGRAM PLUG
# AS ABOVE, A STRING IS NEEDED WITH PLACE HOLDERS FOR THE LOCI, THE PROGRAM+VERSION
# AND FOR THE REFERENCE.
# ANOTHER STRING IS REQUIRED THAT ALSO INCLUDES PAL2NAL
    
##############################################################################################
class AlnConf:
##############################################################################################
    
    """
    >>> coi = Locus('dna', 'CDS', 'coi', ['cox1','COX1','coi','COI','CoI'])
    >>> lsu = Locus('dna', 'rRNA', '28S', ['28s','28S','LSU rRNA','28S ribosomal RNA','28S large subunit ribosomal RNA'])
    >>> pj = Project([coi, lsu], git=False)
    
    # cline_str = muscle = AlnConf(pj, method_name='MuscleDefaults',
    #                                      cmd='muscle', program_name='muscle',
    #                                     cline_args=dict())
    """
    
    def __init__(self, pj, method_name='mafftDefault', CDSAlign=True, codontable=1, program_name='mafft',
                 cmd='mafft', loci='all',
                 cline_args=dict()):
        
        if pj.records_by_locus == {}:
            pj.extract_by_locus()
            
        self.id = str(random.randint(10000,99999))+str(time.time())
        self.method_name=method_name
        self.CDSAlign=CDSAlign
        self.program_name=program_name
        self.loci = pj.loci
        if not loci == 'all':
            self.loci = []
            for locus_name in loci:
                for locus in pj.loci:
                    if locus_name == locus.name:
                        self.loci.append(locus)
        mutable_loci_list = []
        removed_loci = []
        for locus in self.loci: 
            if len(pj.records_by_locus[locus.name]) < 4:
                removed_loci.append(locus.name)
            else:
                mutable_loci_list.append(locus)
                
        if len(removed_loci) > 0:
            print "These loci have less than 4 sequences and will be dropped from this conf object. Don't use them in a concatenation:\n%s\n\n"%removed_loci
        self.loci = mutable_loci_list
        self.CDS_proteins = {}
        self.CDS_in_frame = {}
        self.codontable = codontable 
        self.aln_input_strings = {}
        self.command_lines = {}
        self.timeit = [time.asctime()]
        self.platform = []
        self.cline_args = cline_args
        self.cmd = cmd
        if not program_name == 'mafft' and cmd == 'mafft':
            self.cmd = pj.defaults[program_name]
        elif program_name == 'mafft' and cmd != 'mafft' and not 'mafft' in cmd:
            self.cmd = pj.defaults['mafft']
            
        # make defalut input files

        for key in pj.records_by_locus.keys():
            if key in [l.name for l in self.loci]:
                SeqIO.write(pj.records_by_locus[key], self.id+'_'+key+'.fasta', 'fasta')    
        for locus in self.loci:
            # put default input file filename and string in the AlnConf object
            input_filename=self.id+'_'+locus.name+'.fasta'
            self.aln_input_strings[locus.name] = [open(input_filename,'r').read()]
            # If CDS and CDSAlign, prepare reference protein input file and in frame CDS input file
            if locus.feature_type == 'CDS' and locus.char_type == 'dna' and self.CDSAlign: 
                self.CDS_proteins[locus.name] = []
                self.CDS_in_frame[locus.name] = []
                for record in pj.records:
                    for feature in record.features:
                        if (feature.type == 'CDS' and 'gene' in feature.qualifiers.keys() and
                            feature.qualifiers['gene'][0] in locus.aliases and 
                            feature.qualifiers['feature_id'][0] in [f.id for f in pj.records_by_locus[locus.name]]):
                            S = feature.extract(record.seq)
                            # Make in-frame CDS input file seq start in frame
                            if 'codon_start' in feature.qualifiers.keys():
                                i = feature.qualifiers['codon_start'][0]
                                if i > 1:
                                    S = S[(int(i)-1):]
                            # Make in-frame CDS input file seq end in frame
                            if len(S)%3 == 1:
                                S = S[:-1]
                            elif len(S)%3 == 2:
                                S = S[:-2]  
                            # make protein input file seq
                            if not 'translation' in feature.qualifiers.keys():
                                raise IOError("Feature %s has no 'translation' qualifier"%
                                              feature.qualifiers['feature_id'][0])
                            P = Seq(feature.qualifiers['translation'][0], IUPAC.protein)
                            # Remove 3' positions that are based on partial codons
                            while len(P)*3 > len(S):
                                P = P[:-1]
                            # remove complete termination codon
                            if (len(S)/3)-1 == len(P):
                                S = S[:-3]
                            # make in frame cds record
                            feature_record = SeqRecord(seq = S, id = feature.qualifiers['feature_id'][0],
                                                               description = '')
                            # put it in the object
                            self.CDS_in_frame[locus.name].append(feature_record)
                            # make protein record
                            feature_record = SeqRecord(seq = P, id = feature.qualifiers['feature_id'][0],
                                                       description = '')
                            # Put the protein records in the AlnConf object
                            self.CDS_proteins[locus.name].append(feature_record)
                           
                                    
                # check same number of prot and cds objects                    
                if len(pj.records_by_locus[locus.name]) > len(self.CDS_proteins[locus.name]):
                    raise RuntimeError('For the CDS locus '+locus.name+': more nuc seqs than prot seqs.'+
                                       ' You may miss a \'translate\' or \'gene\' qualifier in some of '+
                                       'the features.')
                elif len(pj.records_by_locus[locus.name]) < len(self.CDS_proteins[locus.name]):
                    raise RuntimeError('For the CDS locus '+locus.name+': less nuc seqs than prot seqs.'+
                                       ' You may miss a \'translate\' or \'gene\' qualifier in some of '+
                                       'the features.')
                unmatched = []
                for record in self.CDS_in_frame[locus.name]:
                    for prot in self.CDS_proteins[locus.name]:
                        if prot.id == record.id:
                            if not len(prot.seq)*3 == len(record.seq):
                                unmatched.append(record.id)
                unmatched_string = ''
                if len(unmatched) > 0:
                    for u in unmatched:
                        unmatched_string += u+' '
                    raise RuntimeError('The following CDS/protein pairs are unmatched: '+unmatched_string)
                    
 
                SeqIO.write(self.CDS_in_frame[locus.name],
                            self.id+'_CDS_in_frame_'+locus.name+'.fasta', 'fasta')
                input_filename2=self.id+'_CDS_in_frame_'+locus.name+'.fasta'
                SeqIO.write(self.CDS_proteins[locus.name],
                            self.id+'_CDS_proteins_'+locus.name+'.fasta', 'fasta')
                input_filename=self.id+'_CDS_proteins_'+locus.name+'.fasta'
                self.aln_input_strings[locus.name][0] = [open(input_filename,'r').read(),
                                                         open(input_filename2,'r').read()]
            cline = dict(dict(input=input_filename), **cline_args)
            if self.program_name == 'mafft':
                self.command_lines[locus.name] = MafftCommandline(cmd=self.cmd)
            elif self.program_name == 'muscle':
                self.command_lines[locus.name] = MuscleCommandline(cmd=self.cmd)
            # PROGRAM PLUG
            # NOTE: MAFFT AND MUSCLE GET FASTA. IF THE NEW PROGRAM GETS SOMETHING
            # ELSE, SOME WORK IS REQUIRED ABOVE, e.g. condition to choose the format
            # THIS WRITE THE CLINE AND PLACES IN PROJECT. 
            # elif self.program_name == 'some program':
            #    self.command_lines[locus.name] = some_program_cline_object(cmd=self.cmd)
            # This assumes Bio.Applications style cline object
            for c in cline.keys():
                self.command_lines[locus.name].__setattr__(c,cline[c])
            print str(self.command_lines[locus.name])
            
            
            
    def __str__(self):
        loci_string = ''
        for n in [i.name for i in self.loci]:
            loci_string += n+','
        loci_string = loci_string[:-1]
        command_lines = ''
        for i in self.command_lines.keys():
            command_lines += i+': '+str(self.command_lines[i])+'\n'
        date = str(self.timeit[0])
        execution = '[This was not executed yet]'
        if len(self.timeit) > 1:
            execution = str(self.timeit[3])
        plat = '[This was not executed yet]'
        if len(self.platform) > 0:
            plat = str(self.platform).replace(",",'\n').replace(']','').replace("'",'').replace('[','')
        output =  str("AlnConf named %s with ID %s\n"+         
                "Loci: %s \n"+       
                "Created on: %s\n"+
                "Commands:\n"+
                "%s\n"+
                "Environment:\n"+    
                "%s\n"+
                "execution time:\n"+
                "%s\n")%(self.method_name, str(self.id), loci_string, date, command_lines, plat, execution) 
        if len(self.platform) > 0 and self.CDSAlign:
            prog = '[This was not executed yet]'
            pal = '[This was not executed yet]'
            progref = '[This was not executed yet]'
            palref = '[This was not executed yet]'
            if self.platform[-1].startswith('Program reference:'):
                prog, pal = self.platform[-2].split(':')[1].strip().rstrip().split('\n')
                progref, palref = self.platform[-1].split('Program reference:')[1].strip().rstrip().split('\n')
            output += AlnConfPalNalMethodsSection%(type_to_single_line_str([l.name for l in self.loci]),
                                                  prog, pal, progref, palref)
        elif len(self.platform) > 0:
            prog = '[This was not executed yet]'
            progref = '[This was not executed yet]'
            if self.platform[-1].startswith('Program reference:'):
                prog = self.platform[-2].split(':')[1].strip().rstrip()
                progref = self.platform[-1].split('Program reference:')[1].strip().rstrip()
            output += AlnConfMethodsSection%(type_to_single_line_str([l.name for l in self.loci]),
                                             prog, progref)
        
        return output
    
TrimalConfMethodsSection=\
"""\n
==============================
Core Methods section sentence:
==============================
The alignment(s) %s were trimmed using the program %s [1].

Reference:
%s
"""
##############################################################################################
class TrimalConf:
##############################################################################################
    def __init__(self, pj, method_name='gappyout', program_name='trimal',
                 cmd='default', alns='all', trimal_commands=dict(gappyout=True)):
        
        if len(pj.alignments) == 0:
            raise RuntimeError("No sequence alignments found")
        self.id = str(random.randint(10000,99999))+str(time.time())
        self.method_name=method_name
        self.program_name=program_name
        self.alignments = pj.alignments
        if not alns == 'all':
            self.alignments = {}
            for aln_name in alns:
                if aln_name in pj.alignments.keys():
                    self.alignments[aln_name] = pj.alignments[aln_name]
        self.command_lines = {}
        self.timeit = [time.asctime()]
        self.platform = []
        self.cline_args = trimal_commands
        self.cmd = cmd
        if cmd == 'default':
            self.cmd = pj.defaults['trimal']
        irelevant = ['out', 'clustal', 'fasta', 'nbrf', 'nexus', 'mega',
                     'phylip3.2', 'phylip', 'sgt', 'scc', 'sct', 'sfc',
                     'sft','sident']
        for aln in self.alignments:
            input_filename = self.id+'_'+aln+'.fasta'
            AlignIO.write(self.alignments[aln], input_filename,'fasta')
            self.command_lines[aln] = "%s -in %s "%(self.cmd, input_filename)
            for kwd in trimal_commands:
                if kwd in irelevant:
                    warnings.simplefilter('always')
                    warnings.warn("%s is irelevant in this context and will be ignored"%kwd)
                else:
                    if not trimal_commands[kwd] == None and not trimal_commands[kwd] == False:
                        if trimal_commands[kwd] == True:
                            self.command_lines[aln] += "-%s "%(kwd)
                        else:
                            self.command_lines[aln] += "-%s %s "%(kwd, trimal_commands[kwd])
            
            self.command_lines[aln+'@'+self.method_name] = self.command_lines[aln][:-1]
            print self.command_lines[aln+'@'+self.method_name]
            self.command_lines.pop(aln, None)
        
    def __str__(self):
        aln_string = ''
        for n in self.alignments.keys():
            aln_string += n+','
        aln_string = aln_string[:-1]
        command_lines = ''
        for i in self.command_lines.keys():
            command_lines += i+': '+str(self.command_lines[i])+'\n'
        date = str(self.timeit[0])
        execution = '[This was not executed yet]'
        if len(self.timeit) > 1:
            execution = str(self.timeit[3])
        
        prog = '[This was not executed yet]'
        plat = '[This was not executed yet]'
        progref = '[This was not executed yet]'
        if len(self.platform) > 0:
            plat = str(self.platform).replace(",",'\n').replace(']','').replace("'",'').replace('[','')
            if self.platform[-1].startswith('Program reference:'):
                prog = self.platform[-2].split(':')[1].strip().rstrip()
                progref = self.platform[-1].split('Program reference:')[1].strip().rstrip()
        
        output= str("TrimalConf named %s with ID %s\n"+         
                "Alignments: %s \n"+       
                "Created on: %s\n"+
                "Commands:\n"+
                "%s\n"+
                "Environment:"+    
                "%s\n"+
                "execution time:\n"+
                "%s")%(self.method_name, str(self.id), aln_string, date, command_lines, plat, execution)
        
        output += TrimalConfMethodsSection%(type_to_single_line_str(self.alignments.keys()),
                                            prog, progref)
        
        return output

def use_sh_support_as_branch_support(tree_filename):
    string = open(tree_filename,'r').read()
    string = re.sub(r'\[',r'[&&NHX:support=',string)
    t = Tree(string)
    t.dist=0
    return t.write(features=[])
    
def transfer_support_same_topo(tree_file_with_support,
                               tree_file_without_support):
    supported = Tree(tree_file_with_support) 
    unsupported = Tree(tree_file_without_support)
    supported_leaf_names = sorted(supported.get_leaf_names())
    unsupported_leaf_names = sorted(unsupported.get_leaf_names())
    if not len(unsupported_leaf_names) == len(supported_leaf_names):
        raise IOError(tree_file_with_support + ' and ' + tree_file_without_support +
                      ' are not the same length')
    for i in range(len(supported_leaf_names)):
        if not supported_leaf_names[i] == unsupported_leaf_names[i]:
            raise IOError('The trees do not share all leaves or leaf names')
    same_root = supported.get_leaf_names()[0]
    unsupported.set_outgroup(same_root)
    supported.set_outgroup(same_root)
    for ns in supported.traverse():
        ns_leaves = ns.get_leaf_names()
        if not unsupported.check_monophyly(values=ns_leaves, target_attr="name"):
            raise RuntimeError('trees do not share topology and/or all the leaf names')
        else:
            unsupported_ancestor = unsupported.get_common_ancestor(ns_leaves)
            unsupported_ancestor.support = ns.support
    unsupported.write(outfile = tree_file_without_support)    
    
    
def make_raxml_partfile(tree_method, pj, trimmed_alignment_name):

    concatenation = None
    for c in pj.concatenations:
        if c.name == trimmed_alignment_name:
            concatenation = c
    
    #concatenation = filter(lambda concatenation: concatenation.name == trimmed_alignment_name, pj.concatenations)[0]
    
    model = []
    for locus in concatenation.loci:
        part_name = None
        for trm_aln in concatenation.used_trimmed_alns.keys():
            if locus.name == trm_aln.partition('@')[0]:
                part_name = trm_aln
        if not part_name:
            warnings.warn('There is no trimmed alignment for locus '+locus.name+' in concatenation '+concatenation.name)
        else:
            part_length = concatenation.used_trimmed_alns[part_name]
            if locus.char_type == 'prot':
                m = None
                if isinstance(tree_method.matrix,dict):
                    m = tree_method.matrix[locus.name]
                elif isinstance(tree_method.matrix,str):
                    m = tree_method.matrix
                else:
                    #todo write error
                    pass
                model.append([m,part_name,part_length])
            elif locus.char_type == 'dna':
                model.append(['DNA',part_name,part_length])
                    
    # make partition file
                    
    partfile = open(tree_method.id+'_'+concatenation.name+'_partfile','wt')
    i = 1
    for m in model:
        partfile.write(m[0]+', '+m[1]+'='+str(i)+'-'+str(m[2]+i-1)+'\n')
        i += m[2]
    partfile.close()
    return tree_method.id+'_'+concatenation.name+'_partfile'

def make_raxml_input_matrix_file(tree_method, trimmed_alignment_name):
    SeqIO.write(tree_method.trimmed_alignments[trimmed_alignment_name],
                tree_method.id+'_'+trimmed_alignment_name+'.fasta','fasta')
    return tree_method.id+'_'+trimmed_alignment_name+'.fasta'

def write_raxml_clines(tree_method, pj, trimmed_alignment_name):
            
    cline_que = 0

    support_replicates = 100
    ML_replicates = 1
    if '-N' in tree_method.cline_args.keys():
        ML_replicates = tree_method.cline_args['-N']
    if '-#' in tree_method.cline_args.keys():
        support_replicates = tree_method.cline_args['-#']
    
    partfile = None
    
    # Check if it is a concatenation and make partfile

    for c in pj.concatenations:
        if c.name == trimmed_alignment_name.partition('@')[0]:
            partfile = make_raxml_partfile(tree_method, pj, trimmed_alignment_name)
    
    input_filename = make_raxml_input_matrix_file(tree_method, trimmed_alignment_name)
    model = tree_method.model
    try:
        locus_char_type = filter(lambda locus: locus.name == trimmed_alignment_name.partition('@')[0], pj.loci)[0].char_type
    except:
        locus_char_type = 'prot'
    
    if partfile:
        model='PROT'+model+'JTT'
    else:
        if locus_char_type == 'dna':
            model = 'GTR'+tree_method.model
        elif  locus_char_type == 'prot':
            if isinstance(tree_method.matrix,str):
                model = 'PROT'+tree_method.model+tree_method.matrix
            elif isinstance(tree_method.matrix,dict):
                model = 'PROT'+tree_method.model+tree_method.matrix[trimmed_alignment_name]
        
    presets = {'fa': [{'-f': 'a',
                           '-p': random.randint(99,999),
                           '-x':  random.randint(99,999),
                           '-s': input_filename,
                           '-N': support_replicates,
                           '-n': tree_method.id+'_'+trimmed_alignment_name+'0',
                           '-m': model}
                      ],
                'fD_fb':[{'-f': 'D',
                          '-p': random.randint(99,999),
                          '-s': input_filename,
                          '-N': ML_replicates,
                           '-n': tree_method.id+'_'+trimmed_alignment_name+'0',
                           '-m': model},{'-f': 'b',
                                                       '-p': random.randint(99,999),
                                                       '-s': input_filename,
                                                       '-n': tree_method.id+'_'+trimmed_alignment_name+'1',
                                                       '-m': model,
                                                       '-t': 'RAxML_bestTree.'+tree_method.id+'_'+trimmed_alignment_name+'0',
                                                       '-z': 'RAxML_rellBootstrap.'+tree_method.id+'_'+trimmed_alignment_name+'0'}
                         ],
                'fd_b_fb':[{'-f': 'd',
                          '-p': random.randint(99,999),
                          '-s': input_filename,
                          '-N': ML_replicates,
                           '-n': tree_method.id+'_'+trimmed_alignment_name+'0',
                           '-m': model},{
                                                       '-p': random.randint(99,999),
                                                       '-b': random.randint(99,999),
                                                       '-s': input_filename,
                                                       '-#': support_replicates,
                                                       '-n': tree_method.id+'_'+trimmed_alignment_name+'1',
                                                       '-m': model,
                                                       '-T': tree_method.threads},
                                                       {'-f': 'b',
                                                       '-p': random.randint(99,999),
                                                       '-s': input_filename,
                                                       '-n': tree_method.id+'_'+trimmed_alignment_name+'2',
                                                       '-m': model,
                                                       '-t': 'RAxML_bestTree.'+tree_method.id+'_'+trimmed_alignment_name+'0',
                                                       '-z': 'RAxML_bootstrap.'+tree_method.id+'_'+trimmed_alignment_name+'1'}
                         ],
                'fF_fJ': [{'-f': 'F',
                           '-p': random.randint(99,999),
                           '-s': input_filename,
                           '-n': tree_method.id+'_'+trimmed_alignment_name+'0',
                           '-m': model},{'-f': 'J',
                                         '-t': 'RAxML_fastTree.'+tree_method.id+'_'+trimmed_alignment_name+'0',
                                         '-p': random.randint(99,999),
                                         '-s': input_filename,
                                         '-n': tree_method.id+'_'+trimmed_alignment_name+'1',
                                         '-m': model}],
                
                'fd_fJ': [{'-f': 'd',
                          '-p': random.randint(99,999),
                          '-s': input_filename,
                          '-N': ML_replicates,
                          '-n': tree_method.id+'_'+trimmed_alignment_name+'0',
                          '-m': model},{'-f': 'J',
                                         '-t': 'RAxML_bestTree.'+tree_method.id+'_'+trimmed_alignment_name+'0',
                                         '-p': random.randint(99,999),
                                         '-s': input_filename,
                                         '-n': tree_method.id+'_'+trimmed_alignment_name+'1',
                                         '-m': model}]
                
                }

    
    if 'PTHREADS' in tree_method.cmd:
        if tree_method.threads < 2:
            tree_method.threads = 2
            warnings.warn('raxmlHPC-PTHREADS requires at least 2 threads. Setting threads to 2')
        for preset in presets:
            for c in presets[preset]:
                c['-T'] = tree_method.threads
    else:
        if tree_method.threads > 1:
            raise RuntimeWarning('This is a serial raxmlHPC. Setting threads to 1.'+ 
                                 'PTHREADS executables have to have explicit filename, eg, raxmlHPC-PTHREADS-SSE3')
    if partfile:
        for preset in presets.keys():
            for cline in range(len(presets[preset])):
                presets[preset][cline] = dict({'-q': partfile}, **presets[preset][cline])               
    return presets[tree_method.preset] 


RaxmlConfMethodsSection=\
"""\n
==============================
Core Methods section sentence:
==============================
Phylogenetic trees were reconstructed from the dataset(s) %s using the program %s [1].

Reference:
%s
"""
##############################################################################################
class RaxmlConf:
##############################################################################################
    
    def __init__(self, pj, method_name='fa', program_name='raxmlHPC-PTHREADS-SSE3', keepfiles=False,
                 cmd='default', preset = 'fa', alns='all', model='GAMMA', matrix='JTT', threads=4,
                 cline_args={}):
        
        
        if len(pj.trimmed_alignments) == 0:
            raise RuntimeError("No trimmed sequence alignments found")
            
        self.id = str(random.randint(10000,99999))+str(time.time())
        self.method_name=method_name
        self.program_name=program_name
        self.preset = preset
        self.cline_args = cline_args
        self.model = model
        self.matrix = matrix
        self.threads = threads
        self.trimmed_alignments = pj.trimmed_alignments
        if not alns == 'all':
            self.trimmed_alignments = {}
            for aln_name in alns:
                if aln_name in pj.trimmed_alignments.keys():
                    self.trimmed_alignments[aln_name] = pj.trimmed_alignments[aln_name]
        self.aln_input_strings = {}
        self.command_lines = {}
        self.timeit = [time.asctime()]
        self.platform = []
        self.cmd = cmd
        self.keepfiles = keepfiles
        if cmd == 'default':
            self.cmd = pj.defaults['raxmlHPC']
        
        for trimmed_alignment in self.trimmed_alignments.keys():
            self.command_lines[trimmed_alignment] = []
            command_lines = write_raxml_clines(self, pj, trimmed_alignment)
            for command_line in command_lines:
                cline_object = RaxmlCommandline(cmd=self.cmd)
                for c in command_line.keys():
                    cline_object.__setattr__(c,command_line[c])
                self.command_lines[trimmed_alignment].append(cline_object)
                print str(cline_object)

    def __str__(self):

        aln_string = ''
        for n in self.trimmed_alignments.keys():
            aln_string += n+','
        aln_string = aln_string[:-1]
        command_lines = ''
        for i in self.command_lines.keys():
            command_lines += i+':\n'
            for k in self.command_lines[i]:
                command_lines += str(k) + '\n'
        date = str(self.timeit[0])
        
        execution = '[This was not executed yet]'
        if len(self.timeit) > 1:
            execution = str(self.timeit[3])
        
        prog = '[This was not executed yet]'
        plat = '[This was not executed yet]'
        progref = '[This was not executed yet]'
        
        if len(self.platform) > 0:
            plat = str(self.platform).replace(",",'\n').replace(']','').replace("'",'').replace('[','')
            if self.platform[-1].startswith('Program reference:'):
                prog = self.platform[-2].split(':')[1].strip().rstrip()
                progref = self.platform[-1].split('Program reference:')[1].strip().rstrip()
                
        output = str("RaxmlConf named %s with ID %s\n"+         
                "Alignments: %s \n"+       
                "Created on: %s\n"+
                "Commands:\n"+
                "%s\n"+
                "Environment:\n"+    
                "%s\n"+
                "execution time:\n"+
                "%s")%(self.method_name, str(self.id), aln_string, date, command_lines, plat, execution)
        
        output += RaxmlConfMethodsSection%(type_to_single_line_str(self.trimmed_alignments.keys()),
                                            prog, progref)
        
        return output

def make_pb_input_matrix_file(conf_obj, trimmed_alignment_name):
    SeqIO.write(conf_obj.trimmed_alignments[trimmed_alignment_name],
                conf_obj.id+'_'+trimmed_alignment_name+'.phylip','phylip-relaxed')
    return conf_obj.id+'_'+trimmed_alignment_name+'.phylip'


def write_pb_cline(conf_obj, pj, trimmed_alignment):
    cline = "%s -d %s"%(conf_obj.cmd, make_pb_input_matrix_file(conf_obj, trimmed_alignment))
    for key in conf_obj.cline_args:
        kw = key
        if key[0] == '-':
            kw = key[1:]
        cline += " -%s"%str(kw)
        if not str(conf_obj.cline_args[key]) == str(True):
            cline += " %s"%str(conf_obj.cline_args[key]) 
    cline += " %s_%s"%(conf_obj.id, trimmed_alignment)
    return cline
            
###################################################################################################
class PbConf:
###################################################################################################
    
    def __init__(self, pj, method_name = 'dna_cat_gtr', program_name='phylobayes', keepfiles=True,
                 cmd='default', alns='all', cline_args=dict(nchain="2 100 0.1 100",
                                                            gtr=True,
                                                            cat=True)):
        self.id = str(random.randint(10000,99999))+str(time.time())
        self.method_name=method_name
        self.program_name=program_name
        self.cline_args = cline_args
        self.trimmed_alignments = pj.trimmed_alignments
        if not alns == 'all':
            self.trimmed_alignments = {}
            for aln_name in alns:
                if aln_name in pj.trimmed_alignments.keys():
                    self.trimmed_alignments[aln_name] = pj.trimmed_alignments[aln_name]
        self.aln_input_strings = {}
        self.command_lines = {}
        self.timeit = [time.asctime()]
        self.platform = []
        self.cmd = cmd
        self.keepfiles = keepfiles
        if cmd == 'default':
            self.cmd = pj.defaults['pb']
            
        for trimmed_alignment in self.trimmed_alignments.keys():
            self.command_lines[trimmed_alignment] = [write_pb_cline(self, pj, trimmed_alignment)]
            print self.command_lines[trimmed_alignment][0]
    
    def __str__(self):

        aln_string = ''
        for n in self.trimmed_alignments.keys():
            aln_string += n+','
        aln_string = aln_string[:-1]
        command_lines = ''
        for i in self.command_lines.keys():
            command_lines += i+': '+str(self.command_lines[i])+'\n'
        date = str(self.timeit[0])
        
        execution = '[This was not executed yet]'
        if len(self.timeit) > 1:
            execution = str(self.timeit[3])
        
        prog = '[This was not executed yet]'
        plat = '[This was not executed yet]'
        progref = '[This was not executed yet]'
        
        if len(self.platform) > 0:
            plat = str(self.platform).replace(",",'\n').replace(']','').replace("'",'').replace('[','')
            if self.platform[-1].startswith('Program reference:'):
                prog = self.platform[-2].split(':')[1].strip().rstrip()
                progref = self.platform[-1].split('Program reference:')[1].strip().rstrip()
                
        output = str("PbConf named %s with ID %s\n"+         
                "Alignments: %s \n"+       
                "Created on: %s\n"+
                "Commands:\n"+
                "%s\n"+
                "Environment:\n"+    
                "%s\n"+
                "execution time:\n"+
                "%s")%(self.method_name, str(self.id), aln_string, date, command_lines, plat, execution)
        
        output += RaxmlConfMethodsSection%(type_to_single_line_str(self.trimmed_alignments.keys()),
                                            prog, progref)
        
        return output
            
from pylab import *
import random

def draw_boxplot(dictionary, y_axis_label, figs_folder): #'locus':[values]
    import numpy as np
    import matplotlib.pyplot as plt
    items = dictionary.items()
    items.sort()
    data = [locus[1] for locus in items]
        
    fig, ax1 = plt.subplots()
    fig.set_size_inches(0.3*len(data),10)
    plt.subplots_adjust(top=0.99, bottom=0.3)

    #bp = plt.boxplot(data, widths=0.75, patch_artist=True)
    bp = plt.boxplot(data, patch_artist=True)
    
    for box in bp['boxes']:
    # change outline color
        box.set( color='black', linewidth=1)
        
    # change fill color
        box.set( facecolor = 'red', alpha=0.85 )
        
    # change color, linestyle and linewidth of the whiskers
    for whisker in bp['whiskers']:
        whisker.set(color='gray', linestyle='solid', linewidth=2.0)

    # change color and linewidth of the caps
    for cap in bp['caps']:
        cap.set(color='gray', linewidth=2.0)

    # change color and linewidth of the medians
    for median in bp['medians']:
        #median.set(color='#b2df8a', linewidth=2)
        median.set(color='white', linewidth=2)

    # change the style of fliers and their fill
    for flier in bp['fliers']:
        flier.set(marker='o', color='#e7298a', alpha=0.5)
    
    # Add a light horizontal grid to the plot
    ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
              alpha=0.7)
    
    # Hide these grid behind plot objects
    ax1.set_axisbelow(True)
    
    #set title and axis labels
    #ax1.set_title('Sequence length distribution per locus\n', size=18)
    
    xlabels = [locus[0] for locus in items]
    
    xticks(range(1,len(data)+1), xlabels, size=14, rotation='vertical')
    #subplots_adjust(left=0.3, bottom=0.8)
    
    ax1.set_ylabel(y_axis_label, size=18)
    
    name = str(random.randint(1000,2000))
    if figs_folder=='inline':
        fig.show()
    else:
        fig.savefig(figs_folder + '/' + name +'.png')
        close('all')
    return figs_folder + '/' + name+'.png'
    
#################################################################################
def report_methods(pj, figs_folder, output_directory, size='small',
                   compare_trees=[], compare_meta=None, trees_to_compare='all',
                   unrooted_trees=False, mp_root=True):
#################################################################################
        """
        Main HTML reporting function. This function iterates over the 
        Project's attributes and generate appropriate html formated report lines.
        
        pj -               The Project object
        
        figs_folder -      The directory to which tree figures were saved. This is
                           specified in the annotate Project method
                           
        output_directory - The directory to which this report will be written.It 
                           can be inherited from the publish function which uses 
                           this function.
        """
        
        #========================================================================
        #                                   HEAD
        #========================================================================
        
        # Checking if 'report_methods' was called by 'publish' in order to infer
        # which folders to create and what errors to raise for existing 
        # directories. If we were called by 'publish' we should allow 
        # output_directory to exist because 'publish' has just created it.
        # 'publish' would have also raised an error if it existed before.
        curframe = inspect.currentframe()
        calframe = inspect.getouterframes(curframe, 2)
        callername = calframe[1][3]
        print "reporter was called by "+str(callername)
        # Here we manage mkdir and error raising for preexisting 
        # output_directory, based of the caller function
        if os.path.isdir(output_directory) and str(callername) != 'publish':
            raise RuntimeError('%s already exists'%output_directory) 
        else:
            # This will make output_directory if it doesnt exist and
            # will add a subdirectory 'files' which will store the
            # stylesheet file and the figure files
            os.makedirs('%s/files'%output_directory)
        
        # This will contain the text that points to the stylesheet file
        # I have named it rp.css. We need to add something that writes
        # it here. Or that get it from the source directory and places it
        # in /files
        
        css_line = '<link rel="stylesheet" type="text/css" href="Bootstrap.css">'

        # This list will contain the report lines/ tables as values. We will append
        # each new report line to it
        report_lines = ['<html>','<head>',css_line,'<h1>']
    
        # Main report title, will print the time in which the 'Project' object
        # was initiated (not the time when the report was made as in previous 
        # versions.
        head = 'reprophylo analysis from '+pj.starttime
        

        report_lines.append(head)
        report_lines += ['</h1>','</head>','<body>','']
                
        #========================================================================
        #                                   BODY
        #========================================================================

        
        #############################  section 1:   DATA  #######################
        
        if pj.user:
            report_lines += ['<h2>','User Info','</h2>', '']
            for item in pj.user:
                report_lines += ['<strong>%s: </strong>'%item[0], str(item[1])]
            report_lines += ['']
        
        report_lines += ['<h2>','Data','</h2>', '']
        
        print "starting species table"
        # Species over loci table
        #------------------------------------------------------------------------
        title = 'species representation in sequence data'.title()
        report_lines += ('<h3>', title, '</h3>', '')
        
        
        # This will write a CSV file of the times each locus occures for each 
        # species. The species are sorted in alphabetical order. The CSV will 
        # be writen to 'outfile_name'. After 'outfile_name' is processed it will
        # be deleted.The CSV will be read with the csv module and the resulting
        # list of lists will be made into a table using HTML and than added to 
        # 'report_lines'. The species an gene counts are made by 'species_vs_loci'
        # based on the sequences record objects (biopython) found in pj.records.
        # pj.records is a list of SeqRecord objects
        outfile_name= str(random.randint(1000,2000))
        pj.species_vs_loci(outfile_name)
        with open(outfile_name, 'rb') as csvfile:
            sp_vs_lc = list(csv.reader(csvfile, delimiter='\t', quotechar='|'))
            report_lines += ['',
                             HTML.table(sp_vs_lc[1:], header_row=sp_vs_lc[0]),
                             '']
            # The following writes a pre text of the same thing
            #field_sizes = []
            #for i in range(len(sp_vs_lc[0])):
            #    lengths = []
            #    for row in sp_vs_lc:
            #        lengths.append(len(row[i]))
            #    field_sizes.append(max(lengths))
            #for row in sp_vs_lc:
            #    string = ''
            #    for i in range(len(row)):
            #        string += row[i].ljust(field_sizes[i]+3)
            #    report_lines.append(string)
        
        os.remove(outfile_name)
        
        if not size == 'small':
            print "starting sequence statistics plots"        
            # Sequence statistic plots
            #------------------------------------------------------------------------
            title = 'Sequence statistic plots'.title()
            report_lines += ('<h3>', title, '</h3>', '')

            # This will plot 4 box plot figures representing the distribution of seq
            # length, GC content, %ambiguity in nuc and prot seqs for each locus.
            if len(pj.records_by_locus.keys())>0:

                # This will determine the with of the figure, 0.5' per locus
                scale = str(len(pj.records_by_locus.keys())*0.5)

                # This will make a list of seq length for each locus. Seq length are calced
                # using the record.seq in 'pj.records_by_locus'. 'pj.records_by_locus is a
                # dict with loci names as keys, and lists of SeqReocrd objects as values
                lengths_dict = {}
                for locus_name in pj.records_by_locus.keys():
                    lengths_dict[locus_name] = []
                    for record in pj.records_by_locus[locus_name]:
                        lengths_dict[locus_name].append(len(record.seq))

                # This draws a box plot of sequence length distributions and puts a png in 
                # the 'files' directory.
                if not size == 'small':
                    fig_filename = draw_boxplot(lengths_dict, 'Seq length (bp)', '%s/files'%output_directory)


                # Distribution of sequence lengths
                #---------------------------------------------------------------------
                title = 'Distribution of sequence lengths'
                report_lines += ( '<h4>', title, '</h4>',  '')

                # This will write the img tag for the seq length boxplot in the report html
                # The src attribute is the png file path. The commented lines are an alternative
                # making an embeded figure.
                if not size=='small' and os.path.isfile(fig_filename):
                    #data_uri = open(fig_filename, 'rb').read().encode('base64').replace('\n', '')
                    #img_tag = '<img height=400 width='+scale+' src="data:image/png;base64,{0}">'.format(data_uri)
                    img_tag = '<img src="%s">'%(fig_filename.partition('/')[-1])
                    report_lines.append(img_tag)
                    #os.remove(fig_filename)


                # This will make GC content, nuc_degen_prop and prot_degen_prop png file,
                # will put them in the /files subdirectory and will write the html sections
                # for them, including img tags. All three params are feature qualifiers of
                # SeqRecord objects found in pj.records, which is a list.
                stats_to_plot = ('GC_content', 'nuc_degen_prop', 'prot_degen_prop')
                if size=='small':
                    stats_to_plot = ()
                for stat in stats_to_plot:
                    # This will make a dict with loci as keys and a list of stat values as
                    # dict values.
                    stat_dict = {}
                    ylabel = 'GC ontent (%)'
                    if not stat == 'GC_content':
                        ylabel = 'Ambiguous positions (prop)'
                    for locus_name in pj.records_by_locus.keys():
                        stat_dict[locus_name] = []
                        for i in pj.records_by_locus[locus_name]:
                            for record in pj.records:
                                for feature in record.features:
                                    if feature.qualifiers['feature_id'][0] == i.id:
                                        if stat in feature.qualifiers.keys():
                                            stat_dict[locus_name].append(float(feature.qualifiers[stat][0]))

                    # This will make the boxplot png and will put in in the /files subdirectory
                    fig_filename = draw_boxplot(stat_dict, ylabel, '%s/files'%output_directory)

                    # Distribution of stat
                    #---------------------------------------------------------------------
                    title = 'Distribution of sequence statistic \"'+stat+'\"'
                    report_lines += ( '<h4>', title, '</h4>', '')

                    # This will make the img tag using the png path as src. The commented lines are an alternative
                    # making an embeded image
                    if os.path.isfile(fig_filename):
                        #data_uri = open(fig_filename, 'rb').read().encode('base64').replace('\n', '')
                        img_tag = '<img src="%s">'%(fig_filename.partition('/')[-1])
                        #img_tag = '<img height=400 width='+scale+' src="data:image/png;base64,{0}">'.format(data_uri)
                        report_lines.append(img_tag)
                        #os.remove(fig_filename)
                
        print "starting concatenations"
        # Description of data concatenations
        #------------------------------------------------------------------------
        title = 'Description of data concatenations'.title()
        report_lines += ('<h3>', title, '</h3>', '')
        
        # filter out concatenation objects that were not used to build a concatenation
        # by taking only concat names that are in the keys of pj.trimmed_alignments.
        # pj.trimmed_alignments is a dict with alignment names as keys and list 
        # lists containing an alignment object (biopython) and an alignment string as
        # values.
        composed_concatenations = []
        for c in pj.concatenations:
            if c.name in pj.trimmed_alignments.keys():
                composed_concatenations.append(c)

        for c in composed_concatenations:
            
            title = ('content of concatenation \"' + c.name + '\"').title()
            report_lines += ('<h4>', title, '</h4>', '')
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            # This will write the concatenation attribute
            report_lines.append('Rules for  \"' + c.name + '\":')
            rule_1 = 'OTUs must have the loci: '
            for locus in c.otu_must_have_all_of:
                rule_1 += locus + ', '
            report_lines.append(rule_1)
            rule_2 = '<pre>'
            for group in c.otu_must_have_one_of:
                rule_2 += '</pre>OTUs must have at least one of the following loci: \n<pre>'
                rule_2 += str(group).replace('[','').replace(']','') +'</pre>\n'
            report_lines += (rule_2, '')
            
            # This are the otus and loci names in the concatenation. c.feature_id_dict
            # is a dict with the otius as keys and dicts as values. Each of these dicts
            # have the loci as keys and the feature id as value
            otus = c.feature_id_dict.keys()
            loci = [locus.name for locus in c.loci]
            
            # This is the table's header
            table_lines = [['','']+[locus.name for locus in c.loci]]
            
            
            # Commented lines are an alternative way to write the table as 
            # pre text
            #otus_max_length = max([len(i) for i in otus])+33
            #loci_columns_max_length = []
            
            #for locus in loci:
            #    lengths = [len(locus)]
            #    for otu in otus:
            #        if locus in c.feature_id_dict[otu].keys():
            #            lengths.append(len(c.feature_id_dict[otu][locus]))
            #        else:
            #           lengths.append(0)
            #    loci_columns_max_length.append(max(lengths)+3)
                
            #concat_header = ''.ljust(otus_max_length)
            #for i in range(len(loci)):
            #    concat_header += loci[i].ljust(loci_columns_max_length[i])
            #report_lines += (concat_header, '~'*len(concat_header))
                
            
            # This will write the table
            for otu in otus:
                
                otu_species = ''
                for locus in loci:
                    if locus in c.feature_id_dict[otu].keys():
                        feature_qualifiers = get_qualifiers_dictionary(pj, c.feature_id_dict[otu][locus])
                        if 'source_organism' in feature_qualifiers.keys():
                            otu_species = feature_qualifiers['source_organism']
                otu_line = [otu, otu_species]    
                #concat_line = (otu+' '+otu_species).ljust(otus_max_length)
                for i in range(len(loci)):
                    if loci[i] in c.feature_id_dict[otu].keys():
                        #concat_line += c.feature_id_dict[otu][loci[i]].ljust(loci_columns_max_length[i])
                        otu_line.append(c.feature_id_dict[otu][loci[i]])
                    else:
                        #concat_line += ''.ljust(loci_columns_max_length[i])
                        otu_line.append('')
                table_lines.append(otu_line)
            
                #report_lines.append(concat_line)
        
            report_lines.append(HTML.table(table_lines[1:], header_row=table_lines[0]))  
        
        #############################  section 2:   METHODS  #######################
        
        # This section prints some attributes of each of the 'Conf' objects used.
        # The 'Conf' objects are found in a list called pj.used_methods. 
        # In an unpickled 'Project' object, the 'Conf' objects are replaced by lists of 
        # strings describing the attributes because the objects themselves do not
        # pickle well. The formating of the list representations when they are printed
        # still needs some beautification. Also, I plan a 'revive_methods' func to turn
        # them back to 'Conf' objects that can be rerun.
        report_lines += ['', '<h2>','Methods','</h2>', '']
        
        print "starting methods"
        for method in pj.used_methods:
            # This will print list representations of the 'Conf' objects
            title = str(pj.used_methods[method]).split('\n')[0]
            report_lines += ('', '<h4>', title, '</h4>','')
            report_lines += ('<pre>',str(pj.used_methods[method]),'</pre>')
       
                
        report_lines += ['',''] 
        
        #############################  section 3:   RESULTS  #######################
        
        report_lines += ['', '<h2>','Results','</h2>', '']
        print "starting alignment statistics"
        # Global alignmnet statistics
        #------------------------------------------------------------------------
        title = 'Global alignmnet statistics'.title()
        report_lines += ('<h3>', title, '</h3>', '')
        
        
        # This prints things like num of unique seqs and num of parsimony informative
        # cloumns. Takes the info from 'pj.aln_summaries' which is a list of strings.
        # Alignment length: 1566
        #Number of rows: 94
        #Unique sequences: 87
        #Average gap prop.: 0.488445
        #Variable columns: 1045
        #Parsimony informative: 402
        #Undetermined sequences: 0
        
        
        if len(pj.aln_summaries)>0:
            report_lines += [('<pre>Name=Alignment name\n'+
                              'NumPos=Alignment length\n'+
                              'NumSeq=Number of sequences\n'+
                              'Unique=Number of unique sequences\n'+
                              'GapProp=Average gap proportion\n'+
                              'VarCols=Total variable positions\n'+
                              'ParsInf=Parsimony informative positions\n'+
                              'UnSeqs=Undetermined sequences (mostly/only gaps)\n'+
                              'UnSeqsCutoff=Length cutoff which defines undetermined\n</pre>')]
            T = [['Name','NumPos','NumSeq','Unique','GapProp','VarCols','ParsInf','UnSeqs','UnSeqsCutoff']]
            comments = []
            for summary in pj.aln_summaries:
                line = []
                for i in summary.splitlines():
                    try:
                        line.append(i.split(': ')[1])
                    except:
                        comments.append(i)
                T.append(line)
            report_lines += ['',
                             HTML.table(T[1:], header_row=T[0]),
                             '','<pre>']+comments+['</pre>']
        else:
            report_lines += ['','No sequence alignments in this Project','']
                
        
        # Per position alignmnet statistics
        #------------------------------------------------------------------------
        title = 'per position alignmnet statistics'.title()
        report_lines += ('<h3>', title, '</h3>', '')
        if len(pj.alignments.keys())>0 and not size == 'small':                    
            title = 'Alignment statistics before trimming'
            report_lines += ('', '<h4>', title, '</h4>', '')
            report_lines += ['<h4>','Trimal\'s Residue Similarity Score (-scc)','</h4>', '']
            
            # draw_trimal_scc(project, num_plots_in_raw, output_dir...
            # 'trimmed' determines if it will analyse trimmed or raw alignments
            # alignments are taken from pj.alignments or pj.trimmed_alignments which are dictionaries
            # with alignment names as keys and lists containing an aln object and al string as values
            
            # scc on raw alignments
            fig_file = draw_trimal_scc(pj, 2, '%s/files'%output_directory, trimmed=False)
            if os.path.isfile(fig_file):
                    #data_uri = open(fig_file, 'rb').read().encode('base64').replace('\n', '')
                    #img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)
                    img_tag = '<img src="%s">'%(fig_file.partition('/')[-1])
                    report_lines.append(img_tag)
                    #os.remove(fig_file)
            report_lines += [ '<h4>','Trimal\'s column gap gcore (-sgc)','</h4>', '']
            
            # sgc on raw alignments
            fig_file = draw_trimal_scc(pj, 2, '%s/files'%output_directory, trimmed=False, alg='-sgc')
            if os.path.isfile(fig_file):
                    #data_uri = open(fig_file, 'rb').read().encode('base64').replace('\n', '')
                    #img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)
                    img_tag = '<img src="%s">'%(fig_file.partition('/')[-1])
                    report_lines.append(img_tag)
                    #os.remove(fig_file)
        else:
            report_lines += ['No alignments or too many alignments in this project','']
            
        if len(pj.trimmed_alignments.keys())>0 and not size=='small':          
            title = 'Alignment statistics after trimming'
            report_lines += ('', '<h4>', title, '</h4>', '')
            report_lines += ['<h4>','"Trimal\'s Residue Similarity Score (-scc)','</h4>', '']
            
            # scc on trimmed alignments
            fig_file = draw_trimal_scc(pj, 2, '%s/files'%output_directory, trimmed=True)
            if os.path.isfile(fig_file):
                    #data_uri = open(fig_file, 'rb').read().encode('base64').replace('\n', '')
                    #img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)
                    img_tag = '<img src="%s">'%(fig_file.partition('/')[-1])
                    report_lines.append(img_tag)
                    #os.remove(fig_file)
            report_lines += ['<h4>','Trimal\'s column gap gcore (-sgc)','</h4>',  '']
            
            # sgc on trimmed alignments
            fig_file = draw_trimal_scc(pj, 2, '%s/files'%output_directory, trimmed=True, alg='-sgc')
            if os.path.isfile(fig_file):
                    #data_uri = open(fig_file, 'rb').read().encode('base64').replace('\n', '')
                    #img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)
                    img_tag = '<img src="%s">'%(fig_file.partition('/')[-1])
                    report_lines.append(img_tag)
                    #os.remove(fig_file)
        else:
            report_lines += ['No trimmed alignments, or too many, in this project','']
        
        print "starting RF matrix(ces)"
        for rf_type in compare_trees:
            title = 'Robinson-Foulds distances (%s)'%rf_type.title()
            report_lines += ('<h3>', title, '</h3>', '')

            if len(pj.trees.keys())>1:
                try:
                    RF_filename, legend = calc_rf(pj, '%s/files'%output_directory,
                                                  rf_type=rf_type, meta=compare_meta, trees=trees_to_compare,
                                                  unrooted_trees=unrooted_trees, mp_root=mp_root)
                    scale = str(len(legend)*60)
                    if os.path.isfile(RF_filename):
                            #data_uri = open(RF_filename, 'rb').read().encode('base64').replace('\n', '')
                            #img_tag = '<img height='+scale+' width='+scale+' src="data:image/png;base64,{0}">'.format(data_uri)
                            img_tag = '<img src="%s">'%(RF_filename.partition('/')[-1])
                            report_lines.append(img_tag)
                            #os.remove(RF_filename)

                    report_lines+=['<h3>Legend<h3>','']
                    report_lines += [HTML.table( legend[1:], 
                                               header_row=legend[0])]
                    report_lines.append('')
                except:
                    report_lines += ['Skipping RF distance calculation']

            else:
                report_lines += ['Less than two trees in this Project','']

                
        #############################  section 4:   TREES  #######################
        print "reporting trees"
        report_lines += ['', '<h2>','Trees','</h2>', '']
        
        for tree in pj.trees.keys():
            report_lines += ('<h2>'+tree.split('@')[0]+'</h2>',
                             '<h3>Alignment method: '+tree.split('@')[1]+'</h3>',
                             '<h3>Trimming method: '+tree.split('@')[2]+'</h3>',
                             '<h3>Tree method: '+tree.split('@')[3]+'</h3>',
                             '<pre style="white-space:normal;">',
                             'Tree Method ID: '+pj.trees[tree][0].get_leaves()[0].tree_method_id,'</pre>')
            
            report_lines += ('<h3>newick format</h3>','','<pre style="white-space:normal;">',pj.trees[tree][0].write(),'</pre>','')
            report_lines += ('<h3>nhx format</h3>','','<pre>',pj.trees[tree][1],'</pre>','','','','')
            
            
            
            if os.path.isfile(figs_folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png'):
                origin = figs_folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png'
                dest = '%s/files/%s'%(output_directory, pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png')
                shutil.copyfile(origin, dest)
                #data_handle = open(figs_folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png','rb')
                #data_uri = data_handle.read().encode('base64').replace('\n', '')
                #data_handle.close()
                #img_tag = '<img width=500 src="data:image/png;base64,{0}">'.format(data_uri)
                img_tag = '<img width=500 src="%s">'%(dest.partition('/')[-1])
                report_lines.append(img_tag)

                
        report_lines.append('</body>')
        report_lines.append('</html>')
        
        lines = []
        
        for line in report_lines:
            if '<' in line:
                lines.append(line)
            else:
                lines.append(line.replace('\n','<br>')+'<br>')
        
        
        return lines
    
        
def pickle_pj(pj, pickle_file_name, track=True):
    import os
    if os.path.exists(pickle_file_name):
        #print "DEBUG pickle_pj: %s"%str(os.stat(pickle_file_name).st_ctime)
        #print "DEBUG pickle_pj: %s"%str(os.stat(pickle_file_name).st_size)
        os.remove(pickle_file_name)
    import cloud.serialization.cloudpickle as pickle
    output = open(pickle_file_name,'wb')
    pickle.dump(pj, output)
    output.close()
    #print "DEBUG pickle_pj: %s"%str(os.stat(pickle_file_name).st_ctime)
    #print "DEBUG pickle_pj: %s"%str(os.stat(pickle_file_name).st_size)
    if __builtin__.git and track:
        import rpgit
        rpgit.gitAdd(pickle_file_name)
        comment = "A pickled Project from %s" % time.asctime()
        rpgit.gitCommit(comment) 
        
    return pickle_file_name
    
def unpickle_pj(pickle_file_name, git=True):
    import cloud.serialization.cloudpickle as pickle
    pickle_handle = open(pickle_file_name, 'rb')
    
    # fix some attr: add git_log if None, turn Confs to strings.
    pkl_pj = pickle.pickle.load(pickle_handle)
    new_pj = Project(pkl_pj.loci, git=False)
    
    # These do not need fixing
    attr_names = [ 'alignments',
                   'aln_summaries',
                   'concatenations',
                   'defaults',
                   'git_log',
                   'pickle_name',
                   'records',
                   'records_by_locus', 
                   'sets',
                   'starttime',
                   'trees',
                   'trimmed_alignments',
                   'user',
                 ]
    
    # Move the content into the new pj, Add git_log if missing
    for attr_name in attr_names:
        try:
            setattr(new_pj,attr_name,getattr(pkl_pj,attr_name))
        except:
            warnings.warn('Upgrading Project to v1')
    
    # upgrade used methods to dict, if list.
    if isinstance(pkl_pj.used_methods, list) and len(pkl_pj.used_methods) == 0:
        pkl_pj.used_methods = {}

    elif isinstance(pkl_pj.used_methods, list) and len(pkl_pj.used_methods) > 0:
        temp = {}
        for m in pkl_pj.used_methods:
            if isinstance(m, basestring):
                name = m.split()[1]
                temp[name] = m
            elif not isinstance(m, basestring):
                temp[m.method_name] = m
        pkl_pj.used_methods = temp
    
    # Turn Confs to strings
    for i in pkl_pj.used_methods:
        if isinstance(pkl_pj.used_methods[i], basestring):
            new_pj.used_methods[i] = pkl_pj.used_methods[i]
        else:
            new_pj.used_methods[i] = str(pkl_pj.used_methods[i])
    if git:
        start_git(new_pj)
    return new_pj


def revert_pickle(pj, commit_hash):
    pickle_filename = pj.pickle_name
    if not os.path.exists(pickle_filename):
        raise RuntimeError('Cannot find %s. Has the pickle been moved'%pickle_filename)
    cmd = 'git checkout %s -- %s'%(commit_hash, pickle_filename)
    pipe = sub.Popen(cmd, shell=True, stdout = sub.PIPE,stderr = sub.PIPE )
    (out, error) = pipe.communicate()
    print 'Git STDOUT: %s'%str(out)
    print 'Git STDERR: %s'%str(error)
    new_pj = unpickle_pj(pickle_filename)
    new_pj.git_log += "<<<<\nThe pickle was reverted to commit %s\nSTDOUT:\n%s\nSTDERR:%s\n>>>>\n"%(commit_hash,
                                                                                                   str(out),
                                                                                                   str(error))
    return new_pj
                                                                                                    
    


def publish(pj, folder_name, figures_folder, size='small',
            compare_trees=[], compare_meta=None, trees_to_compare='all',
            unrooted_trees=False):
    
    import os, time
    folder = None
    zip_file = None
    if folder_name.endswith('.zip'):
        zip_file = folder_name
        folder = folder_name[:-4]
    else:
        folder = folder_name
        zip_file = folder_name + '.zip'
    print "checking if file exists"
    if os.path.exists(folder) or os.path.exists(zip_file):
        raise IOError(folder_name + ' already exists')
    
    os.makedirs(folder)
    pj.write(folder+'/tree_and_alns.xml','phyloxml')
    
    #os.mkdir(folder+'/fasta_alignments')
    #file_names = pj.write_alns(id=['feature_id','original_id'])
    #for f in file_names:
    #    os.rename(f, "%s/fasta_alignments/%s"%(folder,f))
        
    #os.mkdir(folder+'/trimmed_fasta_alignments')
    #file_names = pj.write_alns(id=['feature_id','original_id'])
    #for f in file_names:
    #    os.rename(f, "%s/trimmed_fasta_alignments/%s"%(folder,f))
        
    from glob import glob
    from shutil import copyfile
    notebooks = glob('*.ipynb')
    for n in notebooks:
        copyfile(n, '%s/%s'%(folder,n))
        
    pj.write(folder+'/sequences_and_metadata.gb','genbank')
    report = open(folder+'/report.html','wt')
    lines = report_methods(pj, figures_folder, folder_name, size,
                           compare_trees=compare_trees, compare_meta=compare_meta,
                          trees_to_compare=trees_to_compare, unrooted_trees=unrooted_trees)
    for line in lines:
        report.write(line + '\n')
    report.close()

    #'report_lines' is now taking care of puting the figures in the zip folder, within /files
    #for tree in pj.trees.keys():
    #    if os.path.isfile(figures_folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png'):
    #        from shutil import copyfile
    #        copyfile(figures_folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png',
    #                 folder+'/'+pj.trees[tree][0].get_leaves()[0].tree_method_id+'.png')
            
         
    print "pickling"
    pickle_name = time.strftime("%a_%d_%b_%Y_%X", time.gmtime())+'.pkl'
    pickle_pj(pj, folder + '/' + pickle_name)
    
    hndl = open("%s/Bootstrap.css"%folder, 'wt')
    hndl.write(css.css())
    hndl.close()
    
    import zipfile, shutil
    print "archiving"
    zf = zipfile.ZipFile(zip_file, "w")
    for dirname, subdirs, files in os.walk(folder):
        zf.write(dirname)
        for filename in files:
            zf.write(os.path.join(dirname, filename))
    zf.close()
    shutil.rmtree(folder)
    print "report ready"

##################################################
if False:
    """ Robinson Foulds Pairwise tree distances"""
##################################################


# These will allow to do an RF distance to dendropy, where branch lengths
# are considered, but also to correct the branch length by the tree length
# and thus compare trees with very different evolutionary rates.

def get_tree_length(t):
    tree_length = 0
    for n in t.traverse():
        if not n.dist == 1: # ete puts 1 if there is no blen
            tree_length += n.dist
    return tree_length
    
def correct_branch_length_by_tree_length(branch_length, tree_length):
    return branch_length/float(tree_length)

def get_corrected_blen_dif(cor_blen1, cor_blen2):
    return abs(cor_blen1-cor_blen2)

def get_corrected_blen_dif_s(cor_blen1, cor_blen2):
    from math import pow
    return pow((cor_blen1-cor_blen2),2)
    
def flatten_to_strings(listOfLists):
    """Flatten a list of (lists of (lists of strings)) for any level 
    of nesting"""
    result = []

    for i in listOfLists:
        # Only append if i is a basestring (superclass of string)
        if isinstance(i, basestring):
            result.append(i)
        # Otherwise call this function recursively
        else:
            result.extend(flatten_to_strings(i))
    return result
    
def get_corrected_blen_rf(t1, t2, unrooted_trees=False):
    """
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> t2 = Tree("(A:0.3,(D:0.226,((C:0.784,B:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    
    Trees are the same:
    >>> T1 = T2 = t1
    >>> get_corrected_blen_rf(T1,T2)
    0.0
    
    Trees are different:
    >>> get_corrected_blen_rf(t1,t2)
    0.504654255319149
    
    Dendropy calc of these trees' RF:
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> dt2 = dendropy.Tree.get_from_string(t2.write(), schema="newick")
    >>> dt1.robinson_foulds_distance(dt2)
    3.652
    
    They are the same and blengths are 25% shorter in second tree. This function gives almost 0
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> t1_prop = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> for n in t1_prop.traverse():
    ...    if not n.dist == 1:
    ...        n.dist = 0.75*n.dist
    >>> get_corrected_blen_rf(t1,t1_prop)
    6.938893903907228e-17
    
    >>> str1 = "(A:1.1,((B:1.1,C:1.1):1.1,(D:1.1,E:1.1):1.1):1.1);"
    >>> str2 = "(A:2.2,((B:2.2,C:2.2):2.2,(D:2.2,E:2.2):2.2):2.2);"
    >>> get_corrected_blen_rf(Tree(str1), Tree(str2))
    0.0
    
    Dendropy gives almost 1 for the same trees:
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> dt1_prop = dendropy.Tree.get_from_string(t1_prop.write(), schema="newick")
    >>> dt1.robinson_foulds_distance(dt1_prop)
    0.7269999999999999
    
    So dendropy RF reflects tree length differences, this function cleans them out.
    
    One branch length changes in one tree, topology stays the same:
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.001,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> get_corrected_blen_rf(t1,t1_prop)
    0.23503571001673468
    
    Same trees (same topology, different branch-lengths) in ete's RF:
    >>> rf, max_rf, common_leaves, parts_t1, parts_t2 = t1.robinson_foulds(t1_prop)
    >>> rf
    0
    
    The raw rf calc in ete (the starting point of this func) does not reflect any 
    branch length differences, proportional or otherwise.
    """
    rf, max_rf, common_leaves, parts_t1, parts_t2 = t1.robinson_foulds(t2, unrooted_trees=unrooted_trees)
    #print 'DEBUG:', parts_t1
    #print 'DEBUG:', parts_t2
    tree_length1 = get_tree_length(t1)
    tree_length2 = get_tree_length(t2)
    distance = 0
    for part in parts_t1:
        if not part in parts_t2:
            raw_blen = t1.get_common_ancestor(flatten_to_strings(part)).dist
            if not raw_blen==1.0:
                distance += correct_branch_length_by_tree_length(raw_blen, tree_length1)
        elif part in parts_t2:
            #print 'DEBUG:', part
            raw_blen1 = t1.get_common_ancestor(flatten_to_strings(part)).dist
            raw_blen2 = t2.get_common_ancestor(flatten_to_strings(part)).dist
            if raw_blen1==1:
                raw_blen1 = 0
            if raw_blen2==1:
                raw_blen2 = 0
            cor_blen1 = correct_branch_length_by_tree_length(raw_blen1, tree_length1)
            cor_blen2 = correct_branch_length_by_tree_length(raw_blen2, tree_length2)
            distance += get_corrected_blen_dif(cor_blen1, cor_blen2)
    for part in parts_t2:
        if not part in parts_t1:
            raw_blen = t2.get_common_ancestor(flatten_to_strings(part)).dist
            if not raw_blen==1:
                distance += correct_branch_length_by_tree_length(raw_blen, tree_length2)
            
    #print "Debug: distance: %s"%str(distance)
    return distance
            
def get_corrected_kuhner_felsenstein(t1, t2, unrooted_trees=False):
    """
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> t2 = Tree("(A:0.3,(D:0.226,((C:0.784,B:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    
    Trees are the same:
    >>> T1 = T2 = t1
    >>> get_corrected_kuhner_felsenstein(T1,T2)
    0.0
    
    Trees are different:
    >>> get_corrected_kuhner_felsenstein(t1,t2)
    0.1273379587058624
    
    Dendropy calc of these trees' RF:
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> dt2 = dendropy.Tree.get_from_string(t2.write(), schema="newick")
    >>> dt1.robinson_foulds_distance(dt2)
    3.652
    
    They are the same and blengths are 25% shorter in second tree. This function gives almost 0
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> t1_prop = Tree("(A:0.3,(B:0.226,((C:0.784,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> for n in t1_prop.traverse():
    ...    if not n.dist == 1:
    ...        n.dist = 0.75*n.dist
    >>> get_corrected_kuhner_felsenstein(t1,t1_prop)
    3.274080905458301e-33
    
    >>> str1 = "(A:1.1,((B:1.1,C:1.1):1.1,(D:1.1,E:1.1):1.1):1.1);"
    >>> str2 = "(A:2.2,((B:2.2,C:2.2):2.2,(D:2.2,E:2.2):2.2):2.2);"
    >>> get_corrected_kuhner_felsenstein(Tree(str1), Tree(str2))
    0.0
    
    Dendropy gives almost 1 for the same trees:
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> dt1_prop = dendropy.Tree.get_from_string(t1_prop.write(), schema="newick")
    >>> dt1.robinson_foulds_distance(dt1_prop)
    0.7269999999999999
    
    So dendropy RF reflects tree length differences, this function cleans them out.
    
    One branch length changes in one tree, topology stays the same:
    >>> t1 = Tree("(A:0.3,(B:0.226,((C:0.001,D:0.159):0.759,(e:0.03,f:0.1):0.25)):0.3):0.1;")
    >>> dt1 = dendropy.Tree.get_from_string(t1.write(), schema="newick")
    >>> get_corrected_kuhner_felsenstein(t1,t1_prop)
    0.010930167133307158
    
    Same trees (same topology, different branch-lengths) in ete's RF:
    >>> rf, max_rf, common_leaves, parts_t1, parts_t2 = t1.robinson_foulds(t1_prop)
    >>> rf
    0
    
    The raw rf calc in ete (the starting point of this func) does not reflect any 
    branch length differences, proportional or otherwise.
    """
    from math import pow
    #print 'DEBUG: in kuhner_felsenstein'
    rf, max_rf, common_leaves, parts_t1, parts_t2 = t1.robinson_foulds(t2, unrooted_trees=unrooted_trees)
    #print 'DEBUG:', parts_t1
    #print 'DEBUG:', parts_t2
    tree_length1 = get_tree_length(t1)
    tree_length2 = get_tree_length(t2)
    distance = 0
    for part in parts_t1:
        if not part in parts_t2:
            raw_blen = t1.get_common_ancestor(flatten_to_strings(part)).dist
            if not raw_blen==1.0:
                distance += pow(correct_branch_length_by_tree_length(raw_blen, tree_length1),2)
        elif part in parts_t2:
            #print 'DEBUG:', part
            raw_blen1 = t1.get_common_ancestor(flatten_to_strings(part)).dist
            raw_blen2 = t2.get_common_ancestor(flatten_to_strings(part)).dist
            if raw_blen1==1:
                raw_blen1 = 0
            if raw_blen2==1:
                raw_blen2 = 0
            cor_blen1 = correct_branch_length_by_tree_length(raw_blen1, tree_length1)
            cor_blen2 = correct_branch_length_by_tree_length(raw_blen2, tree_length2)
            distance += get_corrected_blen_dif_s(cor_blen1, cor_blen2)
    for part in parts_t2:
        if not part in parts_t1:
            raw_blen = t2.get_common_ancestor(flatten_to_strings(part)).dist
            if not raw_blen==1:
                distance += pow(correct_branch_length_by_tree_length(raw_blen, tree_length2),2)
            
    #print "Debug: distance: %s"%str(distance)
    return distance            
        
def calc_rf(pj, figs_folder, rf_type='proportional',meta=None, mp_root=False, trees='all', unrooted_trees=False):
    """
    rf_types:
    topology: only topological diff
    branch-length: branch-length diffs
    proportional: branch-length proportion out of tree length diff
    deep-more-important: to-do
    """
    if len(pj.concatenations) > 0 and not meta:
        meta = pj.concatenations[0].otu_meta
    elif not meta:
        raise RuntimeError('RF calc does not know which meta to use to compare leaves')
        

    if trees == 'all':
        trees = pj.trees.keys()

    data = []

    for t1 in trees:
        line = []
        dupT1 = Tree(pj.trees[t1][0].write())
        for l in dupT1:
            for record in pj.records:
                for feature in record.features:
                    if feature.qualifiers['feature_id'][0] == l.name and meta in feature.qualifiers.keys():
                        l.name = feature.qualifiers[meta][0]
        if mp_root:
            R=dupT1.get_midpoint_outgroup()
            dupT1.set_outgroup(R)
        dupT1d = dendropy.Tree.get_from_string(dupT1.write(), schema="newick")
        for t2 in trees:
            dupT2 = Tree(pj.trees[t2][0].write())
            for l in dupT2:
                for record in pj.records:
                    for feature in record.features:
                        if feature.qualifiers['feature_id'][0] == l.name and meta in feature.qualifiers.keys():
                            l.name = feature.qualifiers[meta][0]
            if mp_root:
                R=dupT2.get_midpoint_outgroup()
                dupT2.set_outgroup(R)
            dupT2d = dendropy.Tree.get_from_string(dupT2.write(), schema="newick")
            if rf_type=='branch-length':
                rf = dupT1d.robinson_foulds_distance(dupT2d)
                line.append(rf) 
            elif rf_type=='topology':
                rf, max_rf, common_leaves, parts_t1, parts_t2 = dupT1.robinson_foulds(dupT2, unrooted_trees=unrooted_trees)
                line.append(rf/float(max_rf))   
            elif rf_type == 'proportional':
                #print 'DEBUG: in proportional'
                warnings.warn('proportional branch-distance: Trees must have the same taxa')
                line.append(get_corrected_kuhner_felsenstein(dupT1, dupT2, unrooted_trees=unrooted_trees))
            #to do
            #elif rf_type == deep_nodes more important:
            #    line.append(get_deep_important_rf(dupT1, dupT2))
        data.append(line)   
    
    row_labels = [str(i) for i in range(1, len(trees)+1)]
    column_labels = row_labels
    legend = [['#','LOCUS','ALIGNMENT METHOD','TRIMMING METHOD','TREE METHOD']]
    for i in trees:
        line = [str(trees.index(i)+1)]
        for val in i.split('@'):
            line.append(val)
        legend.append(line)
    fig, ax = plt.subplots()
    data = np.array(data)
    heatmap = ax.pcolor(data, cmap=plt.cm.Blues)

    # put the major ticks at the middle of each cell
    ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)

    # want a more natural, table-like display
    ax.invert_yaxis()
    ax.xaxis.tick_top()

    ax.set_xticklabels(row_labels, minor=False, size=14, rotation='vertical')
    ax.set_yticklabels(column_labels, minor=False, size=14)
    #fig.set_size_inches(12.5,12.5)
    try:
        fig.colorbar(heatmap, cmap=plt.cm.Blues)
    except:
        pass
    name = str(random.randint(1000,2000))
    fig.savefig(figs_folder + '/' + name +'.png')
    close('all')
    return figs_folder + '/' + name+'.png', legend 

def draw_trimal_scc(pj, num_col, figs_folder, trimmed=False, alg = '-scc'):
    import pandas as pd
    import matplotlib.pyplot as plt
    import random, os
    from Bio import AlignIO
    
    # get the alignment objects
    #-------------------------#
    alignments = pj.alignments.items()
    if trimmed:
        alignments = pj.trimmed_alignments.items()
    num_alns = len(alignments)

    subplots_arrangement = []
    #-----------------------#
    num_rows = round(float(num_alns)/num_col)
    if num_rows < float(num_alns)/num_col:
        num_rows += 1
        
    fig = plt.figure(figsize=(10*num_col,2.3*num_rows), dpi=80, frameon = False)
    plt.subplots_adjust(hspace = 0.8)
    subplots_arrangement += [num_rows, num_col]

    
    #Calc with trimal and plot
    #------------------------#
    for i in range(1,num_alns+1):
        import subprocess as sub
        subplot_position = subplots_arrangement +[i]
        aln_name = alignments[i-1][0]
        aln_obj = alignments[i-1][1]
        name = str(random.randint(1000,2000))+'_'+aln_name+'_for_trimal_graph.fasta'
        AlignIO.write(aln_obj, name, 'fasta')
        stderr = open('stderr','wt')
        #stdout = os.popen(programpath+'trimal '+alg+' -in '+name)#.read()
        stdout = sub.Popen(programspath+"trimal "+alg+" -in " + name,
                       shell=True, stdout=sub.PIPE, stderr=stderr).stdout
        stderr.close()
        var = pd.read_table(stdout, sep='\t+', skiprows=3, engine='python')
        os.remove('stderr')
        if alg == '-scc':
            var.columns = ['position', 'variability']
        elif alg == '-sgc':
            var.columns = ['position', 'pct_gaps', 'gap_score']
        
        #Plot residue similarity figure, for nucleotides this is identity value 
        fig.add_subplot(subplot_position[0], subplot_position[1], subplot_position[2])
        if alg == '-scc':
            var.variability.plot(color='g',lw=2)
        elif alg == '-sgc':
            var.pct_gaps.plot(color='g',lw=2)
        plt.title(aln_name.replace('@',' '), fontsize=14)
        plt.axis([1,len(aln_obj[0].seq), 0, 1.1]) #0 to 1 scale for y axis
        xlab = "alignment position"
        ylab = "similarity score"
        if alg == '-sgc':
            ylab = "percent gaps"
            plt.axis([1, len(aln_obj[0].seq), 0, 110]) #0 to 100 scale for y axis, ie percent
        plt.xlabel(xlab, fontsize=10)
        plt.ylabel(ylab, fontsize=10)
        plt.grid(True)
        os.remove(name)
    figname = str(random.randint(1000,2000))
    fig.savefig(figs_folder + '/' + figname +'.png')
    plt.close('all')
    return figs_folder + '/' + figname+'.png'

def view_csv_as_table(csv_filename, delimiter, quotechar='|'):
    with open(csv_filename, 'rb') as csvfile:
        sp_vs_lc = list(csv.reader(csvfile, delimiter=delimiter, quotechar=quotechar))
        field_sizes = []
        for i in range(len(sp_vs_lc[0])):
            lengths = []
            for row in sp_vs_lc:
                lengths.append(len(row[i]))
            field_sizes.append(max(lengths))
        for row in sp_vs_lc:
            string = ''
            for i in range(len(row)):
                string += row[i].ljust(field_sizes[i]+3)
            print string

def rfmt_tree_for_and_char_matirx_bayestraits(pj, qual_list, rootmeta, rootvalue, treefile=None, 
                              treetoken=None, treeburnin=0, treestep=1, treeformat=5):
    if treefile and treetoken:
        raise IOError("Only specify treefile ot treetoken, not both")
    T = None
    if treefile:
        T = open(treefile,'r').readlines()
        T = T[:-1]
    elif treetoken:
        T = [pj.ft(treetoken).write()]
    else:
        raise IOError("Specify treefile or treetoken")
    
    leaf_names = Tree(T[0].rstrip(), format=treeformat).get_leaf_names()
    
    translate = []
    i = 1
    for n in leaf_names:
        translate.append([n,str(i)])
        i +=1 
    
    char_matrix = ""
    for n in leaf_names:
        line = "%s "%n
        quals = get_qualifiers_dictionary(pj, n)
        for qual in qual_list:
            if qual in quals.keys():
                line += "%s "%str(quals[qual])
            else:
                line += "- "
        line = line[:-1]+'\n'
        char_matrix += line
            
    reformatted_tree = """#NEXUS
    Begin Trees;
        TRANSLATE
    """
    for t in translate[:-1]:
        reformatted_tree += '\t\t'+t[1]+'\t'+t[0]+',\n'
    reformatted_tree += '\t\t'+translate[-1][1]+'\t'+translate[-1][0]+';\n'
    
    for i in range(int(len(T)*treeburnin),len(T),treestep):
        j = T[i]
        newick = Tree(j.rstrip(), format=treeformat)
        brlns = []
        for n in newick.traverse():
            brlns.append(n.dist)
        if sorted(brlns)[2] == 0.0: # three 0 length branches - too much
            pass
        else:
            R = None
            count = 0
            for l in newick:
                if get_qualifiers_dictionary(pj, l.name)[rootmeta] == rootvalue:
                    R = l.name
                    count += 1
            if not count == 1:
                raise RuntimeError("%s does not exist or not unique in qualifier %s"%(rootvalue, rootmeta))
            newick.set_outgroup(newick&R)
            newick_str = newick.write(format=5)
            for t in translate:
                newick_str = re.sub(t[0],t[1],newick_str)
            reformatted_tree += 'Tree tree'+str(i)+'= '+newick_str+'\n'
    reformatted_tree += 'End;\n'
    
    return reformatted_tree, char_matrix
    
# Exonerate

def bayestraits(pj, qual_list, rootmeta, rootvalue,
                    treefile=None, treetoken=None,
                    treeburnin=0, treestep=1,
                    treeformat=5,
                    bayestraits = 'BayesTraits',
                    commands = [4,1,'kappa','delta','lambda','run']):
    # make command file
    import random
    rand = random.randint(1000000,9999999)
    cfile = open(str(rand),'wt')
    for i in commands:
        cfile.write(str(i)+'\n')
    cfile.close()
    reformatted_tree, char_matrix = rfmt_tree_for_and_char_matirx_bayestraits(pj, qual_list, rootmeta, rootvalue, treefile=treefile, 
                                                                              treetoken=treetoken, treeburnin=treeburnin, treestep=treestep,
                                                                              treeformat=treeformat)
    
    tfile = open(str(rand)+'.nex','wt')
    tfile.write(reformatted_tree)
    tfile.close()
    
    mfile = open(str(rand)+'.txt','wt')
    mfile.write(char_matrix)
    mfile.close()
    
    cline = "%s %s %s < %s" %(bayestraits, str(rand)+'.nex', str(rand)+'.txt', str(rand))
    import os
    stdout = os.popen(cline).read()
    os.remove(str(rand))
    os.remove(str(rand)+'.nex')
    os.remove(str(rand)+'.txt')
    return stdout


# Exonerate



class ExonerateCommandLine:
    
    """cline object with execute methods"""
        
    def __init__(self,
                 q, #query filename
                 t, #target filename
                 path='exonerate',
                 Q="unknown",# query alphabet
                 T="unknown", # target alphabet
                 querychunkid=0, #query job number
                 targetchunkid=0, #target job number
                 querychunktotal=0, #Num of queries
                 targetchunktotal=0, #Num of targets
                 E="FALSE", #exhaustive search
                 B="FALSE", #rapid comparison between long seqs
                 forcescan="none", #Force FSM scan on query or target sequences q or t
                 saturatethreshold=0, #word saturation threshold
                 customserver="NULL", # Custom command to send non-standard server
                 fastasuffix=".fa", #Fasta file suffix filter (in subdirectories)
                 m="ungapped",
                 s=100, #Score threshold for gapped alignment
                 percent=0.0, #Percent self-score threshold
                 showalignment="TRUE",
                 showsugar="FALSE",
                 showcigar="FALSE",
                 showvulgar="FALSE",
                 showquerygff="FALSE", # Include GFF output on query in results
                 showtargetgff="FALSE", #Include GFF output on target in results
                 ryo="NULL", #Roll-your-own printf-esque output format
                 n=0, #Report best N results per query
                 S="TRUE", #Search for suboptimal alignments
                 g="TRUE", #Use gapped extension
                 refine="none", #none|full|region
                 refineboundary=32, #Refinement region boundary
                 D=32, #Maximum memory to use for DP tracebacks (Mb)
                 C="TRUE", #Use compiled viterbi implementations
                 terminalrangeint=12, #Internal terminal range
                 terminalrangeext=12, #External terminal range
                 joinrangeint=12, #Internal join range
                 joinrangeext=12, #External join range
                 x=50, #Gapped extension threshold
                 singlepass="TRUE", #Generate suboptimal alignment in a single pass
                 joinfilter=0, #BSDP join filter threshold
                 A="none", #Path to sequence annotation file
                 softmaskquery="FALSE", #Allow softmasking on the query sequence
                 softmasktarget="FALSE", #Allow softmasking on the target sequence
                 d="nucleic", #DNA substitution matrix
                 p="blosum62", #Protein substitution matrix
                 M=64, #Memory limit for FSM scanning <Mb>
                 forcefsm="none", #Force FSM type ( normal | compact )
                 wordjump=1, #Jump between query words
                 o=-12, #Affine gap open penalty
                 e=-4, #Affine gap extend penalty
                 codongapopen=-18, #Codon affine gap open penalty
                 codongapextend=-8, #Codon affine gap extend penalty
                 minner=10, #Minimum NER length
                 maxner=50000, #Maximum NER length
                 neropen=-20, #NER open penalty
                 minintron=30, #Minimum intron length
                 maxintron=20000, #Maximum intron length
                 i=-30, #Intron Opening penalty
                 f=-28, #Frameshift creation penalty
                 useaatla="TRUE", #useaatla
                 geneticcode=1, #Use built-in or custom genetic code
                 hspfilter=0, #Aggressive HSP filtering level
                 useworddropoff="TRUE", #Use word neighbourhood dropoff
                 seedrepeat=1, #Seeds per diagonal required for HSP seeding
                 dnawordlen=12, #Wordlength for DNA words
                 proteinwordlen=6, #Wordlength for protein words
                 codonwordlen=12, #Wordlength for codon words
                 dnahspdropoff=30, #DNA HSP dropoff score
                 proteinhspdropoff=20, #Protein HSP dropoff score
                 codonhspdropoff=40, #Codon HSP dropoff score
                 dnahspthreshold=75, #DNA HSP threshold score
                 proteinhspthreshold=30, #Protein HSP threshold score
                 codonhspthreshold=50, #Codon HSP threshold score
                 dnawordlimit=0, #Score limit for dna word neighbourhood
                 proteinwordlimit=4, #Score limit for protein word neighbourhood
                 codonwordlimit=4, #Score limit for codon word neighbourhood
                 geneseed=0, #Geneseed Threshold
                 geneseedrepeat=3, #Seeds per diagonal required for geneseed HSP seeding
                 alignmentwidth=80, #Alignment display width
                 forwardcoordinates="TRUE", #Report all coordinates on the forward strand
                 quality=0, #HSP quality threshold
                 splice3="primate", #Supply frequency matrix for 3' splice sites
                 splice5="primate", #Supply frequency matrix for 5' splice sites
                 forcegtag="FALSE"): #Force use of gt...ag splice sites
        
        import inspect

        frame = inspect.currentframe()
        args, _, _, values = inspect.getargvalues(frame)

        cline = values['path']+' '
        
        for k in ('path','frame','inspect', 'self'):
            del values[k] 
        
        for keyward in values:
            
            dash = '-'
            if len(keyward) > 1:
                 dash = '--'
            cline += dash+keyward+' '+str(values[keyward])+' '
        self.cline_string = cline
        self.stdout = None

    def __str__(self):
        return self.cline_string
    
    def execute(self):
        import subprocess as sub
        p = sub.Popen(self.cline_string, shell=True, stdout=sub.PIPE, stderr=sub.PIPE)
        self.stdout = p.communicate()[0]
        return self.stdout

# an all inclusive exonerate ryo format 

roll = ("STARTRYOqfull = %qas @!!@!!@ qcds = %qcs @!!@!!@ qid = %qi @!!@!!@ qdescription = %qd @!!@!!@ "+
       "qlen = %qal @!!@!!@ qstrand = %qS @!!@!!@ qtype = %qt @!!@!!@ qbegin = %qab @!!@!!@ qend = %qae @!!@!!@ "+
       "tfull = %tas @!!@!!@ tcds = %tcs @!!@!!@ tid = %ti @!!@!!@ tdescription = %td @!!@!!@ "+
       "tlen = %tal @!!@!!@ tstrand = %tS @!!@!!@ ttype = %tt @!!@!!@ tbegin = %tab @!!@!!@ tend = %tae @!!@!!@ "+
       "Etotal = %et @!!@!!@ Eident = %ei @!!@!!@ Esim = %es @!!@!!@ Emis = %em @!!@!!@ Pident = %pi @!!@!!@ "+
       "Psim = %ps @!!@!!@ score = %s @!!@!!@ model = %m @!!@!!@ vulgar = %VENDRYO")

def parse_ryo(exo_results):
    
    """parses exonerate results that include the ryo line above"""
    
    stats = []
    ryos = [i.split("ENDRYO")[0] for i in exo_results.split('STARTRYO')[1:]][:-1]
    if len(ryos) > 0:
        for i in ryos:
            a = {}
            for line in i.split('@!!@!!@'):
                k, v = [line.partition('=')[0], line.partition('=')[2]]
                a[k.strip().rstrip()] = v.strip().rstrip()
            a['qfull'] = a['qfull'].replace('\n','').replace("\s",'')
            a['qcds'] = a['qcds'].replace('\n','').replace("\s",'')
            a['tfull'] = a['tfull'].replace('\n','').replace("\s",'')
            a['tcds'] = a['tcds'].replace('\n','').replace("\s",'')
            stats.append(a)
            
    return stats

def exonerate(q, d, **kwargs):
    
    """Will run exonerate with the ryo format above.
    Returns a dictionary with the ryo line content and the raw output as string"""
    
    if 'ryo' in kwargs.keys():
        kwargs.pop('ryo', None)
    results = ''
    exoCline = ExonerateCommandLine(q, d, ryo="\"%s\""%roll, **kwargs)
    results = exoCline.execute()
    #print results
    stats = parse_ryo(results)
    #print stats
    return stats, results


def exonerate_ryo_to_gb(q, d, stats, results, get_query=False):
    
    """takes the parsed ryo (stats) and the raw output (results) and
    builds a gb file for reprophylo"""
    
    if get_query:
        raise RuntimeError('get_query=True: currently only parses target. maybe Bio.SearchIO can help?')
    
    gencode = int(results.split('geneticcode ')[1].split()[0])
    model = stats[0]['model']
    if not model == 'protein2genome:local':
        raise RuntimeError("only tested with the protein2genome model")
    #gencode = '1'
    tfile = d
    if '/' in d:
        tfile = d.split('/')[-1]
    matches = stats
    b = 0
    records = []
    # making a short enough, yet unique and informative seq id is a challenge
    # The approach here is to take the 4 first and last chars of the input file name
    # and to add a serial number for each seq.
    
    # We add a random three digit number at the start
    # because the eight file name chars are not always unique accorss files.
    from random import randint
        
    rnd = randint(111,999)
    
    for match in matches:
        ID = ("%i|%s|%s|%i"%(rnd, tfile[:4],tfile[-4:],b)).replace('.','')
        
        r = SeqRecord(seq=Seq(match['tfull'],
                              alphabet=IUPAC.unambiguous_dna),
                      id = ID,
                      description = "query: %s %s, target: %s %s, HSPID: %i"%(match['qid'],
                                                                              match['qdescription'],
                                                                              match['tid'],
                                                                              match['tdescription'],
                                                                              b))
        b += 1
        source = SeqFeature(FeatureLocation(0, len(r.seq)), type='source')
        source.qualifiers['file'] = [tfile]
        for key in match:
            source.qualifiers[key] = [match[key]]
        r.features.append(source)
        vulgar = match['vulgar'].split()
        features = [vulgar[i:i+3] for i in range(0, len(vulgar) - 2, 3)]
        pos = 0
        CDS = None
        for feature in features:
            ftype = feature[0]
            flength = int(feature[2])
            f = SeqFeature(FeatureLocation(pos, pos+flength), type = ftype)
            f.qualifiers['gene'] = [match['qid']]
            pos += flength
            r.features.append(f)
        coding_locations = []
        for j in r.features:
            if j.type == 'M' or j.type == 'S':
                coding_locations.append(j.location)
            elif j.type == 'G' and (int(j.location.start) < int(j.location.end)):
                coding_locations.append(j.location)
        coding_locations = sorted(coding_locations, key = lambda l: int(l.start))
        if len(coding_locations) == 1:
            CDS = SeqFeature(coding_locations[0], type='CDS')
        else:
            CDS = SeqFeature(CompoundLocation(coding_locations), type='CDS')
        CDS.qualifiers['gene'] = [match['qid']]
        get = False
        CDS.qualifiers['translation'] = [str(CDS.extract(r.seq).translate(table=gencode)).replace('*','X')]
        if CDS.qualifiers['translation'][0] == str(Seq(match['tcds'].replace(' ','').replace('\n',''),
                                                       alphabet=IUPAC.ambiguous_dna).translate(table=gencode)).replace('*','X'):
            get = True
        else:
            CDS.qualifiers['translation'] = ['something went wrong']
            print "DEBUG bad CDS"
            print match['tid']
            #print len(CDS.extract(r.seq)), len(match['tcds'].replace(' ',''))
            #print str(CDS.extract(r.seq))[:20], str(CDS.extract(r.seq))[-20:]
            #print match['tcds'].replace(' ','')[:20], match['tcds'].replace(' ','')[-20:]
            print (str(CDS.extract(r.seq)) == match['tcds'].replace(' ',''))
            if (str(CDS.extract(r.seq)) == match['tcds'].replace(' ','')):
                print 'CDS retrieved correctly buy biopython could not translate'
            else:
                print 'Error in retrieved CDS (CDS built form vulgar does not match the CDS from ryo \%tcs'
            #print str(CDS.extract(r.seq))
        if get:
            r.features.append(CDS)
            records.append(r)
    a = SeqIO.write(records,'%s.gb'%d,'genbank')
    return "%i in %s.gb"%(a, d)

def report_aln_col_stat(pj, loci_names, num_col, figs_folder, trimmed=False, alg = '-scc'):
    import pandas as pd
    import matplotlib.pyplot as plt
    import random, os
    from Bio import AlignIO
    
    # get the alignment objects
    #-------------------------#
    alignments = pj.alignments.items()
    if trimmed:
        alignments = pj.trimmed_alignments.items()
    alignments = [i for i in alignments if i[0].split('@')[0] in loci_names]
    num_alns = len(alignments)
    subplots_arrangement = []
    #-----------------------#
    num_rows = round(float(num_alns)/num_col)
    if num_rows < float(num_alns)/num_col:
        num_rows += 1
        
    fig = plt.figure(figsize=(10*num_col,2.3*num_rows), dpi=80, frameon = False)
    plt.subplots_adjust(hspace = 0.8)
    subplots_arrangement += [num_rows, num_col]


    #Calc with trimal and plot
    #------------------------#
    for i in range(1,num_alns+1):
        import subprocess as sub
        subplot_position = subplots_arrangement +[i]
        aln_name = alignments[i-1][0]
        aln_obj = alignments[i-1][1]
        name = str(random.randint(1000,2000))+'_'+aln_name+'_for_trimal_graph.fasta'
        AlignIO.write(aln_obj, name, 'fasta')
        stderr = open('stderr','wt')
        #stdout = os.popen('trimal '+alg+' -in '+name)#.read()
        stdout = sub.Popen("trimal "+alg+" -in " + name,
                       shell=True, stdout=sub.PIPE, stderr=stderr).stdout
        stderr.close()
        var = pd.read_table(stdout, sep='\t+', skiprows=3, engine='python')
        os.remove('stderr')
        if alg == '-scc':
            var.columns = ['position', 'variability']
        elif alg == '-sgc':
            var.columns = ['position', 'pct_gaps', 'gap_score']
        
        #Plot residue similarity figure, for nucleotides this is identity value
        fig.add_subplot(subplot_position[0], subplot_position[1], subplot_position[2])
        if alg == '-scc':
            var.variability.plot(color='g',lw=2)
        elif alg == '-sgc':
            var.pct_gaps.plot(color='g',lw=2)
        plt.title(aln_name.replace('@',' '), fontsize=14)
        plt.axis([1,len(aln_obj[0].seq), 0, 1.1]) #0 to 1 scale for y axis
        xlab = "alignment position"
        ylab = "similarity score"
        if alg == '-sgc':
            ylab = "percent gaps"
            plt.axis([1, len(aln_obj[0].seq), 0, 110]) #0 to 100 scale for y axis, ie percent
        plt.xlabel(xlab, fontsize=10)
        plt.ylabel(ylab, fontsize=10)
        plt.grid(True)
        os.remove(name)
    figname = str(random.randint(1000,2000))
    fig.savefig(figs_folder + '/' + figname +'.png')
    plt.close('all')
    return figs_folder + '/' + figname+'.png'

##############################################################################################################
if False:
    """"LociStat class preliminaries"""
##############################################################################################################

def entropy(s, char_type):
    """
    Return the Shannon's entropy value for a column in the alignment provided as a string (s) 
    given the character type (dna or prot).
    
    gaps are ignored, ambiguity is ignored.
    
    
    homogenous column
    
    >>> entropy('tttttttt', 'dna')
    -0.0
    
    hetrogenous column, case insensitive
    
    >>> entropy('ttgGaacC', 'dna')
    2.0
    >>> entropy('ttggaacc', 'dna')
    2.0
    
    ignore gaps
    
    >>> entropy('ttgg--Ss', 'dna')
    1.0
    >>> entropy('ttggSs', 'dna')
    1.0
    
    recognize alphabet
    
    >>> entropy('ttggSs', 'prot')
    1.584962500721156
    """
    missing = None
    if char_type == 'prot':
        missing = 'Xx'
    elif char_type == 'dna':
        missing = 'ryswkmbdhvnRYSWKMBDHVN'
    s = s.replace('-','').replace('.','').replace('?','')
    for m in missing:
        s = s.replace(m,'')
    p, lns = Counter(s.lower()), float(len(s.lower()))
    
    return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

def gapscore(aln_obj):
    
    """
    Use  TrimAl to get a list of gapscores given a MultipleSeqAlignmnet
    object
    """
    
    import pandas as pd
    name = str(random.randint(1000,2000))+'_for_trimal_graph.fasta'
    AlignIO.write(aln_obj, name, 'fasta')
    stderr = open('stderr','wt')
    stdout = sub.Popen(programspath+"trimal -sgc -in " + name,
                       shell=True, stdout=sub.PIPE, stderr=stderr).stdout
    stderr.close()
    var = pd.read_table(stdout, sep='\t+', skiprows=3, engine='python')
    os.remove('stderr')
    var.columns = ['position', 'pct_gaps', 'gap_score']
    os.remove(name)  
    return [i[1] for i in var.to_dict()['gap_score'].items()]

def conservation(aln_obj):
    
    """
    Use  TrimAl to get a list of conservation values given a MultipleSeqAlignmnet
    object
    """
    import pandas as pd
    name = str(random.randint(1000,2000))+'_for_trimal_graph.fasta'
    AlignIO.write(aln_obj, name, 'fasta')
    stderr = open('stderr','wt')
    stdout = sub.Popen(programspath+"trimal -scc -in " + name,
                       shell=True, stdout=sub.PIPE, stderr=stderr).stdout
    stderr.close()
    var = pd.read_table(stdout, sep='\t+', skiprows=3, engine='python')
    os.remove('stderr')
    var.columns = ['position', 'conservation']
    os.remove(name)  
    return [i[1] for i in var.to_dict()['conservation'].items()]

def get_entropies(pj, trimmed = True, alignmnet_method=None, trimming_method=None):
    
    """
    Return a dictionary with alignment names as keys and  entropy keys as values
    given the alignments or trimmed alignmnets dictionary.
    
    
    If a locus has more than one alignment in the dictionary, go for the specified method
    or the first occurance
    """
    entropies = {}
    aln_dict = None
    if trimmed:
        aln_dict = pj.trimmed_alignments
        if aln_dict == {}:
            raise IOError("No trimed alignments in the Project")
                          
    elif not trimmed:
        aln_dict = pj.alignments
        if aln_dict == {}:
            raise IOError("No alignments in the Project")
        
    for aln_name in aln_dict.keys():
        char_type = None
        char_type_list = [l.char_type for l in pj.loci if l.name == aln_name.split('@')[0]]
        get = True
        if len(char_type_list) == 0:
            get = False
            warnings.warn('Cannot find Locus for alignment %s. Is it a supermatrix? Skipping.'%aln_name)            
        elif len(char_type_list) > 1:
            if (alignmnet_method and 
                not aln_name.split('@')[1] == alignmnet_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,alignmnet_method)
            elif (trimmed and trimming_method and 
                not aln_name.split('@')[2] == trimming_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,trimming_method)  
            elif aln_name.split('@')[0] in [i.split('@')[0] for i in entropies.keys()]:
                exists = [i for i in entropies.keys() if i.split('@')[0] == aln_name.split('@')[0]][0]
                get = False
                warning.warn('Skipping %s, already have %s'%aln_name, exists)
        if get:    
            char_type = char_type_list[0]
            aln_obj = aln_dict[aln_name]
            entropies[aln_name] =[]
            for i in range(aln_obj.get_alignment_length()):
                column = aln_obj[:,i]
                entropies[aln_name].append(entropy(column, char_type))

    return entropies

def get_gapscores(pj, trimmed = True, alignmnet_method=None, trimming_method=None):
    
    """
    Return a dictionary with alignment names as keys and  gap scores as values
    given the alignments or trimmed alignmnets dictionary.
    
    
    If a locus has more than one alignment in the dictionary, go for the specified method
    or the first occurance
    """
    gapscores = {}
    aln_dict = None
    if trimmed:
        aln_dict = pj.trimmed_alignments
        if aln_dict == {}:
            raise IOError("No trimed alignments in the Project")
    elif not trimmed:
        aln_dict = pj.alignments
        if aln_dict == {}:
            raise IOError("No alignments in the Project")
        
    for aln_name in aln_dict.keys():
        char_type = None
        char_type_list = [l.char_type for l in pj.loci if l.name == aln_name.split('@')[0]]
        get = True
        if len(char_type_list) == 0:
            get = False
            warnings.warn('Cannot find Locus for alignment %s. Is it a supermatrix? Skipping.'%aln_name)            
        elif len(char_type_list) > 1:
            if (alignmnet_method and 
                not aln_name.split('@')[1] == alignmnet_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,alignmnet_method)
            elif (trimmed and trimming_method and 
                not aln_name.split('@')[2] == trimming_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,trimming_method)  
            elif aln_name.split('@')[0] in [i.split('@')[0] for i in gapscores.keys()]:
                exists = [i for i in gapscores.keys() if i.split('@')[0] == aln_name.split('@')[0]][0]
                get = False
                warnings.warn('Skipping %s, already have %s'%aln_name, exists)   
        if get:    
            char_type = char_type_list[0]
            aln_obj = aln_dict[aln_name]
            gapscores[aln_name] = gapscore(aln_obj)  
    return gapscores

def get_conservations(pj, trimmed = True, alignmnet_method=None, trimming_method=None):
    """
    Return a dictionary with alignment names as keys and  conservation scores as values
    given the alignments or trimmed alignmnets dictionary.
    
    
    If a locus has more thn one alignment in the dictionary, go for the specified method
    or the first occurance
    """
    conservations = {}
    aln_dict = None
    if trimmed:
        aln_dict = pj.trimmed_alignments
        if aln_dict == {}:
            raise IOError("No trimed alignments in the Project")
            
    elif not trimmed:
        aln_dict = pj.alignments
        if aln_dict == {}:
            raise IOError("No alignments in the Project")
        
    for aln_name in aln_dict.keys():
        char_type = None
        char_type_list = [l.char_type for l in pj.loci if l.name == aln_name.split('@')[0]]
        get = True
        if len(char_type_list) == 0:
            get = False
            warnings.warn('Cannot find Locus for alignment %s. Is it a supermatrix? Skipping.'%aln_name)            
        elif len(char_type_list) > 1:
            if (alignmnet_method and 
                not aln_name.split('@')[1] == alignmnet_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,alignmnet_method)
            elif (trimmed and trimming_method and 
                not aln_name.split('@')[2] == trimming_method):
                get = False
                warnings.warn('Skipping %s, taking only %s'%aln_name,trimming_method) 
            elif aln_name.split('@')[0] in [i.split('@')[0] for i in conservations.keys()]:
                exists = [i for i in conservations.keys() if i.split('@')[0] == aln_name.split('@')[0]][0]
                get = False
                warnings.warn('Skipping %s, already have %s'%aln_name, exists)       
        if get:    
            char_type = char_type_list[0]
            aln_obj = aln_dict[aln_name]
            conservations[aln_name] = conservation(aln_obj)  
    return conservations

def get_sequence_lengths(pj):
    
    """
    Return a dictionary with locus names as keys and lists of sequence length as values
    given a Project instance.
    
    The length are calculated from the unaligned and untrimmed sequences
    """
    
    if len(pj.records) == 0:
            raise IOError('No records in the Project')
    
    lengths = {}

    
    if len(pj.records_by_locus) == 0:
        pj.extract_by_locus()
    
    for locus in pj.records_by_locus:
        lengths[locus] = []
        for r in pj.records_by_locus[locus]:
            lengths[locus].append(len(r.seq))
    return lengths

def get_sequence_gcs(pj):
    
    """
    Return a dictionary with locus names as keys and lists of sequence %GC as values
    given a Project instance.
    
    The %GC are calculated from the unaligned and untrimmed sequences
    """
    
    if len(pj.records) == 0:
            raise IOError('No records in the Project')
    
    gcs = {}
    
    if len(pj.records_by_locus) == 0:
        pj.extract_by_locus()
    
    if any([l.char_type == 'prot' for l in pj.loci]):
        warnings.warn('Protein loci GC content will be set to 0.0')
    
    for locus in pj.records_by_locus:
        char_type = [l for l in pj.loci if l.name == locus][0].char_type
        if char_type == 'dna':
            gcs[locus] = []
    
            for r in pj.records_by_locus[locus]:
                gcs[locus].append(GC(r.seq))
        else:
            gcs[locus] = [0.0, 0.0, 0.0, 0.0]
        
    return gcs
        
def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

#########################################################################################
class LociStats:
#########################################################################################
    
    def __init__(self, pj, trimmed=True, alignmnet_method=None, trimming_method=None):
        
        """
        >>> from reprophylo import *
        >>> pj = unpickle_pj('test-data/test_locistats', git=False)
        >>> stats = LociStats(pj)
        >>> entropies = stats.entropies['dummy@ReadDirectly@no_trim']
        >>> stats.sort()
        >>> abs(np.percentile(entropies,25))
        0.0
        >>> abs(np.percentile(entropies,50))
        0.0
        >>> conservation = stats.conservations['dummy1@ReadDirectly@no_trim']
        >>> np.percentile(conservation,25)
        0.0017470471212500001
        """
        
        self.loci = pj.loci
        
        self.entropies = get_entropies(pj,
                                       trimmed = trimmed,
                                       alignmnet_method=alignmnet_method,
                                       trimming_method=trimming_method)
        
        self.gapscores = get_gapscores(pj,
                                       trimmed = trimmed,
                                       alignmnet_method=alignmnet_method,
                                       trimming_method=trimming_method)
        
        self.conservations = get_conservations(pj,
                                               trimmed = trimmed,
                                               alignmnet_method=alignmnet_method,
                                               trimming_method=trimming_method)
        
        self.sequence_lengths = get_sequence_lengths(pj)
        
        self.sequence_gcs = get_sequence_gcs(pj)
        
        combined = []
        
        for key in self.sequence_lengths:
            locus_stats = [key]
            
            entropies = [self.entropies[i] for i in self.entropies.keys() if i.split('@')[0] == key][0]
            gapscores = [self.gapscores[i] for i in self.gapscores.keys() if i.split('@')[0] == key][0]
            conservations = [self.conservations[i] for i in self.conservations.keys() if i.split('@')[0] == key][0]
            
            combined.append([key,
                             self.sequence_lengths[key],
                             self.sequence_gcs[key],
                             entropies,
                             gapscores,
                             conservations])
            
        self.loci_stats = combined
        
        self.loci_stats_sorted = None
        
    def sort(self, parameter = 'entropy', percentile=50, percentile_range=(25,75), reverse = True):
        j = None
        if parameter == 'entropy':
            j = 3
        elif parameter == 'gapscore':
            j = 4
        elif parameter == 'conservation':
            j = 5
        elif parameter == 'sequence_length':
            j = 1
        elif parameter == 'sequence_gc':
            j = 2
        
        self.loci_stats_sorted = sorted(self.loci_stats,
                                        key=lambda i: (np.percentile(i[j], percentile),
                                                       abs(np.percentile(i[j], percentile_range[1])-
                                                           np.percentile(i[j], percentile_range[0]))),
                                        reverse=reverse)
        self.loci_stats_sorted = [list(i) for i in self.loci_stats_sorted]
        
    def plot(self, filename, figsize=(30,10), params='all', lable_fsize=40, xtick_fsize=4, ytick_fsize=4,
            boxcolor='salmon', whiskercolor='gray', capcolor='black', mediancolor='white', medianline_w=3):
        
        parameter_indices = [3,4,5,1,2]
        
        ytitles=['',
                 'Sequence Lengths',
                 'Sequence %GC',
                 'Entropy',
                 'Gap Score', 
                 'Conservation Scores']
        
        if not  params=='all':
            parameter_indices = []
            for param in params:
                if param == 'entropy':
                    parameter_indices.append(3)
                elif param == 'gapscore':
                    parameter_indices.append(4)
                elif param == 'conservation':
                    parameter_indices.append(5)
                elif param == 'sequence_length':
                    parameter_indices.append(1)
                elif param == 'sequence_gc':
                    parameter_indices.append(2)
        if len(parameter_indices) == 0:
            raise IOError('Must specify at least one parameter to plot')
        
        fig, axes = plt.subplots(len(parameter_indices), sharex=True, figsize=figsize, dpi=80, frameon = False)
        
        if len(parameter_indices) == 1:
               axes = [axes]
        
        labels = [k[0] for k in self.loci_stats_sorted]
        
        j = 0
        
        for ax in axes:
            
            values = [k[parameter_indices[j]] for k in self.loci_stats_sorted]
            
            bp = ax.boxplot(values,0,'', positions = range(4,(len(values)*4)+1, 4), patch_artist=True)
            ax.set_ylabel(ytitles[parameter_indices[j]], fontsize=lable_fsize)
            #plt.xlabel("Locus", fontsize=lable_fsize)
            plt.xticks(range(4,((len(values)+1)*4),4), labels, rotation=90, fontsize = xtick_fsize)
            plt.tight_layout()
            
            j += 1
            
            remove_border()
            for box in bp['boxes']:
            # change outline color
                box.set( color=boxcolor, linewidth=1)
                box.set_facecolor(boxcolor)
                    
            # change color, linestyle and linewidth of the whiskers
            for whisker in bp['whiskers']:
                whisker.set(color=whiskercolor, linestyle='solid', linewidth=2.0)
            
            # change color and linewidth of the caps
            for cap in bp['caps']:
                cap.set(color=capcolor, linewidth=2.0)
        
            # change color and linewidth of the medians
            for median in bp['medians']:
                median.set(color=mediancolor, linewidth=medianline_w)
        
        fig.gca().set_ylim(bottom=-0.05)
        fig.savefig(filename)
        
    def slice_loci(self, median_range, otu_meta, parameter='entropy',
                   otu_must_have_all_of=[], otu_must_have_one_of='any'):
        
        j = None
        if parameter == 'entropy':
            j = 3
        elif parameter == 'gapscore':
            j = 4
        elif parameter == 'conservation':
            j = 5
        elif parameter == 'sequence_length':
            j = 1
        elif parameter == 'sequence_gc':
            j = 2
        
        loci_names = [i[0] for i in self.loci_stats_sorted if median_range[0] < np.median(i[j]) < median_range[1]]
        loci = [l for l in self.loci if l.name in loci_names]
        concat_name = "%s_%.2f_%.2f_loci_%s_to_%s"%(parameter, float(median_range[0]), float(median_range[1]),
                                                    loci_names[0],loci_names[-1])
        
        return Concatenation(concat_name, loci,
                             otu_meta,
                             otu_must_have_all_of=otu_must_have_all_of,
                             otu_must_have_one_of=otu_must_have_one_of)
    

    
    def slide_loci(self, otu_meta, median_range='all', parameter='entropy', start=0, length=2, step=1,
                   otu_must_have_all_of=[],
                   otu_must_have_one_of='any'):
        
        j = None
        if parameter == 'entropy':
            j = 3
        elif parameter == 'gapscore':
            j = 4
        elif parameter == 'conservation':
            j = 5
        elif parameter == 'sequence_length':
            j = 1
        elif parameter == 'sequence_gc':
            j = 2
        
        if median_range == 'all':
            medians = [np.median(i) for i in [k[j] for k in self.loci_stats_sorted]]
            median_range = [min(medians), max(medians)]
            
        loci_in_range = [[i[0], np.median(i[j])] for i in self.loci_stats_sorted 
                          if median_range[0] <= np.median(i[j]) <= median_range[1]]
        
        concatenations = []
        
        stop = False
        
        while not stop:
            
            window_loci = loci_in_range[start: start+length]
                
            window_loci_names = [n[0] for n in window_loci]
            loci = [l for l in self.loci if l.name in window_loci_names]
            window_start_median = window_loci[0][1]
            window_end_median = window_loci[-1][1]
            concat_name = "%s_%.2f_%.2f_loci_%i_to_%i"%(parameter, float(window_start_median), float(window_end_median),
                                                        start, start+length-1)
            print concat_name
            concatenations.append(Concatenation(concat_name, loci,
                                                otu_meta,
                                                otu_must_have_all_of=otu_must_have_all_of,
                                                otu_must_have_one_of=otu_must_have_one_of))
            start = start+step
            
            if len(loci_in_range[start:]) < length:
                stop = True
        
        return concatenations
    
    

if __name__ == "__main__":
    import doctest
    doctest.testmod()

Overwriting reprophylo.py


In [3]:
!python reprophylo.py

/usr/local/lib/python2.7/dist-packages/pandas/io/excel.py:626: UserWarning: Installed openpyxl is not supported at this time. Use >=1.6.1 and <2.0.0.
  .format(openpyxl_compat.start_ver, openpyxl_compat.stop_ver))


In [3]:
!cp reprophylo.py /home/amir/work/galaxy-dist_old/tools/reprophylo/.
!cp reprophylo.py /home/amir/Dropbox/python_modules/.
!cp reprophylo.py /home/amir/Dropbox/Docker/ReproPhylo/.
!cp reprophylo.py /home/amir/Dropbox/Docker/ReproPhyloGalaxy/galaxy-dist/tools/reprophylo/.
!cp reprophylo.py /home/amir/galaxy-dist/tools/reprophylo/.
!sed -i "s/programspath \= \"\"/programspath \= \"\%s\/galaxy-dist\/tools\/reprophylo\/programs\/\"\%os.environ\[\'HOME\'\]/" /home/amir/Dropbox/Docker/ReproPhyloGalaxy/galaxy-dist/tools/reprophylo/reprophylo.py 
!sed -i "s/programspath \= \"\"/programspath \= \"\%s\/galaxy-dist\/tools\/reprophylo\/programs\/\"\%os.environ\[\'HOME\'\]/" /home/amir/work/galaxy-dist_old/tools/reprophylo/reprophylo.py
!sed -i "s/programspath \= \"\"/programspath \= \"\%s\/galaxy-dist\/tools\/reprophylo\/programs\/\"\%os.environ\[\'HOME\'\]/" /home/amir/galaxy-dist/tools/reprophylo/reprophylo.py

In [4]:
!cp rpgit.py /home/amir/Dropbox/python_modules/.
!cp rpgit.py /home/amir/Dropbox/Docker/ReproPhylo/.
!cp ./../Dependencies/gb_syn.py /home/amir/Dropbox/Docker/ReproPhylo/.
!cp ./../Dependencies/css.py /home/amir/Dropbox/Docker/ReproPhylo/.

In [2]:
%%file ./../Dependencies/gb_syn.py

def gb_syn():
    return  [
            ['MT-CO1','cox1','COX1','COI','coi','cytochrome oxidase 1','cytochrome oxidase I',
             'cytochrome oxidase subunit 1','cytochrome oxidase subunit I',
             'cox I','coxI', 'cytochrome c oxidase subunit I','CO1','cytochrome oxidase subunit 1',
            'cytochrome oxidase subunit I'],
            ['MT-CO2','cox2','COX2','COII','coii','cytochrome oxidase 2','cytochrome oxidase II',
             'cytochrome oxidase subunit 2','cytochrome oxidase subunit II','cox II',
             'cytochrome c oxidase subunit II'],
            ['MT-CO3','cox3','COX3','COIII','coiii','cytochrome oxidase 3',
             'cytochrome oxidase III','cytochrome oxidase subunit 3','cytochrome oxidase subunit III'
             ,'cox III','cytochrome c oxidase subunit III'],
            ['18s','18S','SSU rRNA','18S ribosomal RNA','small subunit 18S ribosomal RNA', '18S rRNA'],
            ['28s','28S','LSU rRNA','28S ribosomal RNA','28S large subunit ribosomal RNA'],
            ['MT-ATP6','atp6','ATP6','atpase6','atpase 6','ATPase6','ATPASE6'],
            ['MT-ATP8','atp8','ATP8','atpase8','atpase 8','ATPase8','ATPASE8'],
            ['ATP9','atp9'],
            ['MT-CYB','cytochrome-b','cytb','CYTB','Cytb', 'cytochrome B','cytochrome b','Cb',
            'CytB','Cyt_B','Cyt_b','cytB','cyt_b'],
            ['ef1a','Ef1a', 'elongation factor 1-alpha','elongation factor-1 alpha',
             'ef1-alpha','elongation factor 1 alpha'],
            ['MT-ND1','nd1','ND1','nadh1','NADH1','nadh 1','NADH 1',
             'NADH dehydrogenase subunit 1','NADH dehydrogenase subunit I','nad1','nadh1'],
            ['MT-ND2','nd2','ND2','nadh2','NADH2','nadh 2','NADH 2','NADH dehydrogenase subunit 2', 
             'NADH dehydrogenase subunit II','nad2','nadh2'],
            ['MT-ND3','nd3','ND3','nadh3','NADH3','nadh 3','NADH 3','NADH dehydrogenase subunit 3',
             'NADH dehydrogenase subunit III','nad3','nadh3'],
            ['MT-ND4','nd4','ND4','nadh4','NADH4','nadh 4','NADH 4','NADH dehydrogenase subunit 4',
             'NADH dehydrogenase subunit IV','nad4','nadh4'],
            ['MT-ND4L','nd4l','ND4L','nad4l','nadh4l', 'nadh4L', 'NADH dehydrogenase subunit 4l',
             'NADH dehydrogenase subunit 4L'],
            ['MT-ND5','nd5','ND5','nad5', 'nadh5','NADH5','nadh 5','NADH 5','NADH dehydrogenase subunit 5',
             'NADH dehydrogenase subunit V'],
            ['MT-ND6','nd6','ND6','nad6','nadh6','NADH6','nadh 6','NADH 6','NADH dehydrogenase subunit 6',
             'NADH dehydrogenase subunit VI'],
            ['rrnS','12srRNA','12s rRNA','12S ribosomal RNA','12S rRNA','12S r RNA',
             '12S small subunit ribosomal RNA','12 S rRNA','rrns','s-rRNA'],
            ['rrnL','16srRNA','16s rRNA','16S ribosomal RNA','16S rRNA'],
            ['C_mos', 'C-mos','c-mos','C-MOS'],
            ['GAPDH','gapdh'],
            ['RNApol2','RNApolII','RNA polymerase II']
            ]

Overwriting ./../Dependencies/gb_syn.py


In [40]:
%%file ./../Dependencies/css.py

def css():
    return  """/*!
 * Bootstrap v3.3.4 (http://getbootstrap.com)
 * Copyright 2011-2015 Twitter, Inc.
 * Licensed under MIT (https://github.com/twbs/bootstrap/blob/master/LICENSE)
 */

/*! normalize.css v3.0.2 | MIT License | git.io/normalize */
html {
  font-family: sans-serif;
  -webkit-text-size-adjust: 100%;
      -ms-text-size-adjust: 100%;
}
body {
  margin: 0;
}
article,
aside,
details,
figcaption,
figure,
footer,
header,
hgroup,
main,
menu,
nav,
section,
summary {
  display: block;
}
audio,
canvas,
progress,
video {
  display: inline-block;
  vertical-align: baseline;
}
audio:not([controls]) {
  display: none;
  height: 0;
}
[hidden],
template {
  display: none;
}
a {
  background-color: transparent;
}
a:active,
a:hover {
  outline: 0;
}
abbr[title] {
  border-bottom: 1px dotted;
}
b,
strong {
  font-weight: bold;
}
dfn {
  font-style: italic;
}
h1 {
  margin: .67em 0;
  font-size: 2em;
}
mark {
  color: #000;
  background: #ff0;
}
small {
  font-size: 80%;
}
sub,
sup {
  position: relative;
  font-size: 75%;
  line-height: 0;
  vertical-align: baseline;
}
sup {
  top: -.5em;
}
sub {
  bottom: -.25em;
}
img {
  border: 0;
}
svg:not(:root) {
  overflow: hidden;
}
figure {
  margin: 1em 40px;
}
hr {
  height: 0;
  -webkit-box-sizing: content-box;
     -moz-box-sizing: content-box;
          box-sizing: content-box;
}
pre {
  overflow: auto;
}
code,
kbd,
pre,
samp {
  font-family: monospace, monospace;
  font-size: 1em;
}
button,
input,
optgroup,
select,
textarea {
  margin: 0;
  font: inherit;
  color: inherit;
}
button {
  overflow: visible;
}
button,
select {
  text-transform: none;
}
button,
html input[type="button"],
input[type="reset"],
input[type="submit"] {
  -webkit-appearance: button;
  cursor: pointer;
}
button[disabled],
html input[disabled] {
  cursor: default;
}
button::-moz-focus-inner,
input::-moz-focus-inner {
  padding: 0;
  border: 0;
}
input {
  line-height: normal;
}
input[type="checkbox"],
input[type="radio"] {
  -webkit-box-sizing: border-box;
     -moz-box-sizing: border-box;
          box-sizing: border-box;
  padding: 0;
}
input[type="number"]::-webkit-inner-spin-button,
input[type="number"]::-webkit-outer-spin-button {
  height: auto;
}
input[type="search"] {
  -webkit-box-sizing: content-box;
     -moz-box-sizing: content-box;
          box-sizing: content-box;
  -webkit-appearance: textfield;
}
input[type="search"]::-webkit-search-cancel-button,
input[type="search"]::-webkit-search-decoration {
  -webkit-appearance: none;
}
fieldset {
  padding: .35em .625em .75em;
  margin: 0 2px;
  border: 1px solid #c0c0c0;
}
legend {
  padding: 0;
  border: 0;
}
textarea {
  overflow: auto;
}
optgroup {
  font-weight: bold;
}
table {
  border-spacing: 0;
  border-collapse: collapse;
}
td,
th {
  padding: 0;
}
/*! Source: https://github.com/h5bp/html5-boilerplate/blob/master/src/css/main.css */
@media print {
  *,
  *:before,
  *:after {
    color: #000 !important;
    text-shadow: none !important;
    background: transparent !important;
    -webkit-box-shadow: none !important;
            box-shadow: none !important;
  }
  a,
  a:visited {
    text-decoration: underline;
  }
  a[href]:after {
    content: " (" attr(href) ")";
  }
  abbr[title]:after {
    content: " (" attr(title) ")";
  }
  a[href^="#"]:after,
  a[href^="javascript:"]:after {
    content: "";
  }
  pre,
  blockquote {
    border: 1px solid #999;

    page-break-inside: avoid;
  }
  thead {
    display: table-header-group;
  }
  tr,
  img {
    page-break-inside: avoid;
  }
  img {
    max-width: 100% !important;
  }
  p,
  h2,
  h3 {
    orphans: 3;
    widows: 3;
  }
  h2,
  h3 {
    page-break-after: avoid;
  }
  select {
    background: #fff !important;
  }
  .navbar {
    display: none;
  }
  .btn > .caret,
  .dropup > .btn > .caret {
    border-top-color: #000 !important;
  }
  .label {
    border: 1px solid #000;
  }
  .table {
    border-collapse: collapse !important;
  }
  .table td,
  .table th {
    background-color: #fff !important;
  }
  .table-bordered th,
  .table-bordered td {
    border: 1px solid #ddd !important;
  }
}
@font-face {
  font-family: 'Glyphicons Halflings';

  src: url('../fonts/glyphicons-halflings-regular.eot');
  src: url('../fonts/glyphicons-halflings-regular.eot?#iefix') format('embedded-opentype'), url('../fonts/glyphicons-halflings-regular.woff2') format('woff2'), url('../fonts/glyphicons-halflings-regular.woff') format('woff'), url('../fonts/glyphicons-halflings-regular.ttf') format('truetype'), url('../fonts/glyphicons-halflings-regular.svg#glyphicons_halflingsregular') format('svg');
}
.glyphicon {
  position: relative;
  top: 1px;
  display: inline-block;
  font-family: 'Glyphicons Halflings';
  font-style: normal;
  font-weight: normal;
  line-height: 1;

  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}
.glyphicon-asterisk:before {
  content: "\2a";
}
.glyphicon-plus:before {
  content: "\2b";
}
.glyphicon-euro:before,
.glyphicon-eur:before {
  content: "\20ac";
}
.glyphicon-minus:before {
  content: "\2212";
}
.glyphicon-cloud:before {
  content: "\2601";
}
.glyphicon-envelope:before {
  content: "\2709";
}
.glyphicon-pencil:before {
  content: "\270f";
}
.glyphicon-glass:before {
  content: "\e001";
}
.glyphicon-music:before {
  content: "\e002";
}
.glyphicon-search:before {
  content: "\e003";
}
.glyphicon-heart:before {
  content: "\e005";
}
.glyphicon-star:before {
  content: "\e006";
}
.glyphicon-star-empty:before {
  content: "\e007";
}
.glyphicon-user:before {
  content: "\e008";
}
.glyphicon-film:before {
  content: "\e009";
}
.glyphicon-th-large:before {
  content: "\e010";
}
.glyphicon-th:before {
  content: "\e011";
}
.glyphicon-th-list:before {
  content: "\e012";
}
.glyphicon-ok:before {
  content: "\e013";
}
.glyphicon-remove:before {
  content: "\e014";
}
.glyphicon-zoom-in:before {
  content: "\e015";
}
.glyphicon-zoom-out:before {
  content: "\e016";
}
.glyphicon-off:before {
  content: "\e017";
}
.glyphicon-signal:before {
  content: "\e018";
}
.glyphicon-cog:before {
  content: "\e019";
}
.glyphicon-trash:before {
  content: "\e020";
}
.glyphicon-home:before {
  content: "\e021";
}
.glyphicon-file:before {
  content: "\e022";
}
.glyphicon-time:before {
  content: "\e023";
}
.glyphicon-road:before {
  content: "\e024";
}
.glyphicon-download-alt:before {
  content: "\e025";
}
.glyphicon-download:before {
  content: "\e026";
}
.glyphicon-upload:before {
  content: "\e027";
}
.glyphicon-inbox:before {
  content: "\e028";
}
.glyphicon-play-circle:before {
  content: "\e029";
}
.glyphicon-repeat:before {
  content: "\e030";
}
.glyphicon-refresh:before {
  content: "\e031";
}
.glyphicon-list-alt:before {
  content: "\e032";
}
.glyphicon-lock:before {
  content: "\e033";
}
.glyphicon-flag:before {
  content: "\e034";
}
.glyphicon-headphones:before {
  content: "\e035";
}
.glyphicon-volume-off:before {
  content: "\e036";
}
.glyphicon-volume-down:before {
  content: "\e037";
}
.glyphicon-volume-up:before {
  content: "\e038";
}
.glyphicon-qrcode:before {
  content: "\e039";
}
.glyphicon-barcode:before {
  content: "\e040";
}
.glyphicon-tag:before {
  content: "\e041";
}
.glyphicon-tags:before {
  content: "\e042";
}
.glyphicon-book:before {
  content: "\e043";
}
.glyphicon-bookmark:before {
  content: "\e044";
}
.glyphicon-print:before {
  content: "\e045";
}
.glyphicon-camera:before {
  content: "\e046";
}
.glyphicon-font:before {
  content: "\e047";
}
.glyphicon-bold:before {
  content: "\e048";
}
.glyphicon-italic:before {
  content: "\e049";
}
.glyphicon-text-height:before {
  content: "\e050";
}
.glyphicon-text-width:before {
  content: "\e051";
}
.glyphicon-align-left:before {
  content: "\e052";
}
.glyphicon-align-center:before {
  content: "\e053";
}
.glyphicon-align-right:before {
  content: "\e054";
}
.glyphicon-align-justify:before {
  content: "\e055";
}
.glyphicon-list:before {
  content: "\e056";
}
.glyphicon-indent-left:before {
  content: "\e057";
}
.glyphicon-indent-right:before {
  content: "\e058";
}
.glyphicon-facetime-video:before {
  content: "\e059";
}
.glyphicon-picture:before {
  content: "\e060";
}
.glyphicon-map-marker:before {
  content: "\e062";
}
.glyphicon-adjust:before {
  content: "\e063";
}
.glyphicon-tint:before {
  content: "\e064";
}
.glyphicon-edit:before {
  content: "\e065";
}
.glyphicon-share:before {
  content: "\e066";
}
.glyphicon-check:before {
  content: "\e067";
}
.glyphicon-move:before {
  content: "\e068";
}
.glyphicon-step-backward:before {
  content: "\e069";
}
.glyphicon-fast-backward:before {
  content: "\e070";
}
.glyphicon-backward:before {
  content: "\e071";
}
.glyphicon-play:before {
  content: "\e072";
}
.glyphicon-pause:before {
  content: "\e073";
}
.glyphicon-stop:before {
  content: "\e074";
}
.glyphicon-forward:before {
  content: "\e075";
}
.glyphicon-fast-forward:before {
  content: "\e076";
}
.glyphicon-step-forward:before {
  content: "\e077";
}
.glyphicon-eject:before {
  content: "\e078";
}
.glyphicon-chevron-left:before {
  content: "\e079";
}
.glyphicon-chevron-right:before {
  content: "\e080";
}
.glyphicon-plus-sign:before {
  content: "\e081";
}
.glyphicon-minus-sign:before {
  content: "\e082";
}
.glyphicon-remove-sign:before {
  content: "\e083";
}
.glyphicon-ok-sign:before {
  content: "\e084";
}
.glyphicon-question-sign:before {
  content: "\e085";
}
.glyphicon-info-sign:before {
  content: "\e086";
}
.glyphicon-screenshot:before {
  content: "\e087";
}
.glyphicon-remove-circle:before {
  content: "\e088";
}
.glyphicon-ok-circle:before {
  content: "\e089";
}
.glyphicon-ban-circle:before {
  content: "\e090";
}
.glyphicon-arrow-left:before {
  content: "\e091";
}
.glyphicon-arrow-right:before {
  content: "\e092";
}
.glyphicon-arrow-up:before {
  content: "\e093";
}
.glyphicon-arrow-down:before {
  content: "\e094";
}
.glyphicon-share-alt:before {
  content: "\e095";
}
.glyphicon-resize-full:before {
  content: "\e096";
}
.glyphicon-resize-small:before {
  content: "\e097";
}
.glyphicon-exclamation-sign:before {
  content: "\e101";
}
.glyphicon-gift:before {
  content: "\e102";
}
.glyphicon-leaf:before {
  content: "\e103";
}
.glyphicon-fire:before {
  content: "\e104";
}
.glyphicon-eye-open:before {
  content: "\e105";
}
.glyphicon-eye-close:before {
  content: "\e106";
}
.glyphicon-warning-sign:before {
  content: "\e107";
}
.glyphicon-plane:before {
  content: "\e108";
}
.glyphicon-calendar:before {
  content: "\e109";
}
.glyphicon-random:before {
  content: "\e110";
}
.glyphicon-comment:before {
  content: "\e111";
}
.glyphicon-magnet:before {
  content: "\e112";
}
.glyphicon-chevron-up:before {
  content: "\e113";
}
.glyphicon-chevron-down:before {
  content: "\e114";
}
.glyphicon-retweet:before {
  content: "\e115";
}
.glyphicon-shopping-cart:before {
  content: "\e116";
}
.glyphicon-folder-close:before {
  content: "\e117";
}
.glyphicon-folder-open:before {
  content: "\e118";
}
.glyphicon-resize-vertical:before {
  content: "\e119";
}
.glyphicon-resize-horizontal:before {
  content: "\e120";
}
.glyphicon-hdd:before {
  content: "\e121";
}
.glyphicon-bullhorn:before {
  content: "\e122";
}
.glyphicon-bell:before {
  content: "\e123";
}
.glyphicon-certificate:before {
  content: "\e124";
}
.glyphicon-thumbs-up:before {
  content: "\e125";
}
.glyphicon-thumbs-down:before {
  content: "\e126";
}
.glyphicon-hand-right:before {
  content: "\e127";
}
.glyphicon-hand-left:before {
  content: "\e128";
}
.glyphicon-hand-up:before {
  content: "\e129";
}
.glyphicon-hand-down:before {
  content: "\e130";
}
.glyphicon-circle-arrow-right:before {
  content: "\e131";
}
.glyphicon-circle-arrow-left:before {
  content: "\e132";
}
.glyphicon-circle-arrow-up:before {
  content: "\e133";
}
.glyphicon-circle-arrow-down:before {
  content: "\e134";
}
.glyphicon-globe:before {
  content: "\e135";
}
.glyphicon-wrench:before {
  content: "\e136";
}
.glyphicon-tasks:before {
  content: "\e137";
}
.glyphicon-filter:before {
  content: "\e138";
}
.glyphicon-briefcase:before {
  content: "\e139";
}
.glyphicon-fullscreen:before {
  content: "\e140";
}
.glyphicon-dashboard:before {
  content: "\e141";
}
.glyphicon-paperclip:before {
  content: "\e142";
}
.glyphicon-heart-empty:before {
  content: "\e143";
}
.glyphicon-link:before {
  content: "\e144";
}
.glyphicon-phone:before {
  content: "\e145";
}
.glyphicon-pushpin:before {
  content: "\e146";
}
.glyphicon-usd:before {
  content: "\e148";
}
.glyphicon-gbp:before {
  content: "\e149";
}
.glyphicon-sort:before {
  content: "\e150";
}
.glyphicon-sort-by-alphabet:before {
  content: "\e151";
}
.glyphicon-sort-by-alphabet-alt:before {
  content: "\e152";
}
.glyphicon-sort-by-order:before {
  content: "\e153";
}
.glyphicon-sort-by-order-alt:before {
  content: "\e154";
}
.glyphicon-sort-by-attributes:before {
  content: "\e155";
}
.glyphicon-sort-by-attributes-alt:before {
  content: "\e156";
}
.glyphicon-unchecked:before {
  content: "\e157";
}
.glyphicon-expand:before {
  content: "\e158";
}
.glyphicon-collapse-down:before {
  content: "\e159";
}
.glyphicon-collapse-up:before {
  content: "\e160";
}
.glyphicon-log-in:before {
  content: "\e161";
}
.glyphicon-flash:before {
  content: "\e162";
}
.glyphicon-log-out:before {
  content: "\e163";
}
.glyphicon-new-window:before {
  content: "\e164";
}
.glyphicon-record:before {
  content: "\e165";
}
.glyphicon-save:before {
  content: "\e166";
}
.glyphicon-open:before {
  content: "\e167";
}
.glyphicon-saved:before {
  content: "\e168";
}
.glyphicon-import:before {
  content: "\e169";
}
.glyphicon-export:before {
  content: "\e170";
}
.glyphicon-send:before {
  content: "\e171";
}
.glyphicon-floppy-disk:before {
  content: "\e172";
}
.glyphicon-floppy-saved:before {
  content: "\e173";
}
.glyphicon-floppy-remove:before {
  content: "\e174";
}
.glyphicon-floppy-save:before {
  content: "\e175";
}
.glyphicon-floppy-open:before {
  content: "\e176";
}
.glyphicon-credit-card:before {
  content: "\e177";
}
.glyphicon-transfer:before {
  content: "\e178";
}
.glyphicon-cutlery:before {
  content: "\e179";
}
.glyphicon-header:before {
  content: "\e180";
}
.glyphicon-compressed:before {
  content: "\e181";
}
.glyphicon-earphone:before {
  content: "\e182";
}
.glyphicon-phone-alt:before {
  content: "\e183";
}
.glyphicon-tower:before {
  content: "\e184";
}
.glyphicon-stats:before {
  content: "\e185";
}
.glyphicon-sd-video:before {
  content: "\e186";
}
.glyphicon-hd-video:before {
  content: "\e187";
}
.glyphicon-subtitles:before {
  content: "\e188";
}
.glyphicon-sound-stereo:before {
  content: "\e189";
}
.glyphicon-sound-dolby:before {
  content: "\e190";
}
.glyphicon-sound-5-1:before {
  content: "\e191";
}
.glyphicon-sound-6-1:before {
  content: "\e192";
}
.glyphicon-sound-7-1:before {
  content: "\e193";
}
.glyphicon-copyright-mark:before {
  content: "\e194";
}
.glyphicon-registration-mark:before {
  content: "\e195";
}
.glyphicon-cloud-download:before {
  content: "\e197";
}
.glyphicon-cloud-upload:before {
  content: "\e198";
}
.glyphicon-tree-conifer:before {
  content: "\e199";
}
.glyphicon-tree-deciduous:before {
  content: "\e200";
}
.glyphicon-cd:before {
  content: "\e201";
}
.glyphicon-save-file:before {
  content: "\e202";
}
.glyphicon-open-file:before {
  content: "\e203";
}
.glyphicon-level-up:before {
  content: "\e204";
}
.glyphicon-copy:before {
  content: "\e205";
}
.glyphicon-paste:before {
  content: "\e206";
}
.glyphicon-alert:before {
  content: "\e209";
}
.glyphicon-equalizer:before {
  content: "\e210";
}
.glyphicon-king:before {
  content: "\e211";
}
.glyphicon-queen:before {
  content: "\e212";
}
.glyphicon-pawn:before {
  content: "\e213";
}
.glyphicon-bishop:before {
  content: "\e214";
}
.glyphicon-knight:before {
  content: "\e215";
}
.glyphicon-baby-formula:before {
  content: "\e216";
}
.glyphicon-tent:before {
  content: "\26fa";
}
.glyphicon-blackboard:before {
  content: "\e218";
}
.glyphicon-bed:before {
  content: "\e219";
}
.glyphicon-apple:before {
  content: "\f8ff";
}
.glyphicon-erase:before {
  content: "\e221";
}
.glyphicon-hourglass:before {
  content: "\231b";
}
.glyphicon-lamp:before {
  content: "\e223";
}
.glyphicon-duplicate:before {
  content: "\e224";
}
.glyphicon-piggy-bank:before {
  content: "\e225";
}
.glyphicon-scissors:before {
  content: "\e226";
}
.glyphicon-bitcoin:before {
  content: "\e227";
}
.glyphicon-btc:before {
  content: "\e227";
}
.glyphicon-xbt:before {
  content: "\e227";
}
.glyphicon-yen:before {
  content: "\00a5";
}
.glyphicon-jpy:before {
  content: "\00a5";
}
.glyphicon-ruble:before {
  content: "\20bd";
}
.glyphicon-rub:before {
  content: "\20bd";
}
.glyphicon-scale:before {
  content: "\e230";
}
.glyphicon-ice-lolly:before {
  content: "\e231";
}
.glyphicon-ice-lolly-tasted:before {
  content: "\e232";
}
.glyphicon-education:before {
  content: "\e233";
}
.glyphicon-option-horizontal:before {
  content: "\e234";
}
.glyphicon-option-vertical:before {
  content: "\e235";
}
.glyphicon-menu-hamburger:before {
  content: "\e236";
}
.glyphicon-modal-window:before {
  content: "\e237";
}
.glyphicon-oil:before {
  content: "\e238";
}
.glyphicon-grain:before {
  content: "\e239";
}
.glyphicon-sunglasses:before {
  content: "\e240";
}
.glyphicon-text-size:before {
  content: "\e241";
}
.glyphicon-text-color:before {
  content: "\e242";
}
.glyphicon-text-background:before {
  content: "\e243";
}
.glyphicon-object-align-top:before {
  content: "\e244";
}
.glyphicon-object-align-bottom:before {
  content: "\e245";
}
.glyphicon-object-align-horizontal:before {
  content: "\e246";
}
.glyphicon-object-align-left:before {
  content: "\e247";
}
.glyphicon-object-align-vertical:before {
  content: "\e248";
}
.glyphicon-object-align-right:before {
  content: "\e249";
}
.glyphicon-triangle-right:before {
  content: "\e250";
}
.glyphicon-triangle-left:before {
  content: "\e251";
}
.glyphicon-triangle-bottom:before {
  content: "\e252";
}
.glyphicon-triangle-top:before {
  content: "\e253";
}
.glyphicon-console:before {
  content: "\e254";
}
.glyphicon-superscript:before {
  content: "\e255";
}
.glyphicon-subscript:before {
  content: "\e256";
}
.glyphicon-menu-left:before {
  content: "\e257";
}
.glyphicon-menu-right:before {
  content: "\e258";
}
.glyphicon-menu-down:before {
  content: "\e259";
}
.glyphicon-menu-up:before {
  content: "\e260";
}
* {
  -webkit-box-sizing: border-box;
     -moz-box-sizing: border-box;
          box-sizing: border-box;
}
*:before,
*:after {
  -webkit-box-sizing: border-box;
     -moz-box-sizing: border-box;
          box-sizing: border-box;
}
html {
  font-size: 10px;

  -webkit-tap-highlight-color: rgba(0, 0, 0, 0);
}
body {
  font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
  font-size: 14px;
  line-height: 1.42857143;
  color: #333;
  background-color: #fff;
}
input,
button,
select,
textarea {
  font-family: inherit;
  font-size: inherit;
  line-height: inherit;
}
a {
  color: #337ab7;
  text-decoration: none;
}
a:hover,
a:focus {
  color: #23527c;
  text-decoration: underline;
}
a:focus {
  outline: thin dotted;
  outline: 5px auto -webkit-focus-ring-color;
  outline-offset: -2px;
}
figure {
  margin: 0;
}
img {
  vertical-align: middle;
}
.img-responsive,
.thumbnail > img,
.thumbnail a > img,
.carousel-inner > .item > img,
.carousel-inner > .item > a > img {
  display: block;
  max-width: 100%;
  height: auto;
}
.img-rounded {
  border-radius: 6px;
}
.img-thumbnail {
  display: inline-block;
  max-width: 100%;
  height: auto;
  padding: 4px;
  line-height: 1.42857143;
  background-color: #fff;
  border: 1px solid #ddd;
  border-radius: 4px;
  -webkit-transition: all .2s ease-in-out;
       -o-transition: all .2s ease-in-out;
          transition: all .2s ease-in-out;
}
.img-circle {
  border-radius: 50%;
}
hr {
  margin-top: 20px;
  margin-bottom: 20px;
  border: 0;
  border-top: 1px solid #eee;
}
.sr-only {
  position: absolute;
  width: 1px;
  height: 1px;
  padding: 0;
  margin: -1px;
  overflow: hidden;
  clip: rect(0, 0, 0, 0);
  border: 0;
}
.sr-only-focusable:active,
.sr-only-focusable:focus {
  position: static;
  width: auto;
  height: auto;
  margin: 0;
  overflow: visible;
  clip: auto;
}
[role="button"] {
  cursor: pointer;
}
h1,
h2,
h3,
h4,
h5,
h6,
.h1,
.h2,
.h3,
.h4,
.h5,
.h6 {
  font-family: inherit;
  font-weight: 500;
  line-height: 1.1;
  color: inherit;
}
h1 small,
h2 small,
h3 small,
h4 small,
h5 small,
h6 small,
.h1 small,
.h2 small,
.h3 small,
.h4 small,
.h5 small,
.h6 small,
h1 .small,
h2 .small,
h3 .small,
h4 .small,
h5 .small,
h6 .small,
.h1 .small,
.h2 .small,
.h3 .small,
.h4 .small,
.h5 .small,
.h6 .small {
  font-weight: normal;
  line-height: 1;
  color: #777;
}
h1,
.h1,
h2,
.h2,
h3,
.h3 {
  margin-top: 20px;
  margin-bottom: 10px;
}
h1 small,
.h1 small,
h2 small,
.h2 small,
h3 small,
.h3 small,
h1 .small,
.h1 .small,
h2 .small,
.h2 .small,
h3 .small,
.h3 .small {
  font-size: 65%;
}
h4,
.h4,
h5,
.h5,
h6,
.h6 {
  margin-top: 10px;
  margin-bottom: 10px;
}
h4 small,
.h4 small,
h5 small,
.h5 small,
h6 small,
.h6 small,
h4 .small,
.h4 .small,
h5 .small,
.h5 .small,
h6 .small,
.h6 .small {
  font-size: 75%;
}
h1,
.h1 {
  font-size: 36px;
}
h2,
.h2 {
  font-size: 30px;
}
h3,
.h3 {
  font-size: 24px;
}
h4,
.h4 {
  font-size: 18px;
}
h5,
.h5 {
  font-size: 14px;
}
h6,
.h6 {
  font-size: 12px;
}
p {
  margin: 0 0 10px;
}
.lead {
  margin-bottom: 20px;
  font-size: 16px;
  font-weight: 300;
  line-height: 1.4;
}
@media (min-width: 768px) {
  .lead {
    font-size: 21px;
  }
}
small,
.small {
  font-size: 85%;
}
mark,
.mark {
  padding: .2em;
  background-color: #fcf8e3;
}
.text-left {
  text-align: left;
}
.text-right {
  text-align: right;
}
.text-center {
  text-align: center;
}
.text-justify {
  text-align: justify;
}
.text-nowrap {
  white-space: nowrap;
}
.text-lowercase {
  text-transform: lowercase;
}
.text-uppercase {
  text-transform: uppercase;
}
.text-capitalize {
  text-transform: capitalize;
}
.text-muted {
  color: #777;
}
.text-primary {
  color: #337ab7;
}
a.text-primary:hover {
  color: #286090;
}
.text-success {
  color: #3c763d;
}
a.text-success:hover {
  color: #2b542c;
}
.text-info {
  color: #31708f;
}
a.text-info:hover {
  color: #245269;
}
.text-warning {
  color: #8a6d3b;
}
a.text-warning:hover {
  color: #66512c;
}
.text-danger {
  color: #a94442;
}
a.text-danger:hover {
  color: #843534;
}
.bg-primary {
  color: #fff;
  background-color: #337ab7;
}
a.bg-primary:hover {
  background-color: #286090;
}
.bg-success {
  background-color: #dff0d8;
}
a.bg-success:hover {
  background-color: #c1e2b3;
}
.bg-info {
  background-color: #d9edf7;
}
a.bg-info:hover {
  background-color: #afd9ee;
}
.bg-warning {
  background-color: #fcf8e3;
}
a.bg-warning:hover {
  background-color: #f7ecb5;
}
.bg-danger {
  background-color: #f2dede;
}
a.bg-danger:hover {
  background-color: #e4b9b9;
}
.page-header {
  padding-bottom: 9px;
  margin: 40px 0 20px;
  border-bottom: 1px solid #eee;
}
ul,
ol {
  margin-top: 0;
  margin-bottom: 10px;
}
ul ul,
ol ul,
ul ol,
ol ol {
  margin-bottom: 0;
}
.list-unstyled {
  padding-left: 0;
  list-style: none;
}
.list-inline {
  padding-left: 0;
  margin-left: -5px;
  list-style: none;
}
.list-inline > li {
  display: inline-block;
  padding-right: 5px;
  padding-left: 5px;
}
dl {
  margin-top: 0;
  margin-bottom: 20px;
}
dt,
dd {
  line-height: 1.42857143;
}
dt {
  font-weight: bold;
}
dd {
  margin-left: 0;
}
@media (min-width: 768px) {
  .dl-horizontal dt {
    float: left;
    width: 160px;
    overflow: hidden;
    clear: left;
    text-align: right;
    text-overflow: ellipsis;
    white-space: nowrap;
  }
  .dl-horizontal dd {
    margin-left: 180px;
  }
}
abbr[title],
abbr[data-original-title] {
  cursor: help;
  border-bottom: 1px dotted #777;
}
.initialism {
  font-size: 90%;
  text-transform: uppercase;
}
blockquote {
  padding: 10px 20px;
  margin: 0 0 20px;
  font-size: 17.5px;
  border-left: 5px solid #eee;
}
blockquote p:last-child,
blockquote ul:last-child,
blockquote ol:last-child {
  margin-bottom: 0;
}
blockquote footer,
blockquote small,
blockquote .small {
  display: block;
  font-size: 80%;
  line-height: 1.42857143;
  color: #777;
}
blockquote footer:before,
blockquote small:before,
blockquote .small:before {
  content: '\2014 \00A0';
}
.blockquote-reverse,
blockquote.pull-right {
  padding-right: 15px;
  padding-left: 0;
  text-align: right;
  border-right: 5px solid #eee;
  border-left: 0;
}
.blockquote-reverse footer:before,
blockquote.pull-right footer:before,
.blockquote-reverse small:before,
blockquote.pull-right small:before,
.blockquote-reverse .small:before,
blockquote.pull-right .small:before {
  content: '';
}
.blockquote-reverse footer:after,
blockquote.pull-right footer:after,
.blockquote-reverse small:after,
blockquote.pull-right small:after,
.blockquote-reverse .small:after,
blockquote.pull-right .small:after {
  content: '\00A0 \2014';
}
address {
  margin-bottom: 20px;
  font-style: normal;
  line-height: 1.42857143;
}
code,
kbd,
pre,
samp {
  font-family: Menlo, Monaco, Consolas, "Courier New", monospace;
}
code {
  padding: 2px 4px;
  font-size: 90%;
  color: #c7254e;
  background-color: #f9f2f4;
  border-radius: 4px;
}
kbd {
  padding: 2px 4px;
  font-size: 90%;
  color: #fff;
  background-color: #333;
  border-radius: 3px;
  -webkit-box-shadow: inset 0 -1px 0 rgba(0, 0, 0, .25);
          box-shadow: inset 0 -1px 0 rgba(0, 0, 0, .25);
}
kbd kbd {
  padding: 0;
  font-size: 100%;
  font-weight: bold;
  -webkit-box-shadow: none;
          box-shadow: none;
}
pre {
  display: block;
  padding: 9.5px;
  margin: 0 0 10px;
  font-size: 13px;
  line-height: 1.42857143;
  color: #333;
  word-break: break-all;
  word-wrap: break-word;
  background-color: #f5f5f5;
  border: 1px solid #ccc;
  border-radius: 4px;
}
pre code {
  padding: 0;
  font-size: inherit;
  color: inherit;
  white-space: pre-wrap;
  background-color: transparent;
  border-radius: 0;
}
.pre-scrollable {
  max-height: 340px;
  overflow-y: scroll;
}
.container {
  padding-right: 15px;
  padding-left: 15px;
  margin-right: auto;
  margin-left: auto;
}
@media (min-width: 768px) {
  .container {
    width: 750px;
  }
}
@media (min-width: 992px) {
  .container {
    width: 970px;
  }
}
@media (min-width: 1200px) {
  .container {
    width: 1170px;
  }
}
.container-fluid {
  padding-right: 15px;
  padding-left: 15px;
  margin-right: auto;
  margin-left: auto;
}
.row {
  margin-right: -15px;
  margin-left: -15px;
}
.col-xs-1, .col-sm-1, .col-md-1, .col-lg-1, .col-xs-2, .col-sm-2, .col-md-2, .col-lg-2, .col-xs-3, .col-sm-3, .col-md-3, .col-lg-3, .col-xs-4, .col-sm-4, .col-md-4, .col-lg-4, .col-xs-5, .col-sm-5, .col-md-5, .col-lg-5, .col-xs-6, .col-sm-6, .col-md-6, .col-lg-6, .col-xs-7, .col-sm-7, .col-md-7, .col-lg-7, .col-xs-8, .col-sm-8, .col-md-8, .col-lg-8, .col-xs-9, .col-sm-9, .col-md-9, .col-lg-9, .col-xs-10, .col-sm-10, .col-md-10, .col-lg-10, .col-xs-11, .col-sm-11, .col-md-11, .col-lg-11, .col-xs-12, .col-sm-12, .col-md-12, .col-lg-12 {
  position: relative;
  min-height: 1px;
  padding-right: 15px;
  padding-left: 15px;
}
.col-xs-1, .col-xs-2, .col-xs-3, .col-xs-4, .col-xs-5, .col-xs-6, .col-xs-7, .col-xs-8, .col-xs-9, .col-xs-10, .col-xs-11, .col-xs-12 {
  float: left;
}
.col-xs-12 {
  width: 100%;
}
.col-xs-11 {
  width: 91.66666667%;
}
.col-xs-10 {
  width: 83.33333333%;
}
.col-xs-9 {
  width: 75%;
}
.col-xs-8 {
  width: 66.66666667%;
}
.col-xs-7 {
  width: 58.33333333%;
}
.col-xs-6 {
  width: 50%;
}
.col-xs-5 {
  width: 41.66666667%;
}
.col-xs-4 {
  width: 33.33333333%;
}
.col-xs-3 {
  width: 25%;
}
.col-xs-2 {
  width: 16.66666667%;
}
.col-xs-1 {
  width: 8.33333333%;
}
.col-xs-pull-12 {
  right: 100%;
}
.col-xs-pull-11 {
  right: 91.66666667%;
}
.col-xs-pull-10 {
  right: 83.33333333%;
}
.col-xs-pull-9 {
  right: 75%;
}
.col-xs-pull-8 {
  right: 66.66666667%;
}
.col-xs-pull-7 {
  right: 58.33333333%;
}
.col-xs-pull-6 {
  right: 50%;
}
.col-xs-pull-5 {
  right: 41.66666667%;
}
.col-xs-pull-4 {
  right: 33.33333333%;
}
.col-xs-pull-3 {
  right: 25%;
}
.col-xs-pull-2 {
  right: 16.66666667%;
}
.col-xs-pull-1 {
  right: 8.33333333%;
}
.col-xs-pull-0 {
  right: auto;
}
.col-xs-push-12 {
  left: 100%;
}
.col-xs-push-11 {
  left: 91.66666667%;
}
.col-xs-push-10 {
  left: 83.33333333%;
}
.col-xs-push-9 {
  left: 75%;
}
.col-xs-push-8 {
  left: 66.66666667%;
}
.col-xs-push-7 {
  left: 58.33333333%;
}
.col-xs-push-6 {
  left: 50%;
}
.col-xs-push-5 {
  left: 41.66666667%;
}
.col-xs-push-4 {
  left: 33.33333333%;
}
.col-xs-push-3 {
  left: 25%;
}
.col-xs-push-2 {
  left: 16.66666667%;
}
.col-xs-push-1 {
  left: 8.33333333%;
}
.col-xs-push-0 {
  left: auto;
}
.col-xs-offset-12 {
  margin-left: 100%;
}
.col-xs-offset-11 {
  margin-left: 91.66666667%;
}
.col-xs-offset-10 {
  margin-left: 83.33333333%;
}
.col-xs-offset-9 {
  margin-left: 75%;
}
.col-xs-offset-8 {
  margin-left: 66.66666667%;
}
.col-xs-offset-7 {
  margin-left: 58.33333333%;
}
.col-xs-offset-6 {
  margin-left: 50%;
}
.col-xs-offset-5 {
  margin-left: 41.66666667%;
}
.col-xs-offset-4 {
  margin-left: 33.33333333%;
}
.col-xs-offset-3 {
  margin-left: 25%;
}
.col-xs-offset-2 {
  margin-left: 16.66666667%;
}
.col-xs-offset-1 {
  margin-left: 8.33333333%;
}
.col-xs-offset-0 {
  margin-left: 0;
}
@media (min-width: 768px) {
  .col-sm-1, .col-sm-2, .col-sm-3, .col-sm-4, .col-sm-5, .col-sm-6, .col-sm-7, .col-sm-8, .col-sm-9, .col-sm-10, .col-sm-11, .col-sm-12 {
    float: left;
  }
  .col-sm-12 {
    width: 100%;
  }
  .col-sm-11 {
    width: 91.66666667%;
  }
  .col-sm-10 {
    width: 83.33333333%;
  }
  .col-sm-9 {
    width: 75%;
  }
  .col-sm-8 {
    width: 66.66666667%;
  }
  .col-sm-7 {
    width: 58.33333333%;
  }
  .col-sm-6 {
    width: 50%;
  }
  .col-sm-5 {
    width: 41.66666667%;
  }
  .col-sm-4 {
    width: 33.33333333%;
  }
  .col-sm-3 {
    width: 25%;
  }
  .col-sm-2 {
    width: 16.66666667%;
  }
  .col-sm-1 {
    width: 8.33333333%;
  }
  .col-sm-pull-12 {
    right: 100%;
  }
  .col-sm-pull-11 {
    right: 91.66666667%;
  }
  .col-sm-pull-10 {
    right: 83.33333333%;
  }
  .col-sm-pull-9 {
    right: 75%;
  }
  .col-sm-pull-8 {
    right: 66.66666667%;
  }
  .col-sm-pull-7 {
    right: 58.33333333%;
  }
  .col-sm-pull-6 {
    right: 50%;
  }
  .col-sm-pull-5 {
    right: 41.66666667%;
  }
  .col-sm-pull-4 {
    right: 33.33333333%;
  }
  .col-sm-pull-3 {
    right: 25%;
  }
  .col-sm-pull-2 {
    right: 16.66666667%;
  }
  .col-sm-pull-1 {
    right: 8.33333333%;
  }
  .col-sm-pull-0 {
    right: auto;
  }
  .col-sm-push-12 {
    left: 100%;
  }
  .col-sm-push-11 {
    left: 91.66666667%;
  }
  .col-sm-push-10 {
    left: 83.33333333%;
  }
  .col-sm-push-9 {
    left: 75%;
  }
  .col-sm-push-8 {
    left: 66.66666667%;
  }
  .col-sm-push-7 {
    left: 58.33333333%;
  }
  .col-sm-push-6 {
    left: 50%;
  }
  .col-sm-push-5 {
    left: 41.66666667%;
  }
  .col-sm-push-4 {
    left: 33.33333333%;
  }
  .col-sm-push-3 {
    left: 25%;
  }
  .col-sm-push-2 {
    left: 16.66666667%;
  }
  .col-sm-push-1 {
    left: 8.33333333%;
  }
  .col-sm-push-0 {
    left: auto;
  }
  .col-sm-offset-12 {
    margin-left: 100%;
  }
  .col-sm-offset-11 {
    margin-left: 91.66666667%;
  }
  .col-sm-offset-10 {
    margin-left: 83.33333333%;
  }
  .col-sm-offset-9 {
    margin-left: 75%;
  }
  .col-sm-offset-8 {
    margin-left: 66.66666667%;
  }
  .col-sm-offset-7 {
    margin-left: 58.33333333%;
  }
  .col-sm-offset-6 {
    margin-left: 50%;
  }
  .col-sm-offset-5 {
    margin-left: 41.66666667%;
  }
  .col-sm-offset-4 {
    margin-left: 33.33333333%;
  }
  .col-sm-offset-3 {
    margin-left: 25%;
  }
  .col-sm-offset-2 {
    margin-left: 16.66666667%;
  }
  .col-sm-offset-1 {
    margin-left: 8.33333333%;
  }
  .col-sm-offset-0 {
    margin-left: 0;
  }
}
@media (min-width: 992px) {
  .col-md-1, .col-md-2, .col-md-3, .col-md-4, .col-md-5, .col-md-6, .col-md-7, .col-md-8, .col-md-9, .col-md-10, .col-md-11, .col-md-12 {
    float: left;
  }
  .col-md-12 {
    width: 100%;
  }
  .col-md-11 {
    width: 91.66666667%;
  }
  .col-md-10 {
    width: 83.33333333%;
  }
  .col-md-9 {
    width: 75%;
  }
  .col-md-8 {
    width: 66.66666667%;
  }
  .col-md-7 {
    width: 58.33333333%;
  }
  .col-md-6 {
    width: 50%;
  }
  .col-md-5 {
    width: 41.66666667%;
  }
  .col-md-4 {
    width: 33.33333333%;
  }
  .col-md-3 {
    width: 25%;
  }
  .col-md-2 {
    width: 16.66666667%;
  }
  .col-md-1 {
    width: 8.33333333%;
  }
  .col-md-pull-12 {
    right: 100%;
  }
  .col-md-pull-11 {
    right: 91.66666667%;
  }
  .col-md-pull-10 {
    right: 83.33333333%;
  }
  .col-md-pull-9 {
    right: 75%;
  }
  .col-md-pull-8 {
    right: 66.66666667%;
  }
  .col-md-pull-7 {
    right: 58.33333333%;
  }
  .col-md-pull-6 {
    right: 50%;
  }
  .col-md-pull-5 {
    right: 41.66666667%;
  }
  .col-md-pull-4 {
    right: 33.33333333%;
  }
  .col-md-pull-3 {
    right: 25%;
  }
  .col-md-pull-2 {
    right: 16.66666667%;
  }
  .col-md-pull-1 {
    right: 8.33333333%;
  }
  .col-md-pull-0 {
    right: auto;
  }
  .col-md-push-12 {
    left: 100%;
  }
  .col-md-push-11 {
    left: 91.66666667%;
  }
  .col-md-push-10 {
    left: 83.33333333%;
  }
  .col-md-push-9 {
    left: 75%;
  }
  .col-md-push-8 {
    left: 66.66666667%;
  }
  .col-md-push-7 {
    left: 58.33333333%;
  }
  .col-md-push-6 {
    left: 50%;
  }
  .col-md-push-5 {
    left: 41.66666667%;
  }
  .col-md-push-4 {
    left: 33.33333333%;
  }
  .col-md-push-3 {
    left: 25%;
  }
  .col-md-push-2 {
    left: 16.66666667%;
  }
  .col-md-push-1 {
    left: 8.33333333%;
  }
  .col-md-push-0 {
    left: auto;
  }
  .col-md-offset-12 {
    margin-left: 100%;
  }
  .col-md-offset-11 {
    margin-left: 91.66666667%;
  }
  .col-md-offset-10 {
    margin-left: 83.33333333%;
  }
  .col-md-offset-9 {
    margin-left: 75%;
  }
  .col-md-offset-8 {
    margin-left: 66.66666667%;
  }
  .col-md-offset-7 {
    margin-left: 58.33333333%;
  }
  .col-md-offset-6 {
    margin-left: 50%;
  }
  .col-md-offset-5 {
    margin-left: 41.66666667%;
  }
  .col-md-offset-4 {
    margin-left: 33.33333333%;
  }
  .col-md-offset-3 {
    margin-left: 25%;
  }
  .col-md-offset-2 {
    margin-left: 16.66666667%;
  }
  .col-md-offset-1 {
    margin-left: 8.33333333%;
  }
  .col-md-offset-0 {
    margin-left: 0;
  }
}
@media (min-width: 1200px) {
  .col-lg-1, .col-lg-2, .col-lg-3, .col-lg-4, .col-lg-5, .col-lg-6, .col-lg-7, .col-lg-8, .col-lg-9, .col-lg-10, .col-lg-11, .col-lg-12 {
    float: left;
  }
  .col-lg-12 {
    width: 100%;
  }
  .col-lg-11 {
    width: 91.66666667%;
  }
  .col-lg-10 {
    width: 83.33333333%;
  }
  .col-lg-9 {
    width: 75%;
  }
  .col-lg-8 {
    width: 66.66666667%;
  }
  .col-lg-7 {
    width: 58.33333333%;
  }
  .col-lg-6 {
    width: 50%;
  }
  .col-lg-5 {
    width: 41.66666667%;
  }
  .col-lg-4 {
    width: 33.33333333%;
  }
  .col-lg-3 {
    width: 25%;
  }
  .col-lg-2 {
    width: 16.66666667%;
  }
  .col-lg-1 {
    width: 8.33333333%;
  }
  .col-lg-pull-12 {
    right: 100%;
  }
  .col-lg-pull-11 {
    right: 91.66666667%;
  }
  .col-lg-pull-10 {
    right: 83.33333333%;
  }
  .col-lg-pull-9 {
    right: 75%;
  }
  .col-lg-pull-8 {
    right: 66.66666667%;
  }
  .col-lg-pull-7 {
    right: 58.33333333%;
  }
  .col-lg-pull-6 {
    right: 50%;
  }
  .col-lg-pull-5 {
    right: 41.66666667%;
  }
  .col-lg-pull-4 {
    right: 33.33333333%;
  }
  .col-lg-pull-3 {
    right: 25%;
  }
  .col-lg-pull-2 {
    right: 16.66666667%;
  }
  .col-lg-pull-1 {
    right: 8.33333333%;
  }
  .col-lg-pull-0 {
    right: auto;
  }
  .col-lg-push-12 {
    left: 100%;
  }
  .col-lg-push-11 {
    left: 91.66666667%;
  }
  .col-lg-push-10 {
    left: 83.33333333%;
  }
  .col-lg-push-9 {
    left: 75%;
  }
  .col-lg-push-8 {
    left: 66.66666667%;
  }
  .col-lg-push-7 {
    left: 58.33333333%;
  }
  .col-lg-push-6 {
    left: 50%;
  }
  .col-lg-push-5 {
    left: 41.66666667%;
  }
  .col-lg-push-4 {
    left: 33.33333333%;
  }
  .col-lg-push-3 {
    left: 25%;
  }
  .col-lg-push-2 {
    left: 16.66666667%;
  }
  .col-lg-push-1 {
    left: 8.33333333%;
  }
  .col-lg-push-0 {
    left: auto;
  }
  .col-lg-offset-12 {
    margin-left: 100%;
  }
  .col-lg-offset-11 {
    margin-left: 91.66666667%;
  }
  .col-lg-offset-10 {
    margin-left: 83.33333333%;
  }
  .col-lg-offset-9 {
    margin-left: 75%;
  }
  .col-lg-offset-8 {
    margin-left: 66.66666667%;
  }
  .col-lg-offset-7 {
    margin-left: 58.33333333%;
  }
  .col-lg-offset-6 {
    margin-left: 50%;
  }
  .col-lg-offset-5 {
    margin-left: 41.66666667%;
  }
  .col-lg-offset-4 {
    margin-left: 33.33333333%;
  }
  .col-lg-offset-3 {
    margin-left: 25%;
  }
  .col-lg-offset-2 {
    margin-left: 16.66666667%;
  }
  .col-lg-offset-1 {
    margin-left: 8.33333333%;
  }
  .col-lg-offset-0 {
    margin-left: 0;
  }
}
table {
  background-color: transparent;
}
caption {
  padding-top: 8px;
  padding-bottom: 8px;
  color: #777;
  text-align: left;
}
th {
  text-align: left;
}
.table {
  width: 100%;
  max-width: 100%;
  margin-bottom: 20px;
}
.table > thead > tr > th,
.table > tbody > tr > th,
.table > tfoot > tr > th,
.table > thead > tr > td,
.table > tbody > tr > td,
.table > tfoot > tr > td {
  padding: 8px;
  line-height: 1.42857143;
  vertical-align: top;
  border-top: 1px solid #ddd;
}
.table > thead > tr > th {
  vertical-align: bottom;
  border-bottom: 2px solid #ddd;
}
.table > caption + thead > tr:first-child > th,
.table > colgroup + thead > tr:first-child > th,
.table > thead:first-child > tr:first-child > th,
.table > caption + thead > tr:first-child > td,
.table > colgroup + thead > tr:first-child > td,
.table > thead:first-child > tr:first-child > td {
  border-top: 0;
}
.table > tbody + tbody {
  border-top: 2px solid #ddd;
}
.table .table {
  background-color: #fff;
}
.table-condensed > thead > tr > th,
.table-condensed > tbody > tr > th,
.table-condensed > tfoot > tr > th,
.table-condensed > thead > tr > td,
.table-condensed > tbody > tr > td,
.table-condensed > tfoot > tr > td {
  padding: 5px;
}
.table-bordered {
  border: 1px solid #ddd;
}
.table-bordered > thead > tr > th,
.table-bordered > tbody > tr > th,
.table-bordered > tfoot > tr > th,
.table-bordered > thead > tr > td,
.table-bordered > tbody > tr > td,
.table-bordered > tfoot > tr > td {
  border: 1px solid #ddd;
}
.table-bordered > thead > tr > th,
.table-bordered > thead > tr > td {
  border-bottom-width: 2px;
}
.table-striped > tbody > tr:nth-of-type(odd) {
  background-color: #f9f9f9;
}
.table-hover > tbody > tr:hover {
  background-color: #f5f5f5;
}
table col[class*="col-"] {
  position: static;
  display: table-column;
  float: none;
}
table td[class*="col-"],
table th[class*="col-"] {
  position: static;
  display: table-cell;
  float: none;
}
.table > thead > tr > td.active,
.table > tbody > tr > td.active,
.table > tfoot > tr > td.active,
.table > thead > tr > th.active,
.table > tbody > tr > th.active,
.table > tfoot > tr > th.active,
.table > thead > tr.active > td,
.table > tbody > tr.active > td,
.table > tfoot > tr.active > td,
.table > thead > tr.active > th,
.table > tbody > tr.active > th,
.table > tfoot > tr.active > th {
  background-color: #f5f5f5;
}
.table-hover > tbody > tr > td.active:hover,
.table-hover > tbody > tr > th.active:hover,
.table-hover > tbody > tr.active:hover > td,
.table-hover > tbody > tr:hover > .active,
.table-hover > tbody > tr.active:hover > th {
  background-color: #e8e8e8;
}
.table > thead > tr > td.success,
.table > tbody > tr > td.success,
.table > tfoot > tr > td.success,
.table > thead > tr > th.success,
.table > tbody > tr > th.success,
.table > tfoot > tr > th.success,
.table > thead > tr.success > td,
.table > tbody > tr.success > td,
.table > tfoot > tr.success > td,
.table > thead > tr.success > th,
.table > tbody > tr.success > th,
.table > tfoot > tr.success > th {
  background-color: #dff0d8;
}
.table-hover > tbody > tr > td.success:hover,
.table-hover > tbody > tr > th.success:hover,
.table-hover > tbody > tr.success:hover > td,
.table-hover > tbody > tr:hover > .success,
.table-hover > tbody > tr.success:hover > th {
  background-color: #d0e9c6;
}
.table > thead > tr > td.info,
.table > tbody > tr > td.info,
.table > tfoot > tr > td.info,
.table > thead > tr > th.info,
.table > tbody > tr > th.info,
.table > tfoot > tr > th.info,
.table > thead > tr.info > td,
.table > tbody > tr.info > td,
.table > tfoot > tr.info > td,
.table > thead > tr.info > th,
.table > tbody > tr.info > th,
.table > tfoot > tr.info > th {
  background-color: #d9edf7;
}
.table-hover > tbody > tr > td.info:hover,
.table-hover > tbody > tr > th.info:hover,
.table-hover > tbody > tr.info:hover > td,
.table-hover > tbody > tr:hover > .info,
.table-hover > tbody > tr.info:hover > th {
  background-color: #c4e3f3;
}
.table > thead > tr > td.warning,
.table > tbody > tr > td.warning,
.table > tfoot > tr > td.warning,
.table > thead > tr > th.warning,
.table > tbody > tr > th.warning,
.table > tfoot > tr > th.warning,
.table > thead > tr.warning > td,
.table > tbody > tr.warning > td,
.table > tfoot > tr.warning > td,
.table > thead > tr.warning > th,
.table > tbody > tr.warning > th,
.table > tfoot > tr.warning > th {
  background-color: #fcf8e3;
}
.table-hover > tbody > tr > td.warning:hover,
.table-hover > tbody > tr > th.warning:hover,
.table-hover > tbody > tr.warning:hover > td,
.table-hover > tbody > tr:hover > .warning,
.table-hover > tbody > tr.warning:hover > th {
  background-color: #faf2cc;
}
.table > thead > tr > td.danger,
.table > tbody > tr > td.danger,
.table > tfoot > tr > td.danger,
.table > thead > tr > th.danger,
.table > tbody > tr > th.danger,
.table > tfoot > tr > th.danger,
.table > thead > tr.danger > td,
.table > tbody > tr.danger > td,
.table > tfoot > tr.danger > td,
.table > thead > tr.danger > th,
.table > tbody > tr.danger > th,
.table > tfoot > tr.danger > th {
  background-color: #f2dede;
}
.table-hover > tbody > tr > td.danger:hover,
.table-hover > tbody > tr > th.danger:hover,
.table-hover > tbody > tr.danger:hover > td,
.table-hover > tbody > tr:hover > .danger,
.table-hover > tbody > tr.danger:hover > th {
  background-color: #ebcccc;
}
.table-responsive {
  min-height: .01%;
  overflow-x: auto;
}
@media screen and (max-width: 767px) {
  .table-responsive {
    width: 100%;
    margin-bottom: 15px;
    overflow-y: hidden;
    -ms-overflow-style: -ms-autohiding-scrollbar;
    border: 1px solid #ddd;
  }
  .table-responsive > .table {
    margin-bottom: 0;
  }
  .table-responsive > .table > thead > tr > th,
  .table-responsive > .table > tbody > tr > th,
  .table-responsive > .table > tfoot > tr > th,
  .table-responsive > .table > thead > tr > td,
  .table-responsive > .table > tbody > tr > td,
  .table-responsive > .table > tfoot > tr > td {
    white-space: nowrap;
  }
  .table-responsive > .table-bordered {
    border: 0;
  }
  .table-responsive > .table-bordered > thead > tr > th:first-child,
  .table-responsive > .table-bordered > tbody > tr > th:first-child,
  .table-responsive > .table-bordered > tfoot > tr > th:first-child,
  .table-responsive > .table-bordered > thead > tr > td:first-child,
  .table-responsive > .table-bordered > tbody > tr > td:first-child,
  .table-responsive > .table-bordered > tfoot > tr > td:first-child {
    border-left: 0;
  }
  .table-responsive > .table-bordered > thead > tr > th:last-child,
  .table-responsive > .table-bordered > tbody > tr > th:last-child,
  .table-responsive > .table-bordered > tfoot > tr > th:last-child,
  .table-responsive > .table-bordered > thead > tr > td:last-child,
  .table-responsive > .table-bordered > tbody > tr > td:last-child,
  .table-responsive > .table-bordered > tfoot > tr > td:last-child {
    border-right: 0;
  }
  .table-responsive > .table-bordered > tbody > tr:last-child > th,
  .table-responsive > .table-bordered > tfoot > tr:last-child > th,
  .table-responsive > .table-bordered > tbody > tr:last-child > td,
  .table-responsive > .table-bordered > tfoot > tr:last-child > td {
    border-bottom: 0;
  }
}
fieldset {
  min-width: 0;
  padding: 0;
  margin: 0;
  border: 0;
}
legend {
  display: block;
  width: 100%;
  padding: 0;
  margin-bottom: 20px;
  font-size: 21px;
  line-height: inherit;
  color: #333;
  border: 0;
  border-bottom: 1px solid #e5e5e5;
}
label {
  display: inline-block;
  max-width: 100%;
  margin-bottom: 5px;
  font-weight: bold;
}
input[type="search"] {
  -webkit-box-sizing: border-box;
     -moz-box-sizing: border-box;
          box-sizing: border-box;
}
input[type="radio"],
input[type="checkbox"] {
  margin: 4px 0 0;
  margin-top: 1px \9;
  line-height: normal;
}
input[type="file"] {
  display: block;
}
input[type="range"] {
  display: block;
  width: 100%;
}
select[multiple],
select[size] {
  height: auto;
}
input[type="file"]:focus,
input[type="radio"]:focus,
input[type="checkbox"]:focus {
  outline: thin dotted;
  outline: 5px auto -webkit-focus-ring-color;
  outline-offset: -2px;
}
output {
  display: block;
  padding-top: 7px;
  font-size: 14px;
  line-height: 1.42857143;
  color: #555;
}
.form-control {
  display: block;
  width: 100%;
  height: 34px;
  padding: 6px 12px;
  font-size: 14px;
  line-height: 1.42857143;
  color: #555;
  background-color: #fff;
  background-image: none;
  border: 1px solid #ccc;
  border-radius: 4px;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
  -webkit-transition: border-color ease-in-out .15s, -webkit-box-shadow ease-in-out .15s;
       -o-transition: border-color ease-in-out .15s, box-shadow ease-in-out .15s;
          transition: border-color ease-in-out .15s, box-shadow ease-in-out .15s;
}
.form-control:focus {
  border-color: #66afe9;
  outline: 0;
  -webkit-box-shadow: inset 0 1px 1px rgba(0,0,0,.075), 0 0 8px rgba(102, 175, 233, .6);
          box-shadow: inset 0 1px 1px rgba(0,0,0,.075), 0 0 8px rgba(102, 175, 233, .6);
}
.form-control::-moz-placeholder {
  color: #999;
  opacity: 1;
}
.form-control:-ms-input-placeholder {
  color: #999;
}
.form-control::-webkit-input-placeholder {
  color: #999;
}
.form-control[disabled],
.form-control[readonly],
fieldset[disabled] .form-control {
  background-color: #eee;
  opacity: 1;
}
.form-control[disabled],
fieldset[disabled] .form-control {
  cursor: not-allowed;
}
textarea.form-control {
  height: auto;
}
input[type="search"] {
  -webkit-appearance: none;
}
@media screen and (-webkit-min-device-pixel-ratio: 0) {
  input[type="date"],
  input[type="time"],
  input[type="datetime-local"],
  input[type="month"] {
    line-height: 34px;
  }
  input[type="date"].input-sm,
  input[type="time"].input-sm,
  input[type="datetime-local"].input-sm,
  input[type="month"].input-sm,
  .input-group-sm input[type="date"],
  .input-group-sm input[type="time"],
  .input-group-sm input[type="datetime-local"],
  .input-group-sm input[type="month"] {
    line-height: 30px;
  }
  input[type="date"].input-lg,
  input[type="time"].input-lg,
  input[type="datetime-local"].input-lg,
  input[type="month"].input-lg,
  .input-group-lg input[type="date"],
  .input-group-lg input[type="time"],
  .input-group-lg input[type="datetime-local"],
  .input-group-lg input[type="month"] {
    line-height: 46px;
  }
}
.form-group {
  margin-bottom: 15px;
}
.radio,
.checkbox {
  position: relative;
  display: block;
  margin-top: 10px;
  margin-bottom: 10px;
}
.radio label,
.checkbox label {
  min-height: 20px;
  padding-left: 20px;
  margin-bottom: 0;
  font-weight: normal;
  cursor: pointer;
}
.radio input[type="radio"],
.radio-inline input[type="radio"],
.checkbox input[type="checkbox"],
.checkbox-inline input[type="checkbox"] {
  position: absolute;
  margin-top: 4px \9;
  margin-left: -20px;
}
.radio + .radio,
.checkbox + .checkbox {
  margin-top: -5px;
}
.radio-inline,
.checkbox-inline {
  position: relative;
  display: inline-block;
  padding-left: 20px;
  margin-bottom: 0;
  font-weight: normal;
  vertical-align: middle;
  cursor: pointer;
}
.radio-inline + .radio-inline,
.checkbox-inline + .checkbox-inline {
  margin-top: 0;
  margin-left: 10px;
}
input[type="radio"][disabled],
input[type="checkbox"][disabled],
input[type="radio"].disabled,
input[type="checkbox"].disabled,
fieldset[disabled] input[type="radio"],
fieldset[disabled] input[type="checkbox"] {
  cursor: not-allowed;
}
.radio-inline.disabled,
.checkbox-inline.disabled,
fieldset[disabled] .radio-inline,
fieldset[disabled] .checkbox-inline {
  cursor: not-allowed;
}
.radio.disabled label,
.checkbox.disabled label,
fieldset[disabled] .radio label,
fieldset[disabled] .checkbox label {
  cursor: not-allowed;
}
.form-control-static {
  min-height: 34px;
  padding-top: 7px;
  padding-bottom: 7px;
  margin-bottom: 0;
}
.form-control-static.input-lg,
.form-control-static.input-sm {
  padding-right: 0;
  padding-left: 0;
}
.input-sm {
  height: 30px;
  padding: 5px 10px;
  font-size: 12px;
  line-height: 1.5;
  border-radius: 3px;
}
select.input-sm {
  height: 30px;
  line-height: 30px;
}
textarea.input-sm,
select[multiple].input-sm {
  height: auto;
}
.form-group-sm .form-control {
  height: 30px;
  padding: 5px 10px;
  font-size: 12px;
  line-height: 1.5;
  border-radius: 3px;
}
select.form-group-sm .form-control {
  height: 30px;
  line-height: 30px;
}
textarea.form-group-sm .form-control,
select[multiple].form-group-sm .form-control {
  height: auto;
}
.form-group-sm .form-control-static {
  height: 30px;
  min-height: 32px;
  padding: 5px 10px;
  font-size: 12px;
  line-height: 1.5;
}
.input-lg {
  height: 46px;
  padding: 10px 16px;
  font-size: 18px;
  line-height: 1.3333333;
  border-radius: 6px;
}
select.input-lg {
  height: 46px;
  line-height: 46px;
}
textarea.input-lg,
select[multiple].input-lg {
  height: auto;
}
.form-group-lg .form-control {
  height: 46px;
  padding: 10px 16px;
  font-size: 18px;
  line-height: 1.3333333;
  border-radius: 6px;
}
select.form-group-lg .form-control {
  height: 46px;
  line-height: 46px;
}
textarea.form-group-lg .form-control,
select[multiple].form-group-lg .form-control {
  height: auto;
}
.form-group-lg .form-control-static {
  height: 46px;
  min-height: 38px;
  padding: 10px 16px;
  font-size: 18px;
  line-height: 1.3333333;
}
.has-feedback {
  position: relative;
}
.has-feedback .form-control {
  padding-right: 42.5px;
}
.form-control-feedback {
  position: absolute;
  top: 0;
  right: 0;
  z-index: 2;
  display: block;
  width: 34px;
  height: 34px;
  line-height: 34px;
  text-align: center;
  pointer-events: none;
}
.input-lg + .form-control-feedback {
  width: 46px;
  height: 46px;
  line-height: 46px;
}
.input-sm + .form-control-feedback {
  width: 30px;
  height: 30px;
  line-height: 30px;
}
.has-success .help-block,
.has-success .control-label,
.has-success .radio,
.has-success .checkbox,
.has-success .radio-inline,
.has-success .checkbox-inline,
.has-success.radio label,
.has-success.checkbox label,
.has-success.radio-inline label,
.has-success.checkbox-inline label {
  color: #3c763d;
}
.has-success .form-control {
  border-color: #3c763d;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
}
.has-success .form-control:focus {
  border-color: #2b542c;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #67b168;
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #67b168;
}
.has-success .input-group-addon {
  color: #3c763d;
  background-color: #dff0d8;
  border-color: #3c763d;
}
.has-success .form-control-feedback {
  color: #3c763d;
}
.has-warning .help-block,
.has-warning .control-label,
.has-warning .radio,
.has-warning .checkbox,
.has-warning .radio-inline,
.has-warning .checkbox-inline,
.has-warning.radio label,
.has-warning.checkbox label,
.has-warning.radio-inline label,
.has-warning.checkbox-inline label {
  color: #8a6d3b;
}
.has-warning .form-control {
  border-color: #8a6d3b;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
}
.has-warning .form-control:focus {
  border-color: #66512c;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #c0a16b;
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #c0a16b;
}
.has-warning .input-group-addon {
  color: #8a6d3b;
  background-color: #fcf8e3;
  border-color: #8a6d3b;
}
.has-warning .form-control-feedback {
  color: #8a6d3b;
}
.has-error .help-block,
.has-error .control-label,
.has-error .radio,
.has-error .checkbox,
.has-error .radio-inline,
.has-error .checkbox-inline,
.has-error.radio label,
.has-error.checkbox label,
.has-error.radio-inline label,
.has-error.checkbox-inline label {
  color: #a94442;
}
.has-error .form-control {
  border-color: #a94442;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075);
}
.has-error .form-control:focus {
  border-color: #843534;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #ce8483;
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .075), 0 0 6px #ce8483;
}
.has-error .input-group-addon {
  color: #a94442;
  background-color: #f2dede;
  border-color: #a94442;
}
.has-error .form-control-feedback {
  color: #a94442;
}
.has-feedback label ~ .form-control-feedback {
  top: 25px;
}
.has-feedback label.sr-only ~ .form-control-feedback {
  top: 0;
}
.help-block {
  display: block;
  margin-top: 5px;
  margin-bottom: 10px;
  color: #737373;
}
@media (min-width: 768px) {
  .form-inline .form-group {
    display: inline-block;
    margin-bottom: 0;
    vertical-align: middle;
  }
  .form-inline .form-control {
    display: inline-block;
    width: auto;
    vertical-align: middle;
  }
  .form-inline .form-control-static {
    display: inline-block;
  }
  .form-inline .input-group {
    display: inline-table;
    vertical-align: middle;
  }
  .form-inline .input-group .input-group-addon,
  .form-inline .input-group .input-group-btn,
  .form-inline .input-group .form-control {
    width: auto;
  }
  .form-inline .input-group > .form-control {
    width: 100%;
  }
  .form-inline .control-label {
    margin-bottom: 0;
    vertical-align: middle;
  }
  .form-inline .radio,
  .form-inline .checkbox {
    display: inline-block;
    margin-top: 0;
    margin-bottom: 0;
    vertical-align: middle;
  }
  .form-inline .radio label,
  .form-inline .checkbox label {
    padding-left: 0;
  }
  .form-inline .radio input[type="radio"],
  .form-inline .checkbox input[type="checkbox"] {
    position: relative;
    margin-left: 0;
  }
  .form-inline .has-feedback .form-control-feedback {
    top: 0;
  }
}
.form-horizontal .radio,
.form-horizontal .checkbox,
.form-horizontal .radio-inline,
.form-horizontal .checkbox-inline {
  padding-top: 7px;
  margin-top: 0;
  margin-bottom: 0;
}
.form-horizontal .radio,
.form-horizontal .checkbox {
  min-height: 27px;
}
.form-horizontal .form-group {
  margin-right: -15px;
  margin-left: -15px;
}
@media (min-width: 768px) {
  .form-horizontal .control-label {
    padding-top: 7px;
    margin-bottom: 0;
    text-align: right;
  }
}
.form-horizontal .has-feedback .form-control-feedback {
  right: 15px;
}
@media (min-width: 768px) {
  .form-horizontal .form-group-lg .control-label {
    padding-top: 14.333333px;
  }
}
@media (min-width: 768px) {
  .form-horizontal .form-group-sm .control-label {
    padding-top: 6px;
  }
}
.btn {
  display: inline-block;
  padding: 6px 12px;
  margin-bottom: 0;
  font-size: 14px;
  font-weight: normal;
  line-height: 1.42857143;
  text-align: center;
  white-space: nowrap;
  vertical-align: middle;
  -ms-touch-action: manipulation;
      touch-action: manipulation;
  cursor: pointer;
  -webkit-user-select: none;
     -moz-user-select: none;
      -ms-user-select: none;
          user-select: none;
  background-image: none;
  border: 1px solid transparent;
  border-radius: 4px;
}
.btn:focus,
.btn:active:focus,
.btn.active:focus,
.btn.focus,
.btn:active.focus,
.btn.active.focus {
  outline: thin dotted;
  outline: 5px auto -webkit-focus-ring-color;
  outline-offset: -2px;
}
.btn:hover,
.btn:focus,
.btn.focus {
  color: #333;
  text-decoration: none;
}
.btn:active,
.btn.active {
  background-image: none;
  outline: 0;
  -webkit-box-shadow: inset 0 3px 5px rgba(0, 0, 0, .125);
          box-shadow: inset 0 3px 5px rgba(0, 0, 0, .125);
}
.btn.disabled,
.btn[disabled],
fieldset[disabled] .btn {
  pointer-events: none;
  cursor: not-allowed;
  filter: alpha(opacity=65);
  -webkit-box-shadow: none;
          box-shadow: none;
  opacity: .65;
}
.btn-default {
  color: #333;
  background-color: #fff;
  border-color: #ccc;
}
.btn-default:hover,
.btn-default:focus,
.btn-default.focus,
.btn-default:active,
.btn-default.active,
.open > .dropdown-toggle.btn-default {
  color: #333;
  background-color: #e6e6e6;
  border-color: #adadad;
}
.btn-default:active,
.btn-default.active,
.open > .dropdown-toggle.btn-default {
  background-image: none;
}
.btn-default.disabled,
.btn-default[disabled],
fieldset[disabled] .btn-default,
.btn-default.disabled:hover,
.btn-default[disabled]:hover,
fieldset[disabled] .btn-default:hover,
.btn-default.disabled:focus,
.btn-default[disabled]:focus,
fieldset[disabled] .btn-default:focus,
.btn-default.disabled.focus,
.btn-default[disabled].focus,
fieldset[disabled] .btn-default.focus,
.btn-default.disabled:active,
.btn-default[disabled]:active,
fieldset[disabled] .btn-default:active,
.btn-default.disabled.active,
.btn-default[disabled].active,
fieldset[disabled] .btn-default.active {
  background-color: #fff;
  border-color: #ccc;
}
.btn-default .badge {
  color: #fff;
  background-color: #333;
}
.btn-primary {
  color: #fff;
  background-color: #337ab7;
  border-color: #2e6da4;
}
.btn-primary:hover,
.btn-primary:focus,
.btn-primary.focus,
.btn-primary:active,
.btn-primary.active,
.open > .dropdown-toggle.btn-primary {
  color: #fff;
  background-color: #286090;
  border-color: #204d74;
}
.btn-primary:active,
.btn-primary.active,
.open > .dropdown-toggle.btn-primary {
  background-image: none;
}
.btn-primary.disabled,
.btn-primary[disabled],
fieldset[disabled] .btn-primary,
.btn-primary.disabled:hover,
.btn-primary[disabled]:hover,
fieldset[disabled] .btn-primary:hover,
.btn-primary.disabled:focus,
.btn-primary[disabled]:focus,
fieldset[disabled] .btn-primary:focus,
.btn-primary.disabled.focus,
.btn-primary[disabled].focus,
fieldset[disabled] .btn-primary.focus,
.btn-primary.disabled:active,
.btn-primary[disabled]:active,
fieldset[disabled] .btn-primary:active,
.btn-primary.disabled.active,
.btn-primary[disabled].active,
fieldset[disabled] .btn-primary.active {
  background-color: #337ab7;
  border-color: #2e6da4;
}
.btn-primary .badge {
  color: #337ab7;
  background-color: #fff;
}
.btn-success {
  color: #fff;
  background-color: #5cb85c;
  border-color: #4cae4c;
}
.btn-success:hover,
.btn-success:focus,
.btn-success.focus,
.btn-success:active,
.btn-success.active,
.open > .dropdown-toggle.btn-success {
  color: #fff;
  background-color: #449d44;
  border-color: #398439;
}
.btn-success:active,
.btn-success.active,
.open > .dropdown-toggle.btn-success {
  background-image: none;
}
.btn-success.disabled,
.btn-success[disabled],
fieldset[disabled] .btn-success,
.btn-success.disabled:hover,
.btn-success[disabled]:hover,
fieldset[disabled] .btn-success:hover,
.btn-success.disabled:focus,
.btn-success[disabled]:focus,
fieldset[disabled] .btn-success:focus,
.btn-success.disabled.focus,
.btn-success[disabled].focus,
fieldset[disabled] .btn-success.focus,
.btn-success.disabled:active,
.btn-success[disabled]:active,
fieldset[disabled] .btn-success:active,
.btn-success.disabled.active,
.btn-success[disabled].active,
fieldset[disabled] .btn-success.active {
  background-color: #5cb85c;
  border-color: #4cae4c;
}
.btn-success .badge {
  color: #5cb85c;
  background-color: #fff;
}
.btn-info {
  color: #fff;
  background-color: #5bc0de;
  border-color: #46b8da;
}
.btn-info:hover,
.btn-info:focus,
.btn-info.focus,
.btn-info:active,
.btn-info.active,
.open > .dropdown-toggle.btn-info {
  color: #fff;
  background-color: #31b0d5;
  border-color: #269abc;
}
.btn-info:active,
.btn-info.active,
.open > .dropdown-toggle.btn-info {
  background-image: none;
}
.btn-info.disabled,
.btn-info[disabled],
fieldset[disabled] .btn-info,
.btn-info.disabled:hover,
.btn-info[disabled]:hover,
fieldset[disabled] .btn-info:hover,
.btn-info.disabled:focus,
.btn-info[disabled]:focus,
fieldset[disabled] .btn-info:focus,
.btn-info.disabled.focus,
.btn-info[disabled].focus,
fieldset[disabled] .btn-info.focus,
.btn-info.disabled:active,
.btn-info[disabled]:active,
fieldset[disabled] .btn-info:active,
.btn-info.disabled.active,
.btn-info[disabled].active,
fieldset[disabled] .btn-info.active {
  background-color: #5bc0de;
  border-color: #46b8da;
}
.btn-info .badge {
  color: #5bc0de;
  background-color: #fff;
}
.btn-warning {
  color: #fff;
  background-color: #f0ad4e;
  border-color: #eea236;
}
.btn-warning:hover,
.btn-warning:focus,
.btn-warning.focus,
.btn-warning:active,
.btn-warning.active,
.open > .dropdown-toggle.btn-warning {
  color: #fff;
  background-color: #ec971f;
  border-color: #d58512;
}
.btn-warning:active,
.btn-warning.active,
.open > .dropdown-toggle.btn-warning {
  background-image: none;
}
.btn-warning.disabled,
.btn-warning[disabled],
fieldset[disabled] .btn-warning,
.btn-warning.disabled:hover,
.btn-warning[disabled]:hover,
fieldset[disabled] .btn-warning:hover,
.btn-warning.disabled:focus,
.btn-warning[disabled]:focus,
fieldset[disabled] .btn-warning:focus,
.btn-warning.disabled.focus,
.btn-warning[disabled].focus,
fieldset[disabled] .btn-warning.focus,
.btn-warning.disabled:active,
.btn-warning[disabled]:active,
fieldset[disabled] .btn-warning:active,
.btn-warning.disabled.active,
.btn-warning[disabled].active,
fieldset[disabled] .btn-warning.active {
  background-color: #f0ad4e;
  border-color: #eea236;
}
.btn-warning .badge {
  color: #f0ad4e;
  background-color: #fff;
}
.btn-danger {
  color: #fff;
  background-color: #d9534f;
  border-color: #d43f3a;
}
.btn-danger:hover,
.btn-danger:focus,
.btn-danger.focus,
.btn-danger:active,
.btn-danger.active,
.open > .dropdown-toggle.btn-danger {
  color: #fff;
  background-color: #c9302c;
  border-color: #ac2925;
}
.btn-danger:active,
.btn-danger.active,
.open > .dropdown-toggle.btn-danger {
  background-image: none;
}
.btn-danger.disabled,
.btn-danger[disabled],
fieldset[disabled] .btn-danger,
.btn-danger.disabled:hover,
.btn-danger[disabled]:hover,
fieldset[disabled] .btn-danger:hover,
.btn-danger.disabled:focus,
.btn-danger[disabled]:focus,
fieldset[disabled] .btn-danger:focus,
.btn-danger.disabled.focus,
.btn-danger[disabled].focus,
fieldset[disabled] .btn-danger.focus,
.btn-danger.disabled:active,
.btn-danger[disabled]:active,
fieldset[disabled] .btn-danger:active,
.btn-danger.disabled.active,
.btn-danger[disabled].active,
fieldset[disabled] .btn-danger.active {
  background-color: #d9534f;
  border-color: #d43f3a;
}
.btn-danger .badge {
  color: #d9534f;
  background-color: #fff;
}
.btn-link {
  font-weight: normal;
  color: #337ab7;
  border-radius: 0;
}
.btn-link,
.btn-link:active,
.btn-link.active,
.btn-link[disabled],
fieldset[disabled] .btn-link {
  background-color: transparent;
  -webkit-box-shadow: none;
          box-shadow: none;
}
.btn-link,
.btn-link:hover,
.btn-link:focus,
.btn-link:active {
  border-color: transparent;
}
.btn-link:hover,
.btn-link:focus {
  color: #23527c;
  text-decoration: underline;
  background-color: transparent;
}
.btn-link[disabled]:hover,
fieldset[disabled] .btn-link:hover,
.btn-link[disabled]:focus,
fieldset[disabled] .btn-link:focus {
  color: #777;
  text-decoration: none;
}
.btn-lg,
.btn-group-lg > .btn {
  padding: 10px 16px;
  font-size: 18px;
  line-height: 1.3333333;
  border-radius: 6px;
}
.btn-sm,
.btn-group-sm > .btn {
  padding: 5px 10px;
  font-size: 12px;
  line-height: 1.5;
  border-radius: 3px;
}
.btn-xs,
.btn-group-xs > .btn {
  padding: 1px 5px;
  font-size: 12px;
  line-height: 1.5;
  border-radius: 3px;
}
.btn-block {
  display: block;
  width: 100%;
}
.btn-block + .btn-block {
  margin-top: 5px;
}
input[type="submit"].btn-block,
input[type="reset"].btn-block,
input[type="button"].btn-block {
  width: 100%;
}
.fade {
  opacity: 0;
  -webkit-transition: opacity .15s linear;
       -o-transition: opacity .15s linear;
          transition: opacity .15s linear;
}
.fade.in {
  opacity: 1;
}
.collapse {
  display: none;
}
.collapse.in {
  display: block;
}
tr.collapse.in {
  display: table-row;
}
tbody.collapse.in {
  display: table-row-group;
}
.collapsing {
  position: relative;
  height: 0;
  overflow: hidden;
  -webkit-transition-timing-function: ease;
       -o-transition-timing-function: ease;
          transition-timing-function: ease;
  -webkit-transition-duration: .35s;
       -o-transition-duration: .35s;
          transition-duration: .35s;
  -webkit-transition-property: height, visibility;
       -o-transition-property: height, visibility;
          transition-property: height, visibility;
}
.caret {
  display: inline-block;
  width: 0;
  height: 0;
  margin-left: 2px;
  vertical-align: middle;
  border-top: 4px dashed;
  border-right: 4px solid transparent;
  border-left: 4px solid transparent;
}
.dropup,
.dropdown {
  position: relative;
}
.dropdown-toggle:focus {
  outline: 0;
}
.dropdown-menu {
  position: absolute;
  top: 100%;
  left: 0;
  z-index: 1000;
  display: none;
  float: left;
  min-width: 160px;
  padding: 5px 0;
  margin: 2px 0 0;
  font-size: 14px;
  text-align: left;
  list-style: none;
  background-color: #fff;
  -webkit-background-clip: padding-box;
          background-clip: padding-box;
  border: 1px solid #ccc;
  border: 1px solid rgba(0, 0, 0, .15);
  border-radius: 4px;
  -webkit-box-shadow: 0 6px 12px rgba(0, 0, 0, .175);
          box-shadow: 0 6px 12px rgba(0, 0, 0, .175);
}
.dropdown-menu.pull-right {
  right: 0;
  left: auto;
}
.dropdown-menu .divider {
  height: 1px;
  margin: 9px 0;
  overflow: hidden;
  background-color: #e5e5e5;
}
.dropdown-menu > li > a {
  display: block;
  padding: 3px 20px;
  clear: both;
  font-weight: normal;
  line-height: 1.42857143;
  color: #333;
  white-space: nowrap;
}
.dropdown-menu > li > a:hover,
.dropdown-menu > li > a:focus {
  color: #262626;
  text-decoration: none;
  background-color: #f5f5f5;
}
.dropdown-menu > .active > a,
.dropdown-menu > .active > a:hover,
.dropdown-menu > .active > a:focus {
  color: #fff;
  text-decoration: none;
  background-color: #337ab7;
  outline: 0;
}
.dropdown-menu > .disabled > a,
.dropdown-menu > .disabled > a:hover,
.dropdown-menu > .disabled > a:focus {
  color: #777;
}
.dropdown-menu > .disabled > a:hover,
.dropdown-menu > .disabled > a:focus {
  text-decoration: none;
  cursor: not-allowed;
  background-color: transparent;
  background-image: none;
  filter: progid:DXImageTransform.Microsoft.gradient(enabled = false);
}
.open > .dropdown-menu {
  display: block;
}
.open > a {
  outline: 0;
}
.dropdown-menu-right {
  right: 0;
  left: auto;
}
.dropdown-menu-left {
  right: auto;
  left: 0;
}
.dropdown-header {
  display: block;
  padding: 3px 20px;
  font-size: 12px;
  line-height: 1.42857143;
  color: #777;
  white-space: nowrap;
}
.dropdown-backdrop {
  position: fixed;
  top: 0;
  right: 0;
  bottom: 0;
  left: 0;
  z-index: 990;
}
.pull-right > .dropdown-menu {
  right: 0;
  left: auto;
}
.dropup .caret,
.navbar-fixed-bottom .dropdown .caret {
  content: "";
  border-top: 0;
  border-bottom: 4px solid;
}
.dropup .dropdown-menu,
.navbar-fixed-bottom .dropdown .dropdown-menu {
  top: auto;
  bottom: 100%;
  margin-bottom: 2px;
}
@media (min-width: 768px) {
  .navbar-right .dropdown-menu {
    right: 0;
    left: auto;
  }
  .navbar-right .dropdown-menu-left {
    right: auto;
    left: 0;
  }
}
.btn-group,
.btn-group-vertical {
  position: relative;
  display: inline-block;
  vertical-align: middle;
}
.btn-group > .btn,
.btn-group-vertical > .btn {
  position: relative;
  float: left;
}
.btn-group > .btn:hover,
.btn-group-vertical > .btn:hover,
.btn-group > .btn:focus,
.btn-group-vertical > .btn:focus,
.btn-group > .btn:active,
.btn-group-vertical > .btn:active,
.btn-group > .btn.active,
.btn-group-vertical > .btn.active {
  z-index: 2;
}
.btn-group .btn + .btn,
.btn-group .btn + .btn-group,
.btn-group .btn-group + .btn,
.btn-group .btn-group + .btn-group {
  margin-left: -1px;
}
.btn-toolbar {
  margin-left: -5px;
}
.btn-toolbar .btn-group,
.btn-toolbar .input-group {
  float: left;
}
.btn-toolbar > .btn,
.btn-toolbar > .btn-group,
.btn-toolbar > .input-group {
  margin-left: 5px;
}
.btn-group > .btn:not(:first-child):not(:last-child):not(.dropdown-toggle) {
  border-radius: 0;
}
.btn-group > .btn:first-child {
  margin-left: 0;
}
.btn-group > .btn:first-child:not(:last-child):not(.dropdown-toggle) {
  border-top-right-radius: 0;
  border-bottom-right-radius: 0;
}
.btn-group > .btn:last-child:not(:first-child),
.btn-group > .dropdown-toggle:not(:first-child) {
  border-top-left-radius: 0;
  border-bottom-left-radius: 0;
}
.btn-group > .btn-group {
  float: left;
}
.btn-group > .btn-group:not(:first-child):not(:last-child) > .btn {
  border-radius: 0;
}
.btn-group > .btn-group:first-child:not(:last-child) > .btn:last-child,
.btn-group > .btn-group:first-child:not(:last-child) > .dropdown-toggle {
  border-top-right-radius: 0;
  border-bottom-right-radius: 0;
}
.btn-group > .btn-group:last-child:not(:first-child) > .btn:first-child {
  border-top-left-radius: 0;
  border-bottom-left-radius: 0;
}
.btn-group .dropdown-toggle:active,
.btn-group.open .dropdown-toggle {
  outline: 0;
}
.btn-group > .btn + .dropdown-toggle {
  padding-right: 8px;
  padding-left: 8px;
}
.btn-group > .btn-lg + .dropdown-toggle {
  padding-right: 12px;
  padding-left: 12px;
}
.btn-group.open .dropdown-toggle {
  -webkit-box-shadow: inset 0 3px 5px rgba(0, 0, 0, .125);
          box-shadow: inset 0 3px 5px rgba(0, 0, 0, .125);
}
.btn-group.open .dropdown-toggle.btn-link {
  -webkit-box-shadow: none;
          box-shadow: none;
}
.btn .caret {
  margin-left: 0;
}
.btn-lg .caret {
  border-width: 5px 5px 0;
  border-bottom-width: 0;
}
.dropup .btn-lg .caret {
  border-width: 0 5px 5px;
}
.btn-group-vertical > .btn,
.btn-group-vertical > .btn-group,
.btn-group-vertical > .btn-group > .btn {
  display: block;
  float: none;
  width: 100%;
  max-width: 100%;
}
.btn-group-vertical > .btn-group > .btn {
  float: none;
}
.btn-group-vertical > .btn + .btn,
.btn-group-vertical > .btn + .btn-group,
.btn-group-vertical > .btn-group + .btn,
.btn-group-vertical > .btn-group + .btn-group {
  margin-top: -1px;
  margin-left: 0;
}
.btn-group-vertical > .btn:not(:first-child):not(:last-child) {
  border-radius: 0;
}
.btn-group-vertical > .btn:first-child:not(:last-child) {
  border-top-right-radius: 4px;
  border-bottom-right-radius: 0;
  border-bottom-left-radius: 0;
}
.btn-group-vertical > .btn:last-child:not(:first-child) {
  border-top-left-radius: 0;
  border-top-right-radius: 0;
  border-bottom-left-radius: 4px;
}
.btn-group-vertical > .btn-group:not(:first-child):not(:last-child) > .btn {
  border-radius: 0;
}
.btn-group-vertical > .btn-group:first-child:not(:last-child) > .btn:last-child,
.btn-group-vertical > .btn-group:first-child:not(:last-child) > .dropdown-toggle {
  border-bottom-right-radius: 0;
  border-bottom-left-radius: 0;
}
.btn-group-vertical > .btn-group:last-child:not(:first-child) > .btn:first-child {
  border-top-left-radius: 0;
  border-top-right-radius: 0;
}
.btn-group-justified {
  display: table;
  width: 100%;
  table-layout: fixed;
  border-collapse: separate;
}
.btn-group-justified > .btn,
.btn-group-justified > .btn-group {
  display: table-cell;
  float: none;
  width: 1%;
}
.btn-group-justified > .btn-group .btn {
  width: 100%;
}
.btn-group-justified > .btn-group .dropdown-menu {
  left: auto;
}
[data-toggle="buttons"] > .btn input[type="radio"],
[data-toggle="buttons"] > .btn-group > .btn input[type="radio"],
[data-toggle="buttons"] > .btn input[type="checkbox"],
[data-toggle="buttons"] > .btn-group > .btn input[type="checkbox"] {
  position: absolute;
  clip: rect(0, 0, 0, 0);
  pointer-events: none;
}
.input-group {
  position: relative;
  display: table;
  border-collapse: separate;
}
.input-group[class*="col-"] {
  float: none;
  padding-right: 0;
  padding-left: 0;
}
.input-group .form-control {
  position: relative;
  z-index: 2;
  float: left;
  width: 100%;
  margin-bottom: 0;
}
.input-group-lg > .form-control,
.input-group-lg > .input-group-addon,
.input-group-lg > .input-group-btn > .btn {
  height: 46px;
  padding: 10px 16px;
  font-size: 18px;
  line-height: 1.3333333;
  border-radius: 6px;
}
select.input-group-lg > .form-control,
select.input-group-lg > .input-group-addon,
select.input-group-lg > .input-group-btn > .btn {
  height: 46px;
  line-height: 46px;
}
textarea.input-group-lg > .form-control,
textarea.input-group-lg > .input-group-addon,
textarea.input-group-lg > .input-group-btn > .btn,
select[multiple].input-group-lg > .form-control,
select[multiple].input-group-lg > .input-group-addon,
select[multiple].input-group-lg > .input-group-btn > .btn {
  height: auto;
}
.input-group-sm > .form-control,
.input-group-sm > .input-group-addon,
.input-group-sm > .input-group-btn > .btn {
  height: 30px;
  padding: 5px 10px;
  font-size: 12px;
  line-height: 1.5;
  border-radius: 3px;
}
select.input-group-sm > .form-control,
select.input-group-sm > .input-group-addon,
select.input-group-sm > .input-group-btn > .btn {
  height: 30px;
  line-height: 30px;
}
textarea.input-group-sm > .form-control,
textarea.input-group-sm > .input-group-addon,
textarea.input-group-sm > .input-group-btn > .btn,
select[multiple].input-group-sm > .form-control,
select[multiple].input-group-sm > .input-group-addon,
select[multiple].input-group-sm > .input-group-btn > .btn {
  height: auto;
}
.input-group-addon,
.input-group-btn,
.input-group .form-control {
  display: table-cell;
}
.input-group-addon:not(:first-child):not(:last-child),
.input-group-btn:not(:first-child):not(:last-child),
.input-group .form-control:not(:first-child):not(:last-child) {
  border-radius: 0;
}
.input-group-addon,
.input-group-btn {
  width: 1%;
  white-space: nowrap;
  vertical-align: middle;
}
.input-group-addon {
  padding: 6px 12px;
  font-size: 14px;
  font-weight: normal;
  line-height: 1;
  color: #555;
  text-align: center;
  background-color: #eee;
  border: 1px solid #ccc;
  border-radius: 4px;
}
.input-group-addon.input-sm {
  padding: 5px 10px;
  font-size: 12px;
  border-radius: 3px;
}
.input-group-addon.input-lg {
  padding: 10px 16px;
  font-size: 18px;
  border-radius: 6px;
}
.input-group-addon input[type="radio"],
.input-group-addon input[type="checkbox"] {
  margin-top: 0;
}
.input-group .form-control:first-child,
.input-group-addon:first-child,
.input-group-btn:first-child > .btn,
.input-group-btn:first-child > .btn-group > .btn,
.input-group-btn:first-child > .dropdown-toggle,
.input-group-btn:last-child > .btn:not(:last-child):not(.dropdown-toggle),
.input-group-btn:last-child > .btn-group:not(:last-child) > .btn {
  border-top-right-radius: 0;
  border-bottom-right-radius: 0;
}
.input-group-addon:first-child {
  border-right: 0;
}
.input-group .form-control:last-child,
.input-group-addon:last-child,
.input-group-btn:last-child > .btn,
.input-group-btn:last-child > .btn-group > .btn,
.input-group-btn:last-child > .dropdown-toggle,
.input-group-btn:first-child > .btn:not(:first-child),
.input-group-btn:first-child > .btn-group:not(:first-child) > .btn {
  border-top-left-radius: 0;
  border-bottom-left-radius: 0;
}
.input-group-addon:last-child {
  border-left: 0;
}
.input-group-btn {
  position: relative;
  font-size: 0;
  white-space: nowrap;
}
.input-group-btn > .btn {
  position: relative;
}
.input-group-btn > .btn + .btn {
  margin-left: -1px;
}
.input-group-btn > .btn:hover,
.input-group-btn > .btn:focus,
.input-group-btn > .btn:active {
  z-index: 2;
}
.input-group-btn:first-child > .btn,
.input-group-btn:first-child > .btn-group {
  margin-right: -1px;
}
.input-group-btn:last-child > .btn,
.input-group-btn:last-child > .btn-group {
  margin-left: -1px;
}
.nav {
  padding-left: 0;
  margin-bottom: 0;
  list-style: none;
}
.nav > li {
  position: relative;
  display: block;
}
.nav > li > a {
  position: relative;
  display: block;
  padding: 10px 15px;
}
.nav > li > a:hover,
.nav > li > a:focus {
  text-decoration: none;
  background-color: #eee;
}
.nav > li.disabled > a {
  color: #777;
}
.nav > li.disabled > a:hover,
.nav > li.disabled > a:focus {
  color: #777;
  text-decoration: none;
  cursor: not-allowed;
  background-color: transparent;
}
.nav .open > a,
.nav .open > a:hover,
.nav .open > a:focus {
  background-color: #eee;
  border-color: #337ab7;
}
.nav .nav-divider {
  height: 1px;
  margin: 9px 0;
  overflow: hidden;
  background-color: #e5e5e5;
}
.nav > li > a > img {
  max-width: none;
}
.nav-tabs {
  border-bottom: 1px solid #ddd;
}
.nav-tabs > li {
  float: left;
  margin-bottom: -1px;
}
.nav-tabs > li > a {
  margin-right: 2px;
  line-height: 1.42857143;
  border: 1px solid transparent;
  border-radius: 4px 4px 0 0;
}
.nav-tabs > li > a:hover {
  border-color: #eee #eee #ddd;
}
.nav-tabs > li.active > a,
.nav-tabs > li.active > a:hover,
.nav-tabs > li.active > a:focus {
  color: #555;
  cursor: default;
  background-color: #fff;
  border: 1px solid #ddd;
  border-bottom-color: transparent;
}
.nav-tabs.nav-justified {
  width: 100%;
  border-bottom: 0;
}
.nav-tabs.nav-justified > li {
  float: none;
}
.nav-tabs.nav-justified > li > a {
  margin-bottom: 5px;
  text-align: center;
}
.nav-tabs.nav-justified > .dropdown .dropdown-menu {
  top: auto;
  left: auto;
}
@media (min-width: 768px) {
  .nav-tabs.nav-justified > li {
    display: table-cell;
    width: 1%;
  }
  .nav-tabs.nav-justified > li > a {
    margin-bottom: 0;
  }
}
.nav-tabs.nav-justified > li > a {
  margin-right: 0;
  border-radius: 4px;
}
.nav-tabs.nav-justified > .active > a,
.nav-tabs.nav-justified > .active > a:hover,
.nav-tabs.nav-justified > .active > a:focus {
  border: 1px solid #ddd;
}
@media (min-width: 768px) {
  .nav-tabs.nav-justified > li > a {
    border-bottom: 1px solid #ddd;
    border-radius: 4px 4px 0 0;
  }
  .nav-tabs.nav-justified > .active > a,
  .nav-tabs.nav-justified > .active > a:hover,
  .nav-tabs.nav-justified > .active > a:focus {
    border-bottom-color: #fff;
  }
}
.nav-pills > li {
  float: left;
}
.nav-pills > li > a {
  border-radius: 4px;
}
.nav-pills > li + li {
  margin-left: 2px;
}
.nav-pills > li.active > a,
.nav-pills > li.active > a:hover,
.nav-pills > li.active > a:focus {
  color: #fff;
  background-color: #337ab7;
}
.nav-stacked > li {
  float: none;
}
.nav-stacked > li + li {
  margin-top: 2px;
  margin-left: 0;
}
.nav-justified {
  width: 100%;
}
.nav-justified > li {
  float: none;
}
.nav-justified > li > a {
  margin-bottom: 5px;
  text-align: center;
}
.nav-justified > .dropdown .dropdown-menu {
  top: auto;
  left: auto;
}
@media (min-width: 768px) {
  .nav-justified > li {
    display: table-cell;
    width: 1%;
  }
  .nav-justified > li > a {
    margin-bottom: 0;
  }
}
.nav-tabs-justified {
  border-bottom: 0;
}
.nav-tabs-justified > li > a {
  margin-right: 0;
  border-radius: 4px;
}
.nav-tabs-justified > .active > a,
.nav-tabs-justified > .active > a:hover,
.nav-tabs-justified > .active > a:focus {
  border: 1px solid #ddd;
}
@media (min-width: 768px) {
  .nav-tabs-justified > li > a {
    border-bottom: 1px solid #ddd;
    border-radius: 4px 4px 0 0;
  }
  .nav-tabs-justified > .active > a,
  .nav-tabs-justified > .active > a:hover,
  .nav-tabs-justified > .active > a:focus {
    border-bottom-color: #fff;
  }
}
.tab-content > .tab-pane {
  display: none;
}
.tab-content > .active {
  display: block;
}
.nav-tabs .dropdown-menu {
  margin-top: -1px;
  border-top-left-radius: 0;
  border-top-right-radius: 0;
}
.navbar {
  position: relative;
  min-height: 50px;
  margin-bottom: 20px;
  border: 1px solid transparent;
}
@media (min-width: 768px) {
  .navbar {
    border-radius: 4px;
  }
}
@media (min-width: 768px) {
  .navbar-header {
    float: left;
  }
}
.navbar-collapse {
  padding-right: 15px;
  padding-left: 15px;
  overflow-x: visible;
  -webkit-overflow-scrolling: touch;
  border-top: 1px solid transparent;
  -webkit-box-shadow: inset 0 1px 0 rgba(255, 255, 255, .1);
          box-shadow: inset 0 1px 0 rgba(255, 255, 255, .1);
}
.navbar-collapse.in {
  overflow-y: auto;
}
@media (min-width: 768px) {
  .navbar-collapse {
    width: auto;
    border-top: 0;
    -webkit-box-shadow: none;
            box-shadow: none;
  }
  .navbar-collapse.collapse {
    display: block !important;
    height: auto !important;
    padding-bottom: 0;
    overflow: visible !important;
  }
  .navbar-collapse.in {
    overflow-y: visible;
  }
  .navbar-fixed-top .navbar-collapse,
  .navbar-static-top .navbar-collapse,
  .navbar-fixed-bottom .navbar-collapse {
    padding-right: 0;
    padding-left: 0;
  }
}
.navbar-fixed-top .navbar-collapse,
.navbar-fixed-bottom .navbar-collapse {
  max-height: 340px;
}
@media (max-device-width: 480px) and (orientation: landscape) {
  .navbar-fixed-top .navbar-collapse,
  .navbar-fixed-bottom .navbar-collapse {
    max-height: 200px;
  }
}
.container > .navbar-header,
.container-fluid > .navbar-header,
.container > .navbar-collapse,
.container-fluid > .navbar-collapse {
  margin-right: -15px;
  margin-left: -15px;
}
@media (min-width: 768px) {
  .container > .navbar-header,
  .container-fluid > .navbar-header,
  .container > .navbar-collapse,
  .container-fluid > .navbar-collapse {
    margin-right: 0;
    margin-left: 0;
  }
}
.navbar-static-top {
  z-index: 1000;
  border-width: 0 0 1px;
}
@media (min-width: 768px) {
  .navbar-static-top {
    border-radius: 0;
  }
}
.navbar-fixed-top,
.navbar-fixed-bottom {
  position: fixed;
  right: 0;
  left: 0;
  z-index: 1030;
}
@media (min-width: 768px) {
  .navbar-fixed-top,
  .navbar-fixed-bottom {
    border-radius: 0;
  }
}
.navbar-fixed-top {
  top: 0;
  border-width: 0 0 1px;
}
.navbar-fixed-bottom {
  bottom: 0;
  margin-bottom: 0;
  border-width: 1px 0 0;
}
.navbar-brand {
  float: left;
  height: 50px;
  padding: 15px 15px;
  font-size: 18px;
  line-height: 20px;
}
.navbar-brand:hover,
.navbar-brand:focus {
  text-decoration: none;
}
.navbar-brand > img {
  display: block;
}
@media (min-width: 768px) {
  .navbar > .container .navbar-brand,
  .navbar > .container-fluid .navbar-brand {
    margin-left: -15px;
  }
}
.navbar-toggle {
  position: relative;
  float: right;
  padding: 9px 10px;
  margin-top: 8px;
  margin-right: 15px;
  margin-bottom: 8px;
  background-color: transparent;
  background-image: none;
  border: 1px solid transparent;
  border-radius: 4px;
}
.navbar-toggle:focus {
  outline: 0;
}
.navbar-toggle .icon-bar {
  display: block;
  width: 22px;
  height: 2px;
  border-radius: 1px;
}
.navbar-toggle .icon-bar + .icon-bar {
  margin-top: 4px;
}
@media (min-width: 768px) {
  .navbar-toggle {
    display: none;
  }
}
.navbar-nav {
  margin: 7.5px -15px;
}
.navbar-nav > li > a {
  padding-top: 10px;
  padding-bottom: 10px;
  line-height: 20px;
}
@media (max-width: 767px) {
  .navbar-nav .open .dropdown-menu {
    position: static;
    float: none;
    width: auto;
    margin-top: 0;
    background-color: transparent;
    border: 0;
    -webkit-box-shadow: none;
            box-shadow: none;
  }
  .navbar-nav .open .dropdown-menu > li > a,
  .navbar-nav .open .dropdown-menu .dropdown-header {
    padding: 5px 15px 5px 25px;
  }
  .navbar-nav .open .dropdown-menu > li > a {
    line-height: 20px;
  }
  .navbar-nav .open .dropdown-menu > li > a:hover,
  .navbar-nav .open .dropdown-menu > li > a:focus {
    background-image: none;
  }
}
@media (min-width: 768px) {
  .navbar-nav {
    float: left;
    margin: 0;
  }
  .navbar-nav > li {
    float: left;
  }
  .navbar-nav > li > a {
    padding-top: 15px;
    padding-bottom: 15px;
  }
}
.navbar-form {
  padding: 10px 15px;
  margin-top: 8px;
  margin-right: -15px;
  margin-bottom: 8px;
  margin-left: -15px;
  border-top: 1px solid transparent;
  border-bottom: 1px solid transparent;
  -webkit-box-shadow: inset 0 1px 0 rgba(255, 255, 255, .1), 0 1px 0 rgba(255, 255, 255, .1);
          box-shadow: inset 0 1px 0 rgba(255, 255, 255, .1), 0 1px 0 rgba(255, 255, 255, .1);
}
@media (min-width: 768px) {
  .navbar-form .form-group {
    display: inline-block;
    margin-bottom: 0;
    vertical-align: middle;
  }
  .navbar-form .form-control {
    display: inline-block;
    width: auto;
    vertical-align: middle;
  }
  .navbar-form .form-control-static {
    display: inline-block;
  }
  .navbar-form .input-group {
    display: inline-table;
    vertical-align: middle;
  }
  .navbar-form .input-group .input-group-addon,
  .navbar-form .input-group .input-group-btn,
  .navbar-form .input-group .form-control {
    width: auto;
  }
  .navbar-form .input-group > .form-control {
    width: 100%;
  }
  .navbar-form .control-label {
    margin-bottom: 0;
    vertical-align: middle;
  }
  .navbar-form .radio,
  .navbar-form .checkbox {
    display: inline-block;
    margin-top: 0;
    margin-bottom: 0;
    vertical-align: middle;
  }
  .navbar-form .radio label,
  .navbar-form .checkbox label {
    padding-left: 0;
  }
  .navbar-form .radio input[type="radio"],
  .navbar-form .checkbox input[type="checkbox"] {
    position: relative;
    margin-left: 0;
  }
  .navbar-form .has-feedback .form-control-feedback {
    top: 0;
  }
}
@media (max-width: 767px) {
  .navbar-form .form-group {
    margin-bottom: 5px;
  }
  .navbar-form .form-group:last-child {
    margin-bottom: 0;
  }
}
@media (min-width: 768px) {
  .navbar-form {
    width: auto;
    padding-top: 0;
    padding-bottom: 0;
    margin-right: 0;
    margin-left: 0;
    border: 0;
    -webkit-box-shadow: none;
            box-shadow: none;
  }
}
.navbar-nav > li > .dropdown-menu {
  margin-top: 0;
  border-top-left-radius: 0;
  border-top-right-radius: 0;
}
.navbar-fixed-bottom .navbar-nav > li > .dropdown-menu {
  margin-bottom: 0;
  border-top-left-radius: 4px;
  border-top-right-radius: 4px;
  border-bottom-right-radius: 0;
  border-bottom-left-radius: 0;
}
.navbar-btn {
  margin-top: 8px;
  margin-bottom: 8px;
}
.navbar-btn.btn-sm {
  margin-top: 10px;
  margin-bottom: 10px;
}
.navbar-btn.btn-xs {
  margin-top: 14px;
  margin-bottom: 14px;
}
.navbar-text {
  margin-top: 15px;
  margin-bottom: 15px;
}
@media (min-width: 768px) {
  .navbar-text {
    float: left;
    margin-right: 15px;
    margin-left: 15px;
  }
}
@media (min-width: 768px) {
  .navbar-left {
    float: left !important;
  }
  .navbar-right {
    float: right !important;
    margin-right: -15px;
  }
  .navbar-right ~ .navbar-right {
    margin-right: 0;
  }
}
.navbar-default {
  background-color: #f8f8f8;
  border-color: #e7e7e7;
}
.navbar-default .navbar-brand {
  color: #777;
}
.navbar-default .navbar-brand:hover,
.navbar-default .navbar-brand:focus {
  color: #5e5e5e;
  background-color: transparent;
}
.navbar-default .navbar-text {
  color: #777;
}
.navbar-default .navbar-nav > li > a {
  color: #777;
}
.navbar-default .navbar-nav > li > a:hover,
.navbar-default .navbar-nav > li > a:focus {
  color: #333;
  background-color: transparent;
}
.navbar-default .navbar-nav > .active > a,
.navbar-default .navbar-nav > .active > a:hover,
.navbar-default .navbar-nav > .active > a:focus {
  color: #555;
  background-color: #e7e7e7;
}
.navbar-default .navbar-nav > .disabled > a,
.navbar-default .navbar-nav > .disabled > a:hover,
.navbar-default .navbar-nav > .disabled > a:focus {
  color: #ccc;
  background-color: transparent;
}
.navbar-default .navbar-toggle {
  border-color: #ddd;
}
.navbar-default .navbar-toggle:hover,
.navbar-default .navbar-toggle:focus {
  background-color: #ddd;
}
.navbar-default .navbar-toggle .icon-bar {
  background-color: #888;
}
.navbar-default .navbar-collapse,
.navbar-default .navbar-form {
  border-color: #e7e7e7;
}
.navbar-default .navbar-nav > .open > a,
.navbar-default .navbar-nav > .open > a:hover,
.navbar-default .navbar-nav > .open > a:focus {
  color: #555;
  background-color: #e7e7e7;
}
@media (max-width: 767px) {
  .navbar-default .navbar-nav .open .dropdown-menu > li > a {
    color: #777;
  }
  .navbar-default .navbar-nav .open .dropdown-menu > li > a:hover,
  .navbar-default .navbar-nav .open .dropdown-menu > li > a:focus {
    color: #333;
    background-color: transparent;
  }
  .navbar-default .navbar-nav .open .dropdown-menu > .active > a,
  .navbar-default .navbar-nav .open .dropdown-menu > .active > a:hover,
  .navbar-default .navbar-nav .open .dropdown-menu > .active > a:focus {
    color: #555;
    background-color: #e7e7e7;
  }
  .navbar-default .navbar-nav .open .dropdown-menu > .disabled > a,
  .navbar-default .navbar-nav .open .dropdown-menu > .disabled > a:hover,
  .navbar-default .navbar-nav .open .dropdown-menu > .disabled > a:focus {
    color: #ccc;
    background-color: transparent;
  }
}
.navbar-default .navbar-link {
  color: #777;
}
.navbar-default .navbar-link:hover {
  color: #333;
}
.navbar-default .btn-link {
  color: #777;
}
.navbar-default .btn-link:hover,
.navbar-default .btn-link:focus {
  color: #333;
}
.navbar-default .btn-link[disabled]:hover,
fieldset[disabled] .navbar-default .btn-link:hover,
.navbar-default .btn-link[disabled]:focus,
fieldset[disabled] .navbar-default .btn-link:focus {
  color: #ccc;
}
.navbar-inverse {
  background-color: #222;
  border-color: #080808;
}
.navbar-inverse .navbar-brand {
  color: #9d9d9d;
}
.navbar-inverse .navbar-brand:hover,
.navbar-inverse .navbar-brand:focus {
  color: #fff;
  background-color: transparent;
}
.navbar-inverse .navbar-text {
  color: #9d9d9d;
}
.navbar-inverse .navbar-nav > li > a {
  color: #9d9d9d;
}
.navbar-inverse .navbar-nav > li > a:hover,
.navbar-inverse .navbar-nav > li > a:focus {
  color: #fff;
  background-color: transparent;
}
.navbar-inverse .navbar-nav > .active > a,
.navbar-inverse .navbar-nav > .active > a:hover,
.navbar-inverse .navbar-nav > .active > a:focus {
  color: #fff;
  background-color: #080808;
}
.navbar-inverse .navbar-nav > .disabled > a,
.navbar-inverse .navbar-nav > .disabled > a:hover,
.navbar-inverse .navbar-nav > .disabled > a:focus {
  color: #444;
  background-color: transparent;
}
.navbar-inverse .navbar-toggle {
  border-color: #333;
}
.navbar-inverse .navbar-toggle:hover,
.navbar-inverse .navbar-toggle:focus {
  background-color: #333;
}
.navbar-inverse .navbar-toggle .icon-bar {
  background-color: #fff;
}
.navbar-inverse .navbar-collapse,
.navbar-inverse .navbar-form {
  border-color: #101010;
}
.navbar-inverse .navbar-nav > .open > a,
.navbar-inverse .navbar-nav > .open > a:hover,
.navbar-inverse .navbar-nav > .open > a:focus {
  color: #fff;
  background-color: #080808;
}
@media (max-width: 767px) {
  .navbar-inverse .navbar-nav .open .dropdown-menu > .dropdown-header {
    border-color: #080808;
  }
  .navbar-inverse .navbar-nav .open .dropdown-menu .divider {
    background-color: #080808;
  }
  .navbar-inverse .navbar-nav .open .dropdown-menu > li > a {
    color: #9d9d9d;
  }
  .navbar-inverse .navbar-nav .open .dropdown-menu > li > a:hover,
  .navbar-inverse .navbar-nav .open .dropdown-menu > li > a:focus {
    color: #fff;
    background-color: transparent;
  }
  .navbar-inverse .navbar-nav .open .dropdown-menu > .active > a,
  .navbar-inverse .navbar-nav .open .dropdown-menu > .active > a:hover,
  .navbar-inverse .navbar-nav .open .dropdown-menu > .active > a:focus {
    color: #fff;
    background-color: #080808;
  }
  .navbar-inverse .navbar-nav .open .dropdown-menu > .disabled > a,
  .navbar-inverse .navbar-nav .open .dropdown-menu > .disabled > a:hover,
  .navbar-inverse .navbar-nav .open .dropdown-menu > .disabled > a:focus {
    color: #444;
    background-color: transparent;
  }
}
.navbar-inverse .navbar-link {
  color: #9d9d9d;
}
.navbar-inverse .navbar-link:hover {
  color: #fff;
}
.navbar-inverse .btn-link {
  color: #9d9d9d;
}
.navbar-inverse .btn-link:hover,
.navbar-inverse .btn-link:focus {
  color: #fff;
}
.navbar-inverse .btn-link[disabled]:hover,
fieldset[disabled] .navbar-inverse .btn-link:hover,
.navbar-inverse .btn-link[disabled]:focus,
fieldset[disabled] .navbar-inverse .btn-link:focus {
  color: #444;
}
.breadcrumb {
  padding: 8px 15px;
  margin-bottom: 20px;
  list-style: none;
  background-color: #f5f5f5;
  border-radius: 4px;
}
.breadcrumb > li {
  display: inline-block;
}
.breadcrumb > li + li:before {
  padding: 0 5px;
  color: #ccc;
  content: "/\00a0";
}
.breadcrumb > .active {
  color: #777;
}
.pagination {
  display: inline-block;
  padding-left: 0;
  margin: 20px 0;
  border-radius: 4px;
}
.pagination > li {
  display: inline;
}
.pagination > li > a,
.pagination > li > span {
  position: relative;
  float: left;
  padding: 6px 12px;
  margin-left: -1px;
  line-height: 1.42857143;
  color: #337ab7;
  text-decoration: none;
  background-color: #fff;
  border: 1px solid #ddd;
}
.pagination > li:first-child > a,
.pagination > li:first-child > span {
  margin-left: 0;
  border-top-left-radius: 4px;
  border-bottom-left-radius: 4px;
}
.pagination > li:last-child > a,
.pagination > li:last-child > span {
  border-top-right-radius: 4px;
  border-bottom-right-radius: 4px;
}
.pagination > li > a:hover,
.pagination > li > span:hover,
.pagination > li > a:focus,
.pagination > li > span:focus {
  color: #23527c;
  background-color: #eee;
  border-color: #ddd;
}
.pagination > .active > a,
.pagination > .active > span,
.pagination > .active > a:hover,
.pagination > .active > span:hover,
.pagination > .active > a:focus,
.pagination > .active > span:focus {
  z-index: 2;
  color: #fff;
  cursor: default;
  background-color: #337ab7;
  border-color: #337ab7;
}
.pagination > .disabled > span,
.pagination > .disabled > span:hover,
.pagination > .disabled > span:focus,
.pagination > .disabled > a,
.pagination > .disabled > a:hover,
.pagination > .disabled > a:focus {
  color: #777;
  cursor: not-allowed;
  background-color: #fff;
  border-color: #ddd;
}
.pagination-lg > li > a,
.pagination-lg > li > span {
  padding: 10px 16px;
  font-size: 18px;
}
.pagination-lg > li:first-child > a,
.pagination-lg > li:first-child > span {
  border-top-left-radius: 6px;
  border-bottom-left-radius: 6px;
}
.pagination-lg > li:last-child > a,
.pagination-lg > li:last-child > span {
  border-top-right-radius: 6px;
  border-bottom-right-radius: 6px;
}
.pagination-sm > li > a,
.pagination-sm > li > span {
  padding: 5px 10px;
  font-size: 12px;
}
.pagination-sm > li:first-child > a,
.pagination-sm > li:first-child > span {
  border-top-left-radius: 3px;
  border-bottom-left-radius: 3px;
}
.pagination-sm > li:last-child > a,
.pagination-sm > li:last-child > span {
  border-top-right-radius: 3px;
  border-bottom-right-radius: 3px;
}
.pager {
  padding-left: 0;
  margin: 20px 0;
  text-align: center;
  list-style: none;
}
.pager li {
  display: inline;
}
.pager li > a,
.pager li > span {
  display: inline-block;
  padding: 5px 14px;
  background-color: #fff;
  border: 1px solid #ddd;
  border-radius: 15px;
}
.pager li > a:hover,
.pager li > a:focus {
  text-decoration: none;
  background-color: #eee;
}
.pager .next > a,
.pager .next > span {
  float: right;
}
.pager .previous > a,
.pager .previous > span {
  float: left;
}
.pager .disabled > a,
.pager .disabled > a:hover,
.pager .disabled > a:focus,
.pager .disabled > span {
  color: #777;
  cursor: not-allowed;
  background-color: #fff;
}
.label {
  display: inline;
  padding: .2em .6em .3em;
  font-size: 75%;
  font-weight: bold;
  line-height: 1;
  color: #fff;
  text-align: center;
  white-space: nowrap;
  vertical-align: baseline;
  border-radius: .25em;
}
a.label:hover,
a.label:focus {
  color: #fff;
  text-decoration: none;
  cursor: pointer;
}
.label:empty {
  display: none;
}
.btn .label {
  position: relative;
  top: -1px;
}
.label-default {
  background-color: #777;
}
.label-default[href]:hover,
.label-default[href]:focus {
  background-color: #5e5e5e;
}
.label-primary {
  background-color: #337ab7;
}
.label-primary[href]:hover,
.label-primary[href]:focus {
  background-color: #286090;
}
.label-success {
  background-color: #5cb85c;
}
.label-success[href]:hover,
.label-success[href]:focus {
  background-color: #449d44;
}
.label-info {
  background-color: #5bc0de;
}
.label-info[href]:hover,
.label-info[href]:focus {
  background-color: #31b0d5;
}
.label-warning {
  background-color: #f0ad4e;
}
.label-warning[href]:hover,
.label-warning[href]:focus {
  background-color: #ec971f;
}
.label-danger {
  background-color: #d9534f;
}
.label-danger[href]:hover,
.label-danger[href]:focus {
  background-color: #c9302c;
}
.badge {
  display: inline-block;
  min-width: 10px;
  padding: 3px 7px;
  font-size: 12px;
  font-weight: bold;
  line-height: 1;
  color: #fff;
  text-align: center;
  white-space: nowrap;
  vertical-align: baseline;
  background-color: #777;
  border-radius: 10px;
}
.badge:empty {
  display: none;
}
.btn .badge {
  position: relative;
  top: -1px;
}
.btn-xs .badge,
.btn-group-xs > .btn .badge {
  top: 0;
  padding: 1px 5px;
}
a.badge:hover,
a.badge:focus {
  color: #fff;
  text-decoration: none;
  cursor: pointer;
}
.list-group-item.active > .badge,
.nav-pills > .active > a > .badge {
  color: #337ab7;
  background-color: #fff;
}
.list-group-item > .badge {
  float: right;
}
.list-group-item > .badge + .badge {
  margin-right: 5px;
}
.nav-pills > li > a > .badge {
  margin-left: 3px;
}
.jumbotron {
  padding: 30px 15px;
  margin-bottom: 30px;
  color: inherit;
  background-color: #eee;
}
.jumbotron h1,
.jumbotron .h1 {
  color: inherit;
}
.jumbotron p {
  margin-bottom: 15px;
  font-size: 21px;
  font-weight: 200;
}
.jumbotron > hr {
  border-top-color: #d5d5d5;
}
.container .jumbotron,
.container-fluid .jumbotron {
  border-radius: 6px;
}
.jumbotron .container {
  max-width: 100%;
}
@media screen and (min-width: 768px) {
  .jumbotron {
    padding: 48px 0;
  }
  .container .jumbotron,
  .container-fluid .jumbotron {
    padding-right: 60px;
    padding-left: 60px;
  }
  .jumbotron h1,
  .jumbotron .h1 {
    font-size: 63px;
  }
}
.thumbnail {
  display: block;
  padding: 4px;
  margin-bottom: 20px;
  line-height: 1.42857143;
  background-color: #fff;
  border: 1px solid #ddd;
  border-radius: 4px;
  -webkit-transition: border .2s ease-in-out;
       -o-transition: border .2s ease-in-out;
          transition: border .2s ease-in-out;
}
.thumbnail > img,
.thumbnail a > img {
  margin-right: auto;
  margin-left: auto;
}
a.thumbnail:hover,
a.thumbnail:focus,
a.thumbnail.active {
  border-color: #337ab7;
}
.thumbnail .caption {
  padding: 9px;
  color: #333;
}
.alert {
  padding: 15px;
  margin-bottom: 20px;
  border: 1px solid transparent;
  border-radius: 4px;
}
.alert h4 {
  margin-top: 0;
  color: inherit;
}
.alert .alert-link {
  font-weight: bold;
}
.alert > p,
.alert > ul {
  margin-bottom: 0;
}
.alert > p + p {
  margin-top: 5px;
}
.alert-dismissable,
.alert-dismissible {
  padding-right: 35px;
}
.alert-dismissable .close,
.alert-dismissible .close {
  position: relative;
  top: -2px;
  right: -21px;
  color: inherit;
}
.alert-success {
  color: #3c763d;
  background-color: #dff0d8;
  border-color: #d6e9c6;
}
.alert-success hr {
  border-top-color: #c9e2b3;
}
.alert-success .alert-link {
  color: #2b542c;
}
.alert-info {
  color: #31708f;
  background-color: #d9edf7;
  border-color: #bce8f1;
}
.alert-info hr {
  border-top-color: #a6e1ec;
}
.alert-info .alert-link {
  color: #245269;
}
.alert-warning {
  color: #8a6d3b;
  background-color: #fcf8e3;
  border-color: #faebcc;
}
.alert-warning hr {
  border-top-color: #f7e1b5;
}
.alert-warning .alert-link {
  color: #66512c;
}
.alert-danger {
  color: #a94442;
  background-color: #f2dede;
  border-color: #ebccd1;
}
.alert-danger hr {
  border-top-color: #e4b9c0;
}
.alert-danger .alert-link {
  color: #843534;
}
@-webkit-keyframes progress-bar-stripes {
  from {
    background-position: 40px 0;
  }
  to {
    background-position: 0 0;
  }
}
@-o-keyframes progress-bar-stripes {
  from {
    background-position: 40px 0;
  }
  to {
    background-position: 0 0;
  }
}
@keyframes progress-bar-stripes {
  from {
    background-position: 40px 0;
  }
  to {
    background-position: 0 0;
  }
}
.progress {
  height: 20px;
  margin-bottom: 20px;
  overflow: hidden;
  background-color: #f5f5f5;
  border-radius: 4px;
  -webkit-box-shadow: inset 0 1px 2px rgba(0, 0, 0, .1);
          box-shadow: inset 0 1px 2px rgba(0, 0, 0, .1);
}
.progress-bar {
  float: left;
  width: 0;
  height: 100%;
  font-size: 12px;
  line-height: 20px;
  color: #fff;
  text-align: center;
  background-color: #337ab7;
  -webkit-box-shadow: inset 0 -1px 0 rgba(0, 0, 0, .15);
          box-shadow: inset 0 -1px 0 rgba(0, 0, 0, .15);
  -webkit-transition: width .6s ease;
       -o-transition: width .6s ease;
          transition: width .6s ease;
}
.progress-striped .progress-bar,
.progress-bar-striped {
  background-image: -webkit-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:      -o-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:         linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  -webkit-background-size: 40px 40px;
          background-size: 40px 40px;
}
.progress.active .progress-bar,
.progress-bar.active {
  -webkit-animation: progress-bar-stripes 2s linear infinite;
       -o-animation: progress-bar-stripes 2s linear infinite;
          animation: progress-bar-stripes 2s linear infinite;
}
.progress-bar-success {
  background-color: #5cb85c;
}
.progress-striped .progress-bar-success {
  background-image: -webkit-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:      -o-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:         linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
}
.progress-bar-info {
  background-color: #5bc0de;
}
.progress-striped .progress-bar-info {
  background-image: -webkit-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:      -o-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:         linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
}
.progress-bar-warning {
  background-color: #f0ad4e;
}
.progress-striped .progress-bar-warning {
  background-image: -webkit-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:      -o-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:         linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
}
.progress-bar-danger {
  background-color: #d9534f;
}
.progress-striped .progress-bar-danger {
  background-image: -webkit-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:      -o-linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
  background-image:         linear-gradient(45deg, rgba(255, 255, 255, .15) 25%, transparent 25%, transparent 50%, rgba(255, 255, 255, .15) 50%, rgba(255, 255, 255, .15) 75%, transparent 75%, transparent);
}
.media {
  margin-top: 15px;
}
.media:first-child {
  margin-top: 0;
}
.media,
.media-body {
  overflow: hidden;
  zoom: 1;
}
.media-body {
  width: 10000px;
}
.media-object {
  display: block;
}
.media-right,
.media > .pull-right {
  padding-left: 10px;
}
.media-left,
.media > .pull-left {
  padding-right: 10px;
}
.media-left,
.media-right,
.media-body {
  display: table-cell;
  vertical-align: top;
}
.media-middle {
  vertical-align: middle;
}
.media-bottom {
  vertical-align: bottom;
}
.media-heading {
  margin-top: 0;
  margin-bottom: 5px;
}
.media-list {
  padding-left: 0;
  list-style: none;
}
.list-group {
  padding-left: 0;
  margin-bottom: 20px;
}
.list-group-item {
  position: relative;
  display: block;
  padding: 10px 15px;
  margin-bottom: -1px;
  background-color: #fff;
  border: 1px solid #ddd;
}
.list-group-item:first-child {
  border-top-left-radius: 4px;
  border-top-right-radius: 4px;
}
.list-group-item:last-child {
  margin-bottom: 0;
  border-bottom-right-radius: 4px;
  border-bottom-left-radius: 4px;
}
a.list-group-item {
  color: #555;
}
a.list-group-item .list-group-item-heading {
  color: #333;
}
a.list-group-item:hover,
a.list-group-item:focus {
  color: #555;
  text-decoration: none;
  background-color: #f5f5f5;
}
.list-group-item.disabled,
.list-group-item.disabled:hover,
.list-group-item.disabled:focus {
  color: #777;
  cursor: not-allowed;
  background-color: #eee;
}
.list-group-item.disabled .list-group-item-heading,
.list-group-item.disabled:hover .list-group-item-heading,
.list-group-item.disabled:focus .list-group-item-heading {
  color: inherit;
}
.list-group-item.disabled .list-group-item-text,
.list-group-item.disabled:hover .list-group-item-text,
.list-group-item.disabled:focus .list-group-item-text {
  color: #777;
}
.list-group-item.active,
.list-group-item.active:hover,
.list-group-item.active:focus {
  z-index: 2;
  color: #fff;
  background-color: #337ab7;
  border-color: #337ab7;
}
.list-group-item.active .list-group-item-heading,
.list-group-item.active:hover .list-group-item-heading,
.list-group-item.active:focus .list-group-item-heading,
.list-group-item.active .list-group-item-heading > small,
.list-group-item.active:hover .list-group-item-heading > small,
.list-group-item.active:focus .list-group-item-heading > small,
.list-group-item.active .list-group-item-heading > .small,
.list-group-item.active:hover .list-group-item-heading > .small,
.list-group-item.active:focus .list-group-item-heading > .small {
  color: inherit;
}
.list-group-item.active .list-group-item-text,
.list-group-item.active:hover .list-group-item-text,
.list-group-item.active:focus .list-group-item-text {
  color: #c7ddef;
}
.list-group-item-success {
  color: #3c763d;
  background-color: #dff0d8;
}
a.list-group-item-success {
  color: #3c763d;
}
a.list-group-item-success .list-group-item-heading {
  color: inherit;
}
a.list-group-item-success:hover,
a.list-group-item-success:focus {
  color: #3c763d;
  background-color: #d0e9c6;
}
a.list-group-item-success.active,
a.list-group-item-success.active:hover,
a.list-group-item-success.active:focus {
  color: #fff;
  background-color: #3c763d;
  border-color: #3c763d;
}
.list-group-item-info {
  color: #31708f;
  background-color: #d9edf7;
}
a.list-group-item-info {
  color: #31708f;
}
a.list-group-item-info .list-group-item-heading {
  color: inherit;
}
a.list-group-item-info:hover,
a.list-group-item-info:focus {
  color: #31708f;
  background-color: #c4e3f3;
}
a.list-group-item-info.active,
a.list-group-item-info.active:hover,
a.list-group-item-info.active:focus {
  color: #fff;
  background-color: #31708f;
  border-color: #31708f;
}
.list-group-item-warning {
  color: #8a6d3b;
  background-color: #fcf8e3;
}
a.list-group-item-warning {
  color: #8a6d3b;
}
a.list-group-item-warning .list-group-item-heading {
  color: inherit;
}
a.list-group-item-warning:hover,
a.list-group-item-warning:focus {
  color: #8a6d3b;
  background-color: #faf2cc;
}
a.list-group-item-warning.active,
a.list-group-item-warning.active:hover,
a.list-group-item-warning.active:focus {
  color: #fff;
  background-color: #8a6d3b;
  border-color: #8a6d3b;
}
.list-group-item-danger {
  color: #a94442;
  background-color: #f2dede;
}
a.list-group-item-danger {
  color: #a94442;
}
a.list-group-item-danger .list-group-item-heading {
  color: inherit;
}
a.list-group-item-danger:hover,
a.list-group-item-danger:focus {
  color: #a94442;
  background-color: #ebcccc;
}
a.list-group-item-danger.active,
a.list-group-item-danger.active:hover,
a.list-group-item-danger.active:focus {
  color: #fff;
  background-color: #a94442;
  border-color: #a94442;
}
.list-group-item-heading {
  margin-top: 0;
  margin-bottom: 5px;
}
.list-group-item-text {
  margin-bottom: 0;
  line-height: 1.3;
}
.panel {
  margin-bottom: 20px;
  background-color: #fff;
  border: 1px solid transparent;
  border-radius: 4px;
  -webkit-box-shadow: 0 1px 1px rgba(0, 0, 0, .05);
          box-shadow: 0 1px 1px rgba(0, 0, 0, .05);
}
.panel-body {
  padding: 15px;
}
.panel-heading {
  padding: 10px 15px;
  border-bottom: 1px solid transparent;
  border-top-left-radius: 3px;
  border-top-right-radius: 3px;
}
.panel-heading > .dropdown .dropdown-toggle {
  color: inherit;
}
.panel-title {
  margin-top: 0;
  margin-bottom: 0;
  font-size: 16px;
  color: inherit;
}
.panel-title > a,
.panel-title > small,
.panel-title > .small,
.panel-title > small > a,
.panel-title > .small > a {
  color: inherit;
}
.panel-footer {
  padding: 10px 15px;
  background-color: #f5f5f5;
  border-top: 1px solid #ddd;
  border-bottom-right-radius: 3px;
  border-bottom-left-radius: 3px;
}
.panel > .list-group,
.panel > .panel-collapse > .list-group {
  margin-bottom: 0;
}
.panel > .list-group .list-group-item,
.panel > .panel-collapse > .list-group .list-group-item {
  border-width: 1px 0;
  border-radius: 0;
}
.panel > .list-group:first-child .list-group-item:first-child,
.panel > .panel-collapse > .list-group:first-child .list-group-item:first-child {
  border-top: 0;
  border-top-left-radius: 3px;
  border-top-right-radius: 3px;
}
.panel > .list-group:last-child .list-group-item:last-child,
.panel > .panel-collapse > .list-group:last-child .list-group-item:last-child {
  border-bottom: 0;
  border-bottom-right-radius: 3px;
  border-bottom-left-radius: 3px;
}
.panel-heading + .list-group .list-group-item:first-child {
  border-top-width: 0;
}
.list-group + .panel-footer {
  border-top-width: 0;
}
.panel > .table,
.panel > .table-responsive > .table,
.panel > .panel-collapse > .table {
  margin-bottom: 0;
}
.panel > .table caption,
.panel > .table-responsive > .table caption,
.panel > .panel-collapse > .table caption {
  padding-right: 15px;
  padding-left: 15px;
}
.panel > .table:first-child,
.panel > .table-responsive:first-child > .table:first-child {
  border-top-left-radius: 3px;
  border-top-right-radius: 3px;
}
.panel > .table:first-child > thead:first-child > tr:first-child,
.panel > .table-responsive:first-child > .table:first-child > thead:first-child > tr:first-child,
.panel > .table:first-child > tbody:first-child > tr:first-child,
.panel > .table-responsive:first-child > .table:first-child > tbody:first-child > tr:first-child {
  border-top-left-radius: 3px;
  border-top-right-radius: 3px;
}
.panel > .table:first-child > thead:first-child > tr:first-child td:first-child,
.panel > .table-responsive:first-child > .table:first-child > thead:first-child > tr:first-child td:first-child,
.panel > .table:first-child > tbody:first-child > tr:first-child td:first-child,
.panel > .table-responsive:first-child > .table:first-child > tbody:first-child > tr:first-child td:first-child,
.panel > .table:first-child > thead:first-child > tr:first-child th:first-child,
.panel > .table-responsive:first-child > .table:first-child > thead:first-child > tr:first-child th:first-child,
.panel > .table:first-child > tbody:first-child > tr:first-child th:first-child,
.panel > .table-responsive:first-child > .table:first-child > tbody:first-child > tr:first-child th:first-child {
  border-top-left-radius: 3px;
}
.panel > .table:first-child > thead:first-child > tr:first-child td:last-child,
.panel > .table-responsive:first-child > .table:first-child > thead:first-child > tr:first-child td:last-child,
.panel > .table:first-child > tbody:first-child > tr:first-child td:last-child,
.panel > .table-responsive:first-child > .table:first-child > tbody:first-child > tr:first-child td:last-child,
.panel > .table:first-child > thead:first-child > tr:first-child th:last-child,
.panel > .table-responsive:first-child > .table:first-child > thead:first-child > tr:first-child th:last-child,
.panel > .table:first-child > tbody:first-child > tr:first-child th:last-child,
.panel > .table-responsive:first-child > .table:first-child > tbody:first-child > tr:first-child th:last-child {
  border-top-right-radius: 3px;
}
.panel > .table:last-child,
.panel > .table-responsive:last-child > .table:last-child {
  border-bottom-right-radius: 3px;
  border-bottom-left-radius: 3px;
}
.panel > .table:last-child > tbody:last-child > tr:last-child,
.panel > .table-responsive:last-child > .table:last-child > tbody:last-child > tr:last-child,
.panel > .table:last-child > tfoot:last-child > tr:last-child,
.panel > .table-responsive:last-child > .table:last-child > tfoot:last-child > tr:last-child {
  border-bottom-right-radius: 3px;
  border-bottom-left-radius: 3px;
}
.panel > .table:last-child > tbody:last-child > tr:last-child td:first-child,
.panel > .table-responsive:last-child > .table:last-child > tbody:last-child > tr:last-child td:first-child,
.panel > .table:last-child > tfoot:last-child > tr:last-child td:first-child,
.panel > .table-responsive:last-child > .table:last-child > tfoot:last-child > tr:last-child td:first-child,
.panel > .table:last-child > tbody:last-child > tr:last-child th:first-child,
.panel > .table-responsive:last-child > .table:last-child > tbody:last-child > tr:last-child th:first-child,
.panel > .table:last-child > tfoot:last-child > tr:last-child th:first-child,
.panel > .table-responsive:last-child > .table:last-child > tfoot:last-child > tr:last-child th:first-child {
  border-bottom-left-radius: 3px;
}
.panel > .table:last-child > tbody:last-child > tr:last-child td:last-child,
.panel > .table-responsive:last-child > .table:last-child > tbody:last-child > tr:last-child td:last-child,
.panel > .table:last-child > tfoot:last-child > tr:last-child td:last-child,
.panel > .table-responsive:last-child > .table:last-child > tfoot:last-child > tr:last-child td:last-child,
.panel > .table:last-child > tbody:last-child > tr:last-child th:last-child,
.panel > .table-responsive:last-child > .table:last-child > tbody:last-child > tr:last-child th:last-child,
.panel > .table:last-child > tfoot:last-child > tr:last-child th:last-child,
.panel > .table-responsive:last-child > .table:last-child > tfoot:last-child > tr:last-child th:last-child {
  border-bottom-right-radius: 3px;
}
.panel > .panel-body + .table,
.panel > .panel-body + .table-responsive,
.panel > .table + .panel-body,
.panel > .table-responsive + .panel-body {
  border-top: 1px solid #ddd;
}
.panel > .table > tbody:first-child > tr:first-child th,
.panel > .table > tbody:first-child > tr:first-child td {
  border-top: 0;
}
.panel > .table-bordered,
.panel > .table-responsive > .table-bordered {
  border: 0;
}
.panel > .table-bordered > thead > tr > th:first-child,
.panel > .table-responsive > .table-bordered > thead > tr > th:first-child,
.panel > .table-bordered > tbody > tr > th:first-child,
.panel > .table-responsive > .table-bordered > tbody > tr > th:first-child,
.panel > .table-bordered > tfoot > tr > th:first-child,
.panel > .table-responsive > .table-bordered > tfoot > tr > th:first-child,
.panel > .table-bordered > thead > tr > td:first-child,
.panel > .table-responsive > .table-bordered > thead > tr > td:first-child,
.panel > .table-bordered > tbody > tr > td:first-child,
.panel > .table-responsive > .table-bordered > tbody > tr > td:first-child,
.panel > .table-bordered > tfoot > tr > td:first-child,
.panel > .table-responsive > .table-bordered > tfoot > tr > td:first-child {
  border-left: 0;
}
.panel > .table-bordered > thead > tr > th:last-child,
.panel > .table-responsive > .table-bordered > thead > tr > th:last-child,
.panel > .table-bordered > tbody > tr > th:last-child,
.panel > .table-responsive > .table-bordered > tbody > tr > th:last-child,
.panel > .table-bordered > tfoot > tr > th:last-child,
.panel > .table-responsive > .table-bordered > tfoot > tr > th:last-child,
.panel > .table-bordered > thead > tr > td:last-child,
.panel > .table-responsive > .table-bordered > thead > tr > td:last-child,
.panel > .table-bordered > tbody > tr > td:last-child,
.panel > .table-responsive > .table-bordered > tbody > tr > td:last-child,
.panel > .table-bordered > tfoot > tr > td:last-child,
.panel > .table-responsive > .table-bordered > tfoot > tr > td:last-child {
  border-right: 0;
}
.panel > .table-bordered > thead > tr:first-child > td,
.panel > .table-responsive > .table-bordered > thead > tr:first-child > td,
.panel > .table-bordered > tbody > tr:first-child > td,
.panel > .table-responsive > .table-bordered > tbody > tr:first-child > td,
.panel > .table-bordered > thead > tr:first-child > th,
.panel > .table-responsive > .table-bordered > thead > tr:first-child > th,
.panel > .table-bordered > tbody > tr:first-child > th,
.panel > .table-responsive > .table-bordered > tbody > tr:first-child > th {
  border-bottom: 0;
}
.panel > .table-bordered > tbody > tr:last-child > td,
.panel > .table-responsive > .table-bordered > tbody > tr:last-child > td,
.panel > .table-bordered > tfoot > tr:last-child > td,
.panel > .table-responsive > .table-bordered > tfoot > tr:last-child > td,
.panel > .table-bordered > tbody > tr:last-child > th,
.panel > .table-responsive > .table-bordered > tbody > tr:last-child > th,
.panel > .table-bordered > tfoot > tr:last-child > th,
.panel > .table-responsive > .table-bordered > tfoot > tr:last-child > th {
  border-bottom: 0;
}
.panel > .table-responsive {
  margin-bottom: 0;
  border: 0;
}
.panel-group {
  margin-bottom: 20px;
}
.panel-group .panel {
  margin-bottom: 0;
  border-radius: 4px;
}
.panel-group .panel + .panel {
  margin-top: 5px;
}
.panel-group .panel-heading {
  border-bottom: 0;
}
.panel-group .panel-heading + .panel-collapse > .panel-body,
.panel-group .panel-heading + .panel-collapse > .list-group {
  border-top: 1px solid #ddd;
}
.panel-group .panel-footer {
  border-top: 0;
}
.panel-group .panel-footer + .panel-collapse .panel-body {
  border-bottom: 1px solid #ddd;
}
.panel-default {
  border-color: #ddd;
}
.panel-default > .panel-heading {
  color: #333;
  background-color: #f5f5f5;
  border-color: #ddd;
}
.panel-default > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #ddd;
}
.panel-default > .panel-heading .badge {
  color: #f5f5f5;
  background-color: #333;
}
.panel-default > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #ddd;
}
.panel-primary {
  border-color: #337ab7;
}
.panel-primary > .panel-heading {
  color: #fff;
  background-color: #337ab7;
  border-color: #337ab7;
}
.panel-primary > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #337ab7;
}
.panel-primary > .panel-heading .badge {
  color: #337ab7;
  background-color: #fff;
}
.panel-primary > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #337ab7;
}
.panel-success {
  border-color: #d6e9c6;
}
.panel-success > .panel-heading {
  color: #3c763d;
  background-color: #dff0d8;
  border-color: #d6e9c6;
}
.panel-success > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #d6e9c6;
}
.panel-success > .panel-heading .badge {
  color: #dff0d8;
  background-color: #3c763d;
}
.panel-success > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #d6e9c6;
}
.panel-info {
  border-color: #bce8f1;
}
.panel-info > .panel-heading {
  color: #31708f;
  background-color: #d9edf7;
  border-color: #bce8f1;
}
.panel-info > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #bce8f1;
}
.panel-info > .panel-heading .badge {
  color: #d9edf7;
  background-color: #31708f;
}
.panel-info > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #bce8f1;
}
.panel-warning {
  border-color: #faebcc;
}
.panel-warning > .panel-heading {
  color: #8a6d3b;
  background-color: #fcf8e3;
  border-color: #faebcc;
}
.panel-warning > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #faebcc;
}
.panel-warning > .panel-heading .badge {
  color: #fcf8e3;
  background-color: #8a6d3b;
}
.panel-warning > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #faebcc;
}
.panel-danger {
  border-color: #ebccd1;
}
.panel-danger > .panel-heading {
  color: #a94442;
  background-color: #f2dede;
  border-color: #ebccd1;
}
.panel-danger > .panel-heading + .panel-collapse > .panel-body {
  border-top-color: #ebccd1;
}
.panel-danger > .panel-heading .badge {
  color: #f2dede;
  background-color: #a94442;
}
.panel-danger > .panel-footer + .panel-collapse > .panel-body {
  border-bottom-color: #ebccd1;
}
.embed-responsive {
  position: relative;
  display: block;
  height: 0;
  padding: 0;
  overflow: hidden;
}
.embed-responsive .embed-responsive-item,
.embed-responsive iframe,
.embed-responsive embed,
.embed-responsive object,
.embed-responsive video {
  position: absolute;
  top: 0;
  bottom: 0;
  left: 0;
  width: 100%;
  height: 100%;
  border: 0;
}
.embed-responsive-16by9 {
  padding-bottom: 56.25%;
}
.embed-responsive-4by3 {
  padding-bottom: 75%;
}
.well {
  min-height: 20px;
  padding: 19px;
  margin-bottom: 20px;
  background-color: #f5f5f5;
  border: 1px solid #e3e3e3;
  border-radius: 4px;
  -webkit-box-shadow: inset 0 1px 1px rgba(0, 0, 0, .05);
          box-shadow: inset 0 1px 1px rgba(0, 0, 0, .05);
}
.well blockquote {
  border-color: #ddd;
  border-color: rgba(0, 0, 0, .15);
}
.well-lg {
  padding: 24px;
  border-radius: 6px;
}
.well-sm {
  padding: 9px;
  border-radius: 3px;
}
.close {
  float: right;
  font-size: 21px;
  font-weight: bold;
  line-height: 1;
  color: #000;
  text-shadow: 0 1px 0 #fff;
  filter: alpha(opacity=20);
  opacity: .2;
}
.close:hover,
.close:focus {
  color: #000;
  text-decoration: none;
  cursor: pointer;
  filter: alpha(opacity=50);
  opacity: .5;
}
button.close {
  -webkit-appearance: none;
  padding: 0;
  cursor: pointer;
  background: transparent;
  border: 0;
}
.modal-open {
  overflow: hidden;
}
.modal {
  position: fixed;
  top: 0;
  right: 0;
  bottom: 0;
  left: 0;
  z-index: 1050;
  display: none;
  overflow: hidden;
  -webkit-overflow-scrolling: touch;
  outline: 0;
}
.modal.fade .modal-dialog {
  -webkit-transition: -webkit-transform .3s ease-out;
       -o-transition:      -o-transform .3s ease-out;
          transition:         transform .3s ease-out;
  -webkit-transform: translate(0, -25%);
      -ms-transform: translate(0, -25%);
       -o-transform: translate(0, -25%);
          transform: translate(0, -25%);
}
.modal.in .modal-dialog {
  -webkit-transform: translate(0, 0);
      -ms-transform: translate(0, 0);
       -o-transform: translate(0, 0);
          transform: translate(0, 0);
}
.modal-open .modal {
  overflow-x: hidden;
  overflow-y: auto;
}
.modal-dialog {
  position: relative;
  width: auto;
  margin: 10px;
}
.modal-content {
  position: relative;
  background-color: #fff;
  -webkit-background-clip: padding-box;
          background-clip: padding-box;
  border: 1px solid #999;
  border: 1px solid rgba(0, 0, 0, .2);
  border-radius: 6px;
  outline: 0;
  -webkit-box-shadow: 0 3px 9px rgba(0, 0, 0, .5);
          box-shadow: 0 3px 9px rgba(0, 0, 0, .5);
}
.modal-backdrop {
  position: fixed;
  top: 0;
  right: 0;
  bottom: 0;
  left: 0;
  z-index: 1040;
  background-color: #000;
}
.modal-backdrop.fade {
  filter: alpha(opacity=0);
  opacity: 0;
}
.modal-backdrop.in {
  filter: alpha(opacity=50);
  opacity: .5;
}
.modal-header {
  min-height: 16.42857143px;
  padding: 15px;
  border-bottom: 1px solid #e5e5e5;
}
.modal-header .close {
  margin-top: -2px;
}
.modal-title {
  margin: 0;
  line-height: 1.42857143;
}
.modal-body {
  position: relative;
  padding: 15px;
}
.modal-footer {
  padding: 15px;
  text-align: right;
  border-top: 1px solid #e5e5e5;
}
.modal-footer .btn + .btn {
  margin-bottom: 0;
  margin-left: 5px;
}
.modal-footer .btn-group .btn + .btn {
  margin-left: -1px;
}
.modal-footer .btn-block + .btn-block {
  margin-left: 0;
}
.modal-scrollbar-measure {
  position: absolute;
  top: -9999px;
  width: 50px;
  height: 50px;
  overflow: scroll;
}
@media (min-width: 768px) {
  .modal-dialog {
    width: 600px;
    margin: 30px auto;
  }
  .modal-content {
    -webkit-box-shadow: 0 5px 15px rgba(0, 0, 0, .5);
            box-shadow: 0 5px 15px rgba(0, 0, 0, .5);
  }
  .modal-sm {
    width: 300px;
  }
}
@media (min-width: 992px) {
  .modal-lg {
    width: 900px;
  }
}
.tooltip {
  position: absolute;
  z-index: 1070;
  display: block;
  font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
  font-size: 12px;
  font-weight: normal;
  line-height: 1.4;
  filter: alpha(opacity=0);
  opacity: 0;
}
.tooltip.in {
  filter: alpha(opacity=90);
  opacity: .9;
}
.tooltip.top {
  padding: 5px 0;
  margin-top: -3px;
}
.tooltip.right {
  padding: 0 5px;
  margin-left: 3px;
}
.tooltip.bottom {
  padding: 5px 0;
  margin-top: 3px;
}
.tooltip.left {
  padding: 0 5px;
  margin-left: -3px;
}
.tooltip-inner {
  max-width: 200px;
  padding: 3px 8px;
  color: #fff;
  text-align: center;
  text-decoration: none;
  background-color: #000;
  border-radius: 4px;
}
.tooltip-arrow {
  position: absolute;
  width: 0;
  height: 0;
  border-color: transparent;
  border-style: solid;
}
.tooltip.top .tooltip-arrow {
  bottom: 0;
  left: 50%;
  margin-left: -5px;
  border-width: 5px 5px 0;
  border-top-color: #000;
}
.tooltip.top-left .tooltip-arrow {
  right: 5px;
  bottom: 0;
  margin-bottom: -5px;
  border-width: 5px 5px 0;
  border-top-color: #000;
}
.tooltip.top-right .tooltip-arrow {
  bottom: 0;
  left: 5px;
  margin-bottom: -5px;
  border-width: 5px 5px 0;
  border-top-color: #000;
}
.tooltip.right .tooltip-arrow {
  top: 50%;
  left: 0;
  margin-top: -5px;
  border-width: 5px 5px 5px 0;
  border-right-color: #000;
}
.tooltip.left .tooltip-arrow {
  top: 50%;
  right: 0;
  margin-top: -5px;
  border-width: 5px 0 5px 5px;
  border-left-color: #000;
}
.tooltip.bottom .tooltip-arrow {
  top: 0;
  left: 50%;
  margin-left: -5px;
  border-width: 0 5px 5px;
  border-bottom-color: #000;
}
.tooltip.bottom-left .tooltip-arrow {
  top: 0;
  right: 5px;
  margin-top: -5px;
  border-width: 0 5px 5px;
  border-bottom-color: #000;
}
.tooltip.bottom-right .tooltip-arrow {
  top: 0;
  left: 5px;
  margin-top: -5px;
  border-width: 0 5px 5px;
  border-bottom-color: #000;
}
.popover {
  position: absolute;
  top: 0;
  left: 0;
  z-index: 1060;
  display: none;
  max-width: 276px;
  padding: 1px;
  font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
  font-size: 14px;
  font-weight: normal;
  line-height: 1.42857143;
  text-align: left;
  white-space: normal;
  background-color: #fff;
  -webkit-background-clip: padding-box;
          background-clip: padding-box;
  border: 1px solid #ccc;
  border: 1px solid rgba(0, 0, 0, .2);
  border-radius: 6px;
  -webkit-box-shadow: 0 5px 10px rgba(0, 0, 0, .2);
          box-shadow: 0 5px 10px rgba(0, 0, 0, .2);
}
.popover.top {
  margin-top: -10px;
}
.popover.right {
  margin-left: 10px;
}
.popover.bottom {
  margin-top: 10px;
}
.popover.left {
  margin-left: -10px;
}
.popover-title {
  padding: 8px 14px;
  margin: 0;
  font-size: 14px;
  background-color: #f7f7f7;
  border-bottom: 1px solid #ebebeb;
  border-radius: 5px 5px 0 0;
}
.popover-content {
  padding: 9px 14px;
}
.popover > .arrow,
.popover > .arrow:after {
  position: absolute;
  display: block;
  width: 0;
  height: 0;
  border-color: transparent;
  border-style: solid;
}
.popover > .arrow {
  border-width: 11px;
}
.popover > .arrow:after {
  content: "";
  border-width: 10px;
}
.popover.top > .arrow {
  bottom: -11px;
  left: 50%;
  margin-left: -11px;
  border-top-color: #999;
  border-top-color: rgba(0, 0, 0, .25);
  border-bottom-width: 0;
}
.popover.top > .arrow:after {
  bottom: 1px;
  margin-left: -10px;
  content: " ";
  border-top-color: #fff;
  border-bottom-width: 0;
}
.popover.right > .arrow {
  top: 50%;
  left: -11px;
  margin-top: -11px;
  border-right-color: #999;
  border-right-color: rgba(0, 0, 0, .25);
  border-left-width: 0;
}
.popover.right > .arrow:after {
  bottom: -10px;
  left: 1px;
  content: " ";
  border-right-color: #fff;
  border-left-width: 0;
}
.popover.bottom > .arrow {
  top: -11px;
  left: 50%;
  margin-left: -11px;
  border-top-width: 0;
  border-bottom-color: #999;
  border-bottom-color: rgba(0, 0, 0, .25);
}
.popover.bottom > .arrow:after {
  top: 1px;
  margin-left: -10px;
  content: " ";
  border-top-width: 0;
  border-bottom-color: #fff;
}
.popover.left > .arrow {
  top: 50%;
  right: -11px;
  margin-top: -11px;
  border-right-width: 0;
  border-left-color: #999;
  border-left-color: rgba(0, 0, 0, .25);
}
.popover.left > .arrow:after {
  right: 1px;
  bottom: -10px;
  content: " ";
  border-right-width: 0;
  border-left-color: #fff;
}
.carousel {
  position: relative;
}
.carousel-inner {
  position: relative;
  width: 100%;
  overflow: hidden;
}
.carousel-inner > .item {
  position: relative;
  display: none;
  -webkit-transition: .6s ease-in-out left;
       -o-transition: .6s ease-in-out left;
          transition: .6s ease-in-out left;
}
.carousel-inner > .item > img,
.carousel-inner > .item > a > img {
  line-height: 1;
}
@media all and (transform-3d), (-webkit-transform-3d) {
  .carousel-inner > .item {
    -webkit-transition: -webkit-transform .6s ease-in-out;
         -o-transition:      -o-transform .6s ease-in-out;
            transition:         transform .6s ease-in-out;

    -webkit-backface-visibility: hidden;
            backface-visibility: hidden;
    -webkit-perspective: 1000;
            perspective: 1000;
  }
  .carousel-inner > .item.next,
  .carousel-inner > .item.active.right {
    left: 0;
    -webkit-transform: translate3d(100%, 0, 0);
            transform: translate3d(100%, 0, 0);
  }
  .carousel-inner > .item.prev,
  .carousel-inner > .item.active.left {
    left: 0;
    -webkit-transform: translate3d(-100%, 0, 0);
            transform: translate3d(-100%, 0, 0);
  }
  .carousel-inner > .item.next.left,
  .carousel-inner > .item.prev.right,
  .carousel-inner > .item.active {
    left: 0;
    -webkit-transform: translate3d(0, 0, 0);
            transform: translate3d(0, 0, 0);
  }
}
.carousel-inner > .active,
.carousel-inner > .next,
.carousel-inner > .prev {
  display: block;
}
.carousel-inner > .active {
  left: 0;
}
.carousel-inner > .next,
.carousel-inner > .prev {
  position: absolute;
  top: 0;
  width: 100%;
}
.carousel-inner > .next {
  left: 100%;
}
.carousel-inner > .prev {
  left: -100%;
}
.carousel-inner > .next.left,
.carousel-inner > .prev.right {
  left: 0;
}
.carousel-inner > .active.left {
  left: -100%;
}
.carousel-inner > .active.right {
  left: 100%;
}
.carousel-control {
  position: absolute;
  top: 0;
  bottom: 0;
  left: 0;
  width: 15%;
  font-size: 20px;
  color: #fff;
  text-align: center;
  text-shadow: 0 1px 2px rgba(0, 0, 0, .6);
  filter: alpha(opacity=50);
  opacity: .5;
}
.carousel-control.left {
  background-image: -webkit-linear-gradient(left, rgba(0, 0, 0, .5) 0%, rgba(0, 0, 0, .0001) 100%);
  background-image:      -o-linear-gradient(left, rgba(0, 0, 0, .5) 0%, rgba(0, 0, 0, .0001) 100%);
  background-image: -webkit-gradient(linear, left top, right top, from(rgba(0, 0, 0, .5)), to(rgba(0, 0, 0, .0001)));
  background-image:         linear-gradient(to right, rgba(0, 0, 0, .5) 0%, rgba(0, 0, 0, .0001) 100%);
  filter: progid:DXImageTransform.Microsoft.gradient(startColorstr='#80000000', endColorstr='#00000000', GradientType=1);
  background-repeat: repeat-x;
}
.carousel-control.right {
  right: 0;
  left: auto;
  background-image: -webkit-linear-gradient(left, rgba(0, 0, 0, .0001) 0%, rgba(0, 0, 0, .5) 100%);
  background-image:      -o-linear-gradient(left, rgba(0, 0, 0, .0001) 0%, rgba(0, 0, 0, .5) 100%);
  background-image: -webkit-gradient(linear, left top, right top, from(rgba(0, 0, 0, .0001)), to(rgba(0, 0, 0, .5)));
  background-image:         linear-gradient(to right, rgba(0, 0, 0, .0001) 0%, rgba(0, 0, 0, .5) 100%);
  filter: progid:DXImageTransform.Microsoft.gradient(startColorstr='#00000000', endColorstr='#80000000', GradientType=1);
  background-repeat: repeat-x;
}
.carousel-control:hover,
.carousel-control:focus {
  color: #fff;
  text-decoration: none;
  filter: alpha(opacity=90);
  outline: 0;
  opacity: .9;
}
.carousel-control .icon-prev,
.carousel-control .icon-next,
.carousel-control .glyphicon-chevron-left,
.carousel-control .glyphicon-chevron-right {
  position: absolute;
  top: 50%;
  z-index: 5;
  display: inline-block;
}
.carousel-control .icon-prev,
.carousel-control .glyphicon-chevron-left {
  left: 50%;
  margin-left: -10px;
}
.carousel-control .icon-next,
.carousel-control .glyphicon-chevron-right {
  right: 50%;
  margin-right: -10px;
}
.carousel-control .icon-prev,
.carousel-control .icon-next {
  width: 20px;
  height: 20px;
  margin-top: -10px;
  font-family: serif;
  line-height: 1;
}
.carousel-control .icon-prev:before {
  content: '\2039';
}
.carousel-control .icon-next:before {
  content: '\203a';
}
.carousel-indicators {
  position: absolute;
  bottom: 10px;
  left: 50%;
  z-index: 15;
  width: 60%;
  padding-left: 0;
  margin-left: -30%;
  text-align: center;
  list-style: none;
}
.carousel-indicators li {
  display: inline-block;
  width: 10px;
  height: 10px;
  margin: 1px;
  text-indent: -999px;
  cursor: pointer;
  background-color: #000 \9;
  background-color: rgba(0, 0, 0, 0);
  border: 1px solid #fff;
  border-radius: 10px;
}
.carousel-indicators .active {
  width: 12px;
  height: 12px;
  margin: 0;
  background-color: #fff;
}
.carousel-caption {
  position: absolute;
  right: 15%;
  bottom: 20px;
  left: 15%;
  z-index: 10;
  padding-top: 20px;
  padding-bottom: 20px;
  color: #fff;
  text-align: center;
  text-shadow: 0 1px 2px rgba(0, 0, 0, .6);
}
.carousel-caption .btn {
  text-shadow: none;
}
@media screen and (min-width: 768px) {
  .carousel-control .glyphicon-chevron-left,
  .carousel-control .glyphicon-chevron-right,
  .carousel-control .icon-prev,
  .carousel-control .icon-next {
    width: 30px;
    height: 30px;
    margin-top: -15px;
    font-size: 30px;
  }
  .carousel-control .glyphicon-chevron-left,
  .carousel-control .icon-prev {
    margin-left: -15px;
  }
  .carousel-control .glyphicon-chevron-right,
  .carousel-control .icon-next {
    margin-right: -15px;
  }
  .carousel-caption {
    right: 20%;
    left: 20%;
    padding-bottom: 30px;
  }
  .carousel-indicators {
    bottom: 20px;
  }
}
.clearfix:before,
.clearfix:after,
.dl-horizontal dd:before,
.dl-horizontal dd:after,
.container:before,
.container:after,
.container-fluid:before,
.container-fluid:after,
.row:before,
.row:after,
.form-horizontal .form-group:before,
.form-horizontal .form-group:after,
.btn-toolbar:before,
.btn-toolbar:after,
.btn-group-vertical > .btn-group:before,
.btn-group-vertical > .btn-group:after,
.nav:before,
.nav:after,
.navbar:before,
.navbar:after,
.navbar-header:before,
.navbar-header:after,
.navbar-collapse:before,
.navbar-collapse:after,
.pager:before,
.pager:after,
.panel-body:before,
.panel-body:after,
.modal-footer:before,
.modal-footer:after {
  display: table;
  content: " ";
}
.clearfix:after,
.dl-horizontal dd:after,
.container:after,
.container-fluid:after,
.row:after,
.form-horizontal .form-group:after,
.btn-toolbar:after,
.btn-group-vertical > .btn-group:after,
.nav:after,
.navbar:after,
.navbar-header:after,
.navbar-collapse:after,
.pager:after,
.panel-body:after,
.modal-footer:after {
  clear: both;
}
.center-block {
  display: block;
  margin-right: auto;
  margin-left: auto;
}
.pull-right {
  float: right !important;
}
.pull-left {
  float: left !important;
}
.hide {
  display: none !important;
}
.show {
  display: block !important;
}
.invisible {
  visibility: hidden;
}
.text-hide {
  font: 0/0 a;
  color: transparent;
  text-shadow: none;
  background-color: transparent;
  border: 0;
}
.hidden {
  display: none !important;
}
.affix {
  position: fixed;
}
@-ms-viewport {
  width: device-width;
}
.visible-xs,
.visible-sm,
.visible-md,
.visible-lg {
  display: none !important;
}
.visible-xs-block,
.visible-xs-inline,
.visible-xs-inline-block,
.visible-sm-block,
.visible-sm-inline,
.visible-sm-inline-block,
.visible-md-block,
.visible-md-inline,
.visible-md-inline-block,
.visible-lg-block,
.visible-lg-inline,
.visible-lg-inline-block {
  display: none !important;
}
@media (max-width: 767px) {
  .visible-xs {
    display: block !important;
  }
  table.visible-xs {
    display: table;
  }
  tr.visible-xs {
    display: table-row !important;
  }
  th.visible-xs,
  td.visible-xs {
    display: table-cell !important;
  }
}
@media (max-width: 767px) {
  .visible-xs-block {
    display: block !important;
  }
}
@media (max-width: 767px) {
  .visible-xs-inline {
    display: inline !important;
  }
}
@media (max-width: 767px) {
  .visible-xs-inline-block {
    display: inline-block !important;
  }
}
@media (min-width: 768px) and (max-width: 991px) {
  .visible-sm {
    display: block !important;
  }
  table.visible-sm {
    display: table;
  }
  tr.visible-sm {
    display: table-row !important;
  }
  th.visible-sm,
  td.visible-sm {
    display: table-cell !important;
  }
}
@media (min-width: 768px) and (max-width: 991px) {
  .visible-sm-block {
    display: block !important;
  }
}
@media (min-width: 768px) and (max-width: 991px) {
  .visible-sm-inline {
    display: inline !important;
  }
}
@media (min-width: 768px) and (max-width: 991px) {
  .visible-sm-inline-block {
    display: inline-block !important;
  }
}
@media (min-width: 992px) and (max-width: 1199px) {
  .visible-md {
    display: block !important;
  }
  table.visible-md {
    display: table;
  }
  tr.visible-md {
    display: table-row !important;
  }
  th.visible-md,
  td.visible-md {
    display: table-cell !important;
  }
}
@media (min-width: 992px) and (max-width: 1199px) {
  .visible-md-block {
    display: block !important;
  }
}
@media (min-width: 992px) and (max-width: 1199px) {
  .visible-md-inline {
    display: inline !important;
  }
}
@media (min-width: 992px) and (max-width: 1199px) {
  .visible-md-inline-block {
    display: inline-block !important;
  }
}
@media (min-width: 1200px) {
  .visible-lg {
    display: block !important;
  }
  table.visible-lg {
    display: table;
  }
  tr.visible-lg {
    display: table-row !important;
  }
  th.visible-lg,
  td.visible-lg {
    display: table-cell !important;
  }
}
@media (min-width: 1200px) {
  .visible-lg-block {
    display: block !important;
  }
}
@media (min-width: 1200px) {
  .visible-lg-inline {
    display: inline !important;
  }
}
@media (min-width: 1200px) {
  .visible-lg-inline-block {
    display: inline-block !important;
  }
}
@media (max-width: 767px) {
  .hidden-xs {
    display: none !important;
  }
}
@media (min-width: 768px) and (max-width: 991px) {
  .hidden-sm {
    display: none !important;
  }
}
@media (min-width: 992px) and (max-width: 1199px) {
  .hidden-md {
    display: none !important;
  }
}
@media (min-width: 1200px) {
  .hidden-lg {
    display: none !important;
  }
}
.visible-print {
  display: none !important;
}
@media print {
  .visible-print {
    display: block !important;
  }
  table.visible-print {
    display: table;
  }
  tr.visible-print {
    display: table-row !important;
  }
  th.visible-print,
  td.visible-print {
    display: table-cell !important;
  }
}
.visible-print-block {
  display: none !important;
}
@media print {
  .visible-print-block {
    display: block !important;
  }
}
.visible-print-inline {
  display: none !important;
}
@media print {
  .visible-print-inline {
    display: inline !important;
  }
}
.visible-print-inline-block {
  display: none !important;
}
@media print {
  .visible-print-inline-block {
    display: inline-block !important;
  }
}
@media print {
  .hidden-print {
    display: none !important;
  }
}
/*# sourceMappingURL=bootstrap.css.map */
"""

Writing ./../Dependencies/css.py


In [3]:
%%file USER
name=Amir Szitenberg
experiment=debug reprophylo
sugar=one
one more thing=I forget, never mind
email=A.Szitenberg@hull.ac.uk
second email=szitenberg@gmail.com

Writing USER


In [7]:
!python reprophylo.py

reprophylo.py:610: UserWarning: Version control off
  warnings.warn('Version control off')


In [1]:
import random
def random_rgb():
    import random
    r = lambda: random.randint(0,255)
    return ('#%02X%02X%02X' % (r(),r(),r()))

In [2]:
print random_rgb()

#39156D


# Tree reconstruction tools in reprophylo
I think that a tool that will simplify the RAxML command line while providing all the possible options will be more appreciated as a starting point than a small range of programs for which you have to write the command line from scratch. In addition, I think that instead of TreeReconstructionMethod object, as I did for alignments, I should make program specific object, ie, RaxmlMethodObject. The reason for this is that the user would usually want to fit the best alignment approach for each gene, but will want to use a uniform tree reconstruction method ( or several of them, but on all the datasets for each method).  
  
I plan to simplify the raxml command line by excluding keywards that are not related to tree reconstruction because other operations do not make sense, coming after sequence alignment in the reprophylo workflow (everything will be accesible, but not everything will be bundled into predesigned analyses)  
The bundles will be either 'tree reconstruction' bundles, 'node support' bundels and 'both' bundles. It will be possible to add optional keywords to the bundes. In the description below I don't show all the keywards that will be included. e.g., -m (model) is always required and I don't write it here.



# RAxML keyword blocks
  
## ML search combined with branch support
*This section include one liners or a two liners that would be meaningless when ran individually*  

#### Single best tree search With rappid bootstrap
-f a -p 12345 -x 12345 -N (number or AUTO_MRE)  
#### Multipe best tree searches With REL bootstrap
-f D -p 12345 -N (number)  
**then**  
-f b -t RAxML_bestTree.name -z RAxML_rellBootstrap.name -m (model)  
#### Fast tree with sh-like support and branch length
-f F -p 12345  
**then**  
-f J -p 12345 -t RAxML_fastTree  
*comment: if not followed by bootstrap analysis, do  use_sh_support_as_branch_support*
#### Very fast tree with sh-like support and branch length
-f E -p 12345  
**then**  
-f J -p 12345 -t RAxML_fastTree  
*comment: if not followed by bootstrap analysis, do  use_sh_support_as_branch_support*  

## Best tree search approaches
*In liners that focus on thorogh best tree search, possibly with more than on randomized parsimony/ completely random starting tree*  
  
#### Best tree default method approach
-f d -p 12345 -N (number)  
#### Slow best tree search with better likelihood
-f o -p 12345 -N (number)  
  
## Bipartition tree calculation approaches (calc supports and put on best tree)
*Methods which focus on branch support calculation and their superimposition on a precalculated best tree or on an nni optimized fast tree with branch-lengths*  
  
#### Rapid bootstrap
-x 12345 -N (number/autoMRE)  
**then**  
-f b -t RAxML_bestTree/RAxML_fastTreeSH_Support -z RAxML_Bootstrap -m (model)  
####  Thorough bootstrap
-b 12345 -N (number/autoMRE)  
**then**  
-f b -t RAxML_bestTree/RAxML_fastTreeSH_Support -z RAxML_Bootstrap -m (model)  
  
####  IC/TC approach
*This approach is based on the level of incongruence between the concatenated tree and the gene trees. I tries replace bootstrap support or posterior probabilities which are inefficient when a lot of data is involved. 
Salichos and Rokas 2013 http://www.ncbi.nlm.nih.gov/pubmed/23657258  
and http://mbe.oxfordjournals.org/content/early/2014/02/07/molbev.msu061.abstract*  
  
*Technically, from the reprophylo point of view, it differs from other approaches by the fact that it integrates all the gene trees and a concatenated tree, while other approches are performed indipendantly on each alignment. So it needs to pull all the trees calculated at one point and placed in the reprophylo project and intgrate them.*

*The analysis is done by calling **-f i** and providing the concat tree to **-t** and the gene trees to **-z**. This will produce the concat tree with the IC and ICA scores as follows **(A,B):0.1325[IC,ICA]**. The scores will then need to be reformated as node features as I have done for the sh-like supports. There are many modifiers to this approach which I have to check carefully in the RAxML manual*  
  
## Parameter optimization appraches
#### optimize br-len and other model parameters
-f e -t (RAxML_fastTreeSH_Support or RAxML_fastTree or RAxML_bestTree)  
*Comment: This will requlire the use of transfer_support_same_topo on the input and output trees*  
  
# General analysis modifiers
*I excluded keywards that don't make sense in the tree reconstruction phase of reprophylo*  
  
A-scondary structure model, together with S  
B-specify threshold for autoMR  
c-specifiy num of CAT categories  
d-start at random instead of parsimony randomized tree  
D-ML search convergence criterion to stop the analysis (ever needed in raxml?)  
e-parameter percision criterion  
fI-ML based descision on midpoint for a tree passed with -t    
ft-another way to search for a tree, not so sure  
fT-optimization via through SPR moves  
F-Just CAT, no GAMMA at all  
g-constraint tree  
H-No so sure  
m-models, will need to be read from a control file   
M-per partition branch length  
n-derived from method.id  
O-disable check of undetermined alignments  
p-random seed for parsimony. set automatically  
P-Used provided AA substitution model  
q-partition file, created automatically  
r-constraint tree, binary  
s-input alignment  
S-secondary structue  
t-tree file  
T-num of cores  
u-use median instead of mean for GAMMA based rates  
z-pass trees file  